In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [5]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [6]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name

-- 2025.4.29
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(61651, 71)

## 1_12. 剔除商家数据

In [7]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(58624, 71)

## 1_13.历史订单渠道名称补充

In [19]:
# # 1148 条历史订单渠道名称补充
# qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')没有这个excel文件
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# # dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys#.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称"]#.fillna(dfzh1.渠道名称_y)
# dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name_cc"])
# dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

# 更新渠道，同时给客群打标签
# 2025.4.29
# 是否巨量、是否号卡
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1["channel_name_cc"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name_cc"])
dfzh1.loc[:,"来源渠道"]=dfzh1["channel_name_cc"].fillna("未知渠道")
dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', dfzh1.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道.isin(conditions_1), 'S量', dfzh1.归属渠道)


# dfzh1[dfzh1.归属渠道 == '顶部搜索框的搜索结果页']，channel_name 为 顶部搜索框的搜索结果页，channel_name_cc 为 空，下单日期为 2023-03 及之前
# A20250113184842199 channel_name为 顶部搜索框的搜索结果页， channel_name_cc 为顶部搜索框的搜索结果页，归属渠道 为抖音

dfzh1['是否免人审'] = np.where((dfzh1.is_vip=='1')&(dfzh1.status_result=='0'), 1 , 0)

# df['是否拒量'] = np.where(df.qvt_risk == '1', 1 , 0)
# df['是否拒量'] = np.where((df.qvt_risk == '0') & (df.qvt_result == '1'), 1 , df.是否拒量)
dfzh1['是否拒量'] = np.where(dfzh1.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
dfzh1['是否号卡'] = np.where(dfzh1.type == 4, 1, 0)

# df_zhys.columns
# df_zhys[['channel_name','channel_name_cc']]
# df_zhys.channel_name.value_counts()
# df_zhys.channel_name_cc.value_counts()
# 芝麻信用 芝麻租物 

# dfzh1[dfzh1.order_number == 'A2023110319165440'][['归属渠道']]

## 1_14.渠道归属

In [20]:
# def qudao_type(a,b,c):
#     a = str(a)
#     b = str(b)
#     if "租物" in b:
#         return "芝麻租物"
#     elif "芝麻" in b:
#         return "芝麻租物"
#     elif "抖音" in b:
#         return "抖音渠道"
#     elif "搜索" in a:
#         return "搜索渠道"
#     elif "租物" in a:
#         return "芝麻租物"
#     elif "芝麻" in a:
#         return "芝麻租物"
#     elif c == 1:
#         return "芝麻租物" 
#     elif "叮咚直播" in a:
#         return "叮咚直播"
#     elif "八派信息" in a:
#         return "八派信息"
#     elif "租瓜直播2号" in a:
#         return "租瓜直播2号"
#     elif "租瓜直播" in a:
#         return "租瓜直播"
#     elif "直播" in a:
#         return "小舞直播"
#     elif "繁星" in a:
#         return "繁星"
#     elif "生活号" in a:
#         return "生活号"
#     elif "社群" in a:
#         return "支付宝社群"
#     elif "付费灯火" in a:
#         return "付费灯火"
#     else :
#         return a

In [21]:
# # df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
# dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
# dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

# dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# # pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [22]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [23]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    57389
二手     1235
Name: count, dtype: int64

In [30]:
'''
Author: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
Date: 2025-06-13 14:10:04
LastEditors: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
LastEditTime: 2025-08-06 14:28:12
FilePath: \Project_Business\风控\1_租后表分析20250429.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "F:\myfile\p站数据\台账数据\维客壹佰2025年台账.xlsx"
# path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/0_参数表/'
dfck = pd.read_excel(f_path_ck,sheet_name="2025")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁_x'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

'\nAuthor: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git\nDate: 2025-06-13 14:10:04\nLastEditors: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git\nLastEditTime: 2025-08-06 14:28:12\nFilePath: \\Project_Business\\风控\x01_租后表分析20250429.ipynb\nDescription: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE\n'

(28563, 2)

是否带锁
未带锁    26579
带锁      1984
Name: count, dtype: int64

(58624, 82)

是否带锁,带锁,未带锁
是否二手,,
二手,383,852
全新,1556,55833


In [29]:
dfzh1.columns.tolist()

['id',
 'order_id',
 'order_number',
 'order_time',
 'search_time',
 'end_date',
 'merchant_id',
 'merchant_name',
 'channel_id',
 'channel_name',
 'model_number',
 'specification',
 'purchase_amount',
 'platform_package',
 'all_deposit',
 'all_rental',
 'liquidated_damages',
 'buyout_balance_payment',
 'advance_periods',
 'advance_sum',
 'order_create_time',
 'rembursement_status',
 'paid_periods',
 'begin_overdue_time',
 'overdue_periods',
 'max_overdue_days',
 'now_overdue_days',
 'total_receivable',
 'contract_price',
 'rent_received',
 'unearned_rent_revenue',
 'paid_buyout_balance_payment',
 'total_received',
 'total_unreceived',
 'unrecovered_cost',
 'overdue_type',
 'gross_profit_receivable',
 'overdue_loss',
 'actual_gross_profit',
 'overdue_total_unreceived',
 'relet_periods',
 'relet_days',
 'relet_rent_paid',
 'activity_id',
 'activity_source',
 'order_label',
 'create_time',
 'update_time',
 'order_finish_date',
 'commodity_premium',
 'overdue_fine',
 'pause_reimbursement'

## 1_17.日期处理

In [ ]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

In [33]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
2季度    20583
3季度    17350
4季度    11213
1季度     9478
Name: count, dtype: int64

## 1_21.获取_买断数据

In [34]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(6899, 7)

## 1_22.租后数据 关联 买断数据

In [35]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(58624, 90)

(58624, 86)

In [36]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(8485)

In [37]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      52025
已买断      6469
部分买断      130
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [38]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(3343, 7)

## 1_32.租后数据关联分期取消数据

In [39]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(58624, 94)

(58624, 91)

## 1_41.按风控策略调整时间节点划分时间段

In [40]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [41]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(765141, 23)

In [42]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(765141, 26)

(765141, 23)

In [43]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [44]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [45]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [46]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [47]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(765362)

(765362, 116)

(765141, 23)

(58624, 94)

## 1_62.去重

In [48]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
# 选取所有重复数据
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
41780     431052   1.0
41792     431052   1.0
674865    431052   1.0
674877    431052   1.0
41781     431052   2.0
...          ...   ...
500757   2040621  23.0
447981   2040621  24.0
448005   2040621  24.0
500734   2040621  24.0
500758   2040621  24.0

[816 rows x 2 columns]


In [49]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(764750, 116)

In [50]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2.规则命中数据整理

# 3.计算逾期天数

## 3_11.异常查询

In [51]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 3_12.实还日期修正

In [52]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [53]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
494411,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
494412,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
494413,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
494414,A2025032020360980,2259696,666.32,666.32,2025-06-23,2025-06-23,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
494415,A2025032020360980,2259696,666.32,666.32,2025-07-23,2025-07-23,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT


In [54]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
494411,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
494412,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
494413,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
494414,A2025032020360980,2259696,666.32,666.32,2025-06-23,2025-06-23,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
494415,A2025032020360980,2259696,666.32,666.32,2025-07-23,2025-07-23,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT


In [55]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60976,2024-07-22


In [56]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [57]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
595089,A20250522124206372,2707156,1082.84,0.0,2026-03-25,NaT,1.0,11.0,已买断,2025-07-01 14:36:57,8,2025-06-29,10.0,2025-06-29
675470,A20250630102202280,3067220,1082.94,0.0,2026-05-03,NaT,1.0,11.0,已买断,2025-07-08 19:58:34,8,2025-07-08,10.0,2025-07-08
684411,A20250703102702367,3103364,1085.46,0.0,2026-06-06,NaT,1.0,12.0,已买断,2025-08-05 16:34:15,8,2025-08-05,11.0,2025-08-05
698048,A20250708114831820,3161546,1073.43,0.0,2026-05-11,NaT,1.0,11.0,已买断,2025-07-28 16:38:42,8,2025-07-28,10.0,2025-07-28
704777,A2025071014014127,3188905,1082.94,0.0,2026-05-13,NaT,1.0,11.0,已买断,2025-07-25 15:05:05,8,2025-07-25,10.0,2025-07-25


In [58]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [59]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [60]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [61]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [62]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [63]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [64]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [65]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 3_13.获取逾期截止时间

In [66]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-08-06
1   2025-08-06
2   2025-08-06
3   2025-08-06
4   2025-08-06
Name: now_date, dtype: datetime64[s]

## 3_14.重新定义状态

In [67]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    142260
已逾期                     77901
未到首期还款日                 59768
续租中                     19965
检测中                        82
Name: count, dtype: int64

订单状态
租赁中     288796
已退款       8364
待归还       6040
待收货       4210
待发货        727
已完成        221
检测中         89
订单取消        84
Name: count, dtype: int64

In [68]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     356059
未到还款日    296962
已逾期       51787
已逾期支付     51357
已取消        8585
Name: count, dtype: int64

In [69]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 3_15.重新定义逾期天数

In [70]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(308561)

Series([], Name: order_number, dtype: object)

In [71]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(764750, 120)

(764750, 120)

In [72]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        73917
已买断        11291
还款中        10639
续租中         5712
已完成         1363
检测中          155
未到首期还款日       54
Name: count, dtype: int64

In [73]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 3_16.逾期天数处理

In [74]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 3_17.稽查

In [75]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 3_18.确认是否到表现期

In [76]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [77]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [78]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [79]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [80]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-08-06,1
1,2022-09-04,2022-08-05,2025-08-06,1
2,2022-10-05,2022-09-05,2025-08-06,1
3,2022-11-04,2022-10-05,2025-08-06,1
4,2022-12-05,2022-11-05,2025-08-06,1


In [81]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 3_19.观察日定义 每月月底时间

In [82]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [83]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 3_20.更新逾期天数

In [84]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(1467, 2)

label
0.0    747218
1.0     17532
Name: count, dtype: int64

(1446, 172)

In [85]:
# #（3）客群-拒量
# # 2025.3.24
# path = 'C:/Users/Administrator/Desktop/mine/3_临时分析需求/2.数据需求/'
# # df_reject_list = pd.read_excel(path + '拒量数据_20250210.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250217.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250220.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250303.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250310.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250317.xlsx')
# # df_reject_list = pd.read_excel(path + '拒量数据_20250407.xlsx')
# df_reject_list = pd.read_excel(path + '拒量数据_20250422.xlsx')

# df_reject_list['是否拒量'] = 1
# df_reject_list.shape
# df_reject_list = df_reject_list.drop_duplicates(subset = 'order_number')
# df_reject_list.shape
# df_reject_list[:1]

# # 更新 df_reject_list
# df_xz_concat.shape
# df_xz_concat = df_xz_concat.merge(df_reject_list[['order_number','是否拒量']], on = 'order_number', how = 'left')
# # df_xz_concat = df_xz_concat[df_xz_concat.是否拒量 != 1]
# df_xz_concat.shape

# 4.非续租订单

<!-- （1）每个mob 观察日 对应的逾期天数
（2）实付日期逻辑：如实付日期大于观察日，实付日期清空
（3）达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期
（4）逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期 -->

## 4_11.数据处理

In [86]:
# 非续租租期数据
df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

(696466, 172)

is_relet
0.0    696466
1.0     68213
2.0        71
Name: count, dtype: int64

In [87]:
# note 20241505 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)
df_mob_total.tail(10)

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,MOB
703683,A2025080519074686,2025-08-05,292.75,0,292.75,mob_12
703684,A20250805191441153,2025-08-05,602.05,0,602.05,mob_12
703685,A20250805191535168,2025-08-05,602.05,0,602.05,mob_12
703686,A20250805192101247,2025-08-05,602.05,0,602.05,mob_12
703687,A20250805192744335,2025-08-05,520.93,0,520.93,mob_12
703688,A20250805193519435,2025-08-05,520.93,0,520.93,mob_12
703689,A20250805195741688,2025-08-05,520.93,0,510.42,mob_12
703690,A202508052001199,2025-08-05,602.05,0,602.05,mob_12
703691,A20250805201637152,2025-08-05,520.93,0,510.42,mob_12
703692,A20250805201648157,2025-08-05,520.93,0,520.93,mob_12


In [88]:
df_fq_concat.shape
df_mob_total.shape
# df_xz_concat[df_xz_concat.order_number == 'A2023120500472233'][['order_number','应付日期','实付日期new','now_date','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023120500472233']

(696466, 173)

(703693, 6)

In [89]:
df_mob_total['overdue_days_1'] = np.where(df_mob_total['overdue_days_mob']>= 1,1,0)
df_mob_total['overdue_days_4'] = np.where(df_mob_total['overdue_days_mob']>= 4,1,0)
df_mob_total['overdue_days_7'] = np.where(df_mob_total['overdue_days_mob']>= 7,1,0)
df_mob_total['overdue_days_15'] = np.where(df_mob_total['overdue_days_mob']>= 15,1,0)
df_mob_total['overdue_days_31'] = np.where(df_mob_total['overdue_days_mob']>= 31,1,0)
df_mob_total['下单月份'] = df_mob_total['下单日期'].dt.strftime('%Y-%m')  

In [90]:
# df_mob_total['采购金额'] 
df_mob_total.shape
df_mob_total = df_mob_total.merge(dfzh3[['order_number','purchase_amount','归属渠道','是否二手','是否拒量', '是否号卡', '是否免人审']],on = 'order_number',how='left')
df_mob_total.shape
df_mob_total[:1]

# note 20241127 707条订单采购金额是0，客户取消
# A2023032216313863
# A202207131230485
# dfzh3[dfzh3.purchase_amount == 0].shape

(703693, 12)

(703901, 18)

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,下单月份,purchase_amount,归属渠道,是否二手,是否拒量,是否号卡,是否免人审
0,A202207050851373,2022-07-05,357.95,0,357.95,mob_0,0,0,0,0,0,2022-07,5150.0,未知渠道,二手,0,0,0


In [91]:
from datetime import datetime as dt    
today_date = dt.now().date()  
date_str = today_date.strftime('%Y-%m-%d')  

# df_mob_total.to_excel(f'E:/myfile/业务相关数据/df_mob_total-{date_str}.xlsx')

## 4_12.每月对应的 用户数及应还金额

In [92]:
# note 20241127 增加条件 (df_fq_concat.purchase_amount > 0) 
# note 20241205 删除条件  (df_fq_concat.purchase_amount > 0)  A202207131230485：purchase_amount 为0，有多种情况，已发货取消，未发货取消，故不删除
# note 20250429 修改条件
# （1）总体
# df_mon2 = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount > 0)&(df_fq_concat.归属渠道!='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon2 = df_fq_concat[(df_fq_concat.当前期数==1)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon2 

# （2）总体非抖音非拒量非号卡
conditions_all_2 = ((df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon2_1 = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_all_2].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon2_1 
 
# （3）租物渠道
conditions_zm = ((df_fq_concat.归属渠道=='芝麻租物')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon_zm = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_zm].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_zm

# （4）搜索渠道
conditions_ss = ((df_fq_concat.归属渠道=='搜索渠道')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon_ss = df_fq_concat[(df_fq_concat.当前期数==1) & conditions_ss].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_ss

# （5）单人会话
conditions_drhh = ((df_fq_concat.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon_drhh = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_drhh].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_drhh

# （6）S量
conditions_sl = ((df_fq_concat.归属渠道=='S量')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon_sl = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_sl].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_sl

# （7）抖音
conditions_dy = (df_fq_concat.归属渠道=='抖音渠道')
df_mon_dy = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_dy].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_dy

# note 20241209增加
# （8）二手非抖音
conditions_esfdy = ((df_fq_concat["是否二手"]=="二手")&(df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.是否拒量 != 1)&(df_fq_concat.是否号卡 != 1))
df_mon_esfdy = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_esfdy].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_esfdy

# （9）拒量
conditions_jl = (df_fq_concat.是否拒量 == 1)
df_mon_jl = df_fq_concat[(df_fq_concat.当前期数==1)&conditions_jl].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_jl

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,22,195693.00,183127.09,125405.00
2022-08,56,527606.04,458082.68,358451.00
2022-09,44,402654.00,351061.75,277982.75
2022-10,41,337968.00,297219.33,249543.09
2022-11,43,440057.00,383153.68,344258.81
2022-12,66,740353.20,660888.72,566276.59
2023-01,221,2173455.52,1912421.71,1533459.00
2023-02,332,3218625.44,2724072.32,2181552.82
2023-03,316,3063400.23,2534569.25,2083021.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,22,195693.00,183127.09,125405.00
2022-08,56,527606.04,458082.68,358451.00
2022-09,44,402654.00,351061.75,277982.75
2022-10,41,337968.00,297219.33,249543.09
2022-11,43,440057.00,383153.68,344258.81
2022-12,66,740353.20,660888.72,566276.59
2023-01,221,2173455.52,1912421.71,1533459.00
2023-02,332,3218625.44,2724072.32,2181552.82
2023-03,316,3063400.23,2534569.25,2083021.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2023-04,201,2039063.00,1468678.64,1355156.00
2023-05,133,1318932.00,960192.81,844546.00
2023-06,247,2519766.00,1827246.68,1607377.00
2023-07,515,5652263.00,4081509.78,3594170.00
2023-08,500,5440598.10,4009938.02,3504454.00
2023-09,550,5906043.10,4501268.18,3988756.12
2023-10,753,8158820.50,6714534.60,5723013.25
2023-11,954,10098851.90,8911153.52,7234215.00
2023-12,469,4849272.90,4220972.80,3352399.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2023-04,195,1980016.84,1686039.92,1305173.00
2023-05,141,1451775.11,1242747.18,925666.00
2023-06,122,1307918.00,1103958.87,830115.00
2023-07,149,1669987.00,1405985.45,1062805.00
2023-08,202,2201518.30,1860750.72,1418637.00
2023-09,119,1368111.50,1146637.91,910558.00
2023-10,102,1250515.60,1073486.26,873717.50
2023-11,194,2225368.80,2079481.60,1579325.00
2023-12,169,1997400.00,1845674.45,1373729.75


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2023-04,1,10399.0,10399.00,6840.00
2023-06,12,119458.0,106151.68,79168.00
2023-07,19,231901.0,191903.85,149045.00
2023-08,15,191700.7,155436.21,126234.00
2023-09,31,384569.9,317102.33,261610.00
2023-10,54,678625.2,568941.22,474985.00
2023-11,143,1751384.6,1512317.63,1226260.00
2023-12,248,3000265.9,2626175.55,2049806.25
2024-01,136,1685533.8,1480965.10,1104144.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2023-04,35,368575.00,311575.63,244612.0
2023-05,31,339310.00,275922.53,208404.0
2023-06,18,208523.00,182736.86,132717.0
2023-07,35,440665.00,424615.36,277310.0
2023-08,46,585661.40,572601.90,378466.0
2023-09,42,529353.30,519609.49,356086.0
2023-10,46,572570.30,501753.68,373112.0
2023-11,81,986962.80,959480.91,710870.0
2023-12,1,12996.70,12996.70,9010.0


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-04,4,26875.0,21921.15,15930.0
2024-05,3,17517.0,17877.71,12690.0
2024-06,49,306191.0,308162.14,220205.0
2024-07,111,642379.0,656738.48,448562.0
2024-08,65,808429.0,660802.15,271022.0
2024-09,8,41644.0,41886.81,30173.0
2024-11,3,23037.0,13360.08,9865.0
2024-12,59,374556.6,367755.55,105399.0
2025-01,51,225493.0,201906.61,95598.0


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,2,12287.00,10470.16,8350.00
2022-08,4,27056.04,27362.84,19130.00
2022-09,9,65456.00,57834.71,48327.68
2022-10,9,83354.00,67603.42,62825.00
2022-11,9,91006.00,69405.89,69285.22
2022-12,3,21029.00,18383.30,13685.00
2023-01,17,143408.00,118972.28,89642.00
2023-02,30,234615.00,193244.76,147754.00
2023-03,44,347452.00,280435.60,226883.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-12,30,333501.78,341495.28,236025.0
2025-01,31,391088.90,392163.37,269323.0
2025-02,251,3182283.81,3090452.35,2132295.0
2025-03,304,3815664.43,3737638.04,2499753.0
2025-04,417,5244799.49,5206557.25,3521285.0
2025-05,368,4620650.60,4535473.06,2961684.0
2025-06,303,3830398.40,3756436.17,2380146.0
2025-07,210,2670236.45,2618491.41,1674599.0
2025-08,21,269051.00,269051.00,149441.0


## 4_13.vintage 4+ 订单 口径

In [93]:
# note 20241127 df_mon_nody 改为 df_mon2
# note 20250429

conditions_all_2 = ((df_mob_total.归属渠道!='抖音渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_zm = ((df_mob_total.归属渠道=='芝麻租物')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_ss = ((df_mob_total.归属渠道=='搜索渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_drhh = ((df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_sl = ((df_mob_total.归属渠道=='S量')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_dy = (df_mob_total.归属渠道=='抖音渠道')
conditions_esfdy = ((df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_jl = (df_mob_total.是否拒量 == 1)

# （1）总体
vintage_df_d4 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4 = vintage_df_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4 = vintage_df_d4.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4['订单量'] = df_mon2["order_number"]
vintage_divide_d4[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
# df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')]
vintage_df_d4 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_all_2],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4 = vintage_df_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4 = vintage_df_d4.divide(df_mon2_1["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4['订单量'] = df_mon2_1["order_number"]
vintage_divide_d4[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）芝麻租物渠道
vintage_df_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_zm],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_zm = vintage_df_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_zm = vintage_df_d4_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d4_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）搜索渠道
vintage_df_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_ss],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_ss = vintage_df_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_ss = vintage_df_d4_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d4_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （5）单人会话
vintage_df_d4_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)&conditions_drhh],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_drhh = vintage_df_d4_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_drhh = vintage_df_d4_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d4_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_df_d4_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)&conditions_sl],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_sl = vintage_df_d4_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_sl = vintage_df_d4_sl.divide(df_mon_sl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_sl['订单量'] = df_mon_sl["order_number"]
vintage_divide_d4_sl[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （7）抖音渠道
vintage_df_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_dy],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_dy = vintage_df_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_dy = vintage_df_d4_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_dy['订单量'] = df_mon_dy["order_number"]
vintage_divide_d4_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （8）二手非抖音
vintage_df_d4_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_esfdy],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_esfdy = vintage_df_d4_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_esfdy = vintage_df_d4_esfdy.divide(df_mon_esfdy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_esfdy['订单量'] = df_mon_esfdy["order_number"]
vintage_divide_d4_esfdy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （9）拒量
vintage_df_d4_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & conditions_jl],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_jl = vintage_df_d4_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_d4_jl = vintage_df_d4_jl.divide(df_mon_jl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_jl['订单量'] = df_mon_jl["order_number"]
vintage_divide_d4_jl[['订单量','mob_1','mob_2','mob_3','mob_4']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,31.82%,40.91%,40.91%,36.36%,36.36%
2022-08,56,14.29%,21.43%,26.79%,28.57%,30.36%,35.71%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,25.00%,29.55%,27.27%,25.00%,27.27%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,21.95%,19.51%,17.07%,21.95%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,13.95%,13.95%,23.26%,23.26%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,3.03%,7.58%,13.64%,19.70%,19.70%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.43%,11.31%,14.48%,15.84%,20.81%,24.89%,26.24%,28.51%,29.86%,31.22%,30.32%,29.41%
2023-02,332,7.53%,11.45%,15.66%,17.17%,20.48%,22.29%,23.19%,25.00%,24.10%,24.40%,23.49%,23.19%
2023-03,316,6.33%,12.03%,15.51%,19.94%,22.78%,25.63%,25.95%,27.53%,28.48%,30.06%,28.48%,27.85%
2023-04,634,3.63%,8.04%,13.09%,15.62%,18.30%,20.98%,22.24%,23.82%,25.08%,25.55%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,31.82%,40.91%,40.91%,36.36%,36.36%
2022-08,56,14.29%,21.43%,26.79%,28.57%,30.36%,35.71%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,25.00%,29.55%,27.27%,25.00%,27.27%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,21.95%,19.51%,17.07%,21.95%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,13.95%,13.95%,23.26%,23.26%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,3.03%,7.58%,13.64%,19.70%,19.70%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.43%,11.31%,14.48%,15.84%,20.81%,24.89%,26.24%,28.51%,29.86%,31.22%,30.32%,29.41%
2023-02,332,7.53%,11.45%,15.66%,17.17%,20.48%,22.29%,23.19%,25.00%,24.10%,24.40%,23.49%,23.19%
2023-03,316,6.33%,12.03%,15.51%,19.94%,22.78%,25.63%,25.95%,27.53%,28.48%,30.06%,28.48%,27.85%
2023-04,634,3.63%,8.04%,13.09%,15.62%,18.30%,20.98%,22.24%,23.82%,25.08%,25.55%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,201,1.99%,5.97%,9.45%,10.45%,11.44%,16.42%,17.41%,18.91%,19.40%,18.41%,18.91%,18.91%
2023-05,133,nan%,3.76%,9.02%,8.27%,11.28%,9.77%,15.79%,18.80%,20.30%,18.80%,21.05%,19.55%
2023-06,247,2.83%,6.48%,11.34%,14.98%,16.60%,19.03%,22.27%,24.70%,24.70%,27.53%,26.72%,26.32%
2023-07,515,2.91%,8.35%,13.59%,15.73%,17.28%,19.61%,20.97%,23.30%,26.02%,27.38%,24.27%,23.30%
2023-08,500,3.20%,8.40%,9.60%,12.60%,16.60%,17.20%,19.00%,21.60%,24.20%,24.00%,22.40%,22.20%
2023-09,550,2.55%,5.09%,7.27%,11.27%,10.91%,12.91%,16.00%,18.91%,21.45%,21.82%,21.09%,20.36%
2023-10,753,3.32%,5.58%,10.36%,13.01%,15.94%,18.59%,21.91%,25.63%,27.36%,29.22%,28.42%,27.89%
2023-11,954,2.20%,6.60%,8.70%,11.01%,14.47%,17.61%,19.81%,21.70%,22.96%,24.32%,24.21%,22.64%
2023-12,469,2.13%,5.33%,8.53%,11.51%,13.86%,17.91%,20.90%,22.17%,24.31%,25.37%,26.44%,25.16%
2024-01,342,1.75%,4.68%,8.48%,11.40%,13.45%,16.67%,19.88%,19.59%,22.22%,24.56%,23.68%,23.10%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,195,4.10%,10.26%,16.41%,18.46%,21.03%,20.51%,24.62%,26.15%,28.21%,28.21%,30.26%,28.72%
2023-05,141,2.84%,14.89%,19.86%,24.11%,30.50%,27.66%,27.66%,28.37%,31.91%,34.04%,35.46%,33.33%
2023-06,122,4.10%,9.84%,21.31%,28.69%,32.79%,36.07%,38.52%,37.70%,40.98%,41.80%,39.34%,38.52%
2023-07,149,5.37%,15.44%,18.79%,20.13%,25.50%,30.87%,31.54%,34.23%,35.57%,34.90%,33.56%,30.87%
2023-08,202,5.45%,10.89%,16.34%,20.30%,25.25%,28.22%,28.22%,35.64%,37.62%,37.62%,36.63%,35.64%
2023-09,119,2.52%,8.40%,10.08%,17.65%,19.33%,22.69%,26.05%,29.41%,31.93%,31.93%,30.25%,30.25%
2023-10,102,0.98%,9.80%,14.71%,17.65%,22.55%,26.47%,30.39%,29.41%,31.37%,29.41%,30.39%,30.39%
2023-11,194,2.06%,9.79%,11.34%,17.53%,22.68%,24.74%,25.77%,29.90%,30.93%,31.96%,32.47%,31.44%
2023-12,169,1.18%,2.37%,10.06%,12.43%,17.16%,18.93%,21.30%,25.44%,26.63%,29.59%,28.40%,27.22%
2024-01,136,1.47%,4.41%,5.88%,11.76%,16.18%,17.65%,19.12%,23.53%,25.00%,24.26%,25.74%,22.79%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,12,nan%,nan%,nan%,8.33%,16.67%,16.67%,16.67%,25.00%,41.67%,41.67%,25.00%,25.00%
2023-07,19,5.26%,5.26%,15.79%,10.53%,21.05%,26.32%,26.32%,21.05%,31.58%,26.32%,26.32%,26.32%
2023-08,15,13.33%,13.33%,13.33%,13.33%,20.00%,20.00%,26.67%,33.33%,33.33%,26.67%,26.67%,26.67%
2023-09,31,9.68%,9.68%,9.68%,16.13%,19.35%,19.35%,22.58%,29.03%,32.26%,35.48%,35.48%,35.48%
2023-10,54,5.56%,9.26%,20.37%,22.22%,25.93%,29.63%,27.78%,37.04%,33.33%,31.48%,27.78%,27.78%
2023-11,143,4.20%,14.69%,18.18%,23.78%,25.17%,30.07%,31.47%,36.36%,38.46%,39.16%,38.46%,37.76%
2023-12,248,2.42%,6.45%,10.48%,14.92%,21.37%,23.39%,25.00%,30.24%,31.85%,31.45%,30.24%,29.84%
2024-01,136,2.94%,7.35%,11.03%,13.97%,16.91%,23.53%,22.79%,26.47%,27.21%,25.00%,25.00%,24.26%
2024-06,5,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,35,2.86%,14.29%,22.86%,25.71%,25.71%,31.43%,31.43%,34.29%,28.57%,25.71%,28.57%,28.57%
2023-05,31,3.23%,6.45%,6.45%,12.90%,6.45%,9.68%,12.90%,12.90%,19.35%,19.35%,19.35%,19.35%
2023-06,18,nan%,nan%,5.56%,5.56%,5.56%,5.56%,5.56%,11.11%,5.56%,11.11%,11.11%,11.11%
2023-07,35,11.43%,20.00%,37.14%,45.71%,40.00%,45.71%,54.29%,54.29%,57.14%,60.00%,54.29%,51.43%
2023-08,46,21.74%,32.61%,39.13%,50.00%,50.00%,52.17%,56.52%,58.70%,65.22%,63.04%,56.52%,54.35%
2023-09,42,9.52%,19.05%,33.33%,38.10%,52.38%,50.00%,50.00%,64.29%,59.52%,57.14%,54.76%,52.38%
2023-10,46,8.70%,23.91%,36.96%,45.65%,47.83%,43.48%,47.83%,47.83%,52.17%,47.83%,47.83%,47.83%
2023-11,81,3.70%,18.52%,23.46%,27.16%,32.10%,34.57%,33.33%,35.80%,35.80%,38.27%,39.51%,34.57%
2023-12,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2024-01,1,nan%,nan%,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,20.41%,26.53%,26.53%,38.78%,40.82%,44.90%,48.98%,48.98%,46.94%,46.94%,51.02%,51.02%
2024-07,111,16.22%,25.23%,34.23%,39.64%,39.64%,43.24%,45.95%,49.55%,50.45%,53.15%,55.86%,55.86%
2024-08,65,29.23%,40.00%,40.00%,30.77%,27.69%,32.31%,32.31%,40.00%,40.00%,40.00%,40.00%,nan%
2024-09,8,25.00%,37.50%,50.00%,62.50%,75.00%,62.50%,75.00%,50.00%,75.00%,75.00%,nan%,nan%
2024-11,3,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,nan%,nan%,nan%,nan%
2024-12,59,15.25%,16.95%,27.12%,28.81%,32.20%,32.20%,30.51%,nan%,nan%,nan%,nan%,nan%
2025-01,51,9.80%,13.73%,19.61%,21.57%,23.53%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,58,8.62%,8.62%,24.14%,22.41%,31.03%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,44.44%,44.44%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,22.22%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,33.33%,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,17.65%,17.65%,17.65%,23.53%,29.41%,23.53%,29.41%,23.53%
2023-02,30,13.33%,13.33%,16.67%,23.33%,16.67%,26.67%,30.00%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,4.55%,11.36%,15.91%,20.45%,25.00%,31.82%,29.55%,31.82%,31.82%,31.82%,31.82%,29.55%
2023-04,74,1.35%,4.05%,6.76%,12.16%,12.16%,12.16%,13.51%,9.46%,10.81%,12.16%,13.51%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4
2024-12,30,nan%,3.33%,6.67%,10.00%
2025-01,31,6.45%,nan%,19.35%,22.58%
2025-02,251,7.57%,15.14%,22.71%,24.70%
2025-03,304,6.25%,17.76%,22.04%,28.29%
2025-04,417,9.35%,20.86%,26.62%,nan%
2025-05,368,8.97%,19.29%,nan%,nan%
2025-06,303,7.92%,nan%,nan%,nan%
2025-07,210,nan%,nan%,nan%,nan%
2025-08,21,nan%,nan%,nan%,nan%


## 4_13.vintage 7+ 订单 口径

In [94]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# note 20241127 df_mon_nody 改为 df_mon2
# note 20250429

conditions_all_2 = ((df_mob_total.归属渠道!='抖音渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_zm = ((df_mob_total.归属渠道=='芝麻租物')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_ss = ((df_mob_total.归属渠道=='搜索渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_drhh = ((df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_sl = ((df_mob_total.归属渠道=='S量')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_dy = (df_mob_total.归属渠道=='抖音渠道')
conditions_esfdy = ((df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')&(df_mob_total.是否拒量 != 1)&(df_mob_total.是否号卡 != 1))
conditions_jl = (df_mob_total.是否拒量 == 1)

# （1）总体
vintage_df_d7 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7 = vintage_df_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7 = vintage_df_d7.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7['订单量'] = df_mon2["order_number"]
vintage_divide_d7[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
# df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='抖音渠道')]
vintage_df_d7 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_all_2],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7 = vintage_df_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7 = vintage_df_d7.divide(df_mon2_1["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7['订单量'] = df_mon2_1["order_number"]
vintage_divide_d7[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）芝麻租物渠道
vintage_df_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_zm],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_zm = vintage_df_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_zm = vintage_df_d7_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d7_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）搜索渠道
vintage_df_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_ss],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_ss = vintage_df_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_ss = vintage_df_d7_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d7_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （5）单人会话
vintage_df_d7_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)&conditions_drhh],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_drhh = vintage_df_d7_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_drhh = vintage_df_d7_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d7_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_df_d7_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)&conditions_sl],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_sl = vintage_df_d7_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_sl = vintage_df_d7_sl.divide(df_mon_sl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_sl['订单量'] = df_mon_sl["order_number"]
vintage_divide_d7_sl[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （7）抖音渠道
vintage_df_d7_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_dy],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_dy = vintage_df_d7_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_dy = vintage_df_d7_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_dy['订单量'] = df_mon_dy["order_number"]
vintage_divide_d7_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （8）二手非抖音
vintage_df_d7_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_esfdy],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_esfdy = vintage_df_d7_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_esfdy = vintage_df_d7_esfdy.divide(df_mon_esfdy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_esfdy['订单量'] = df_mon_esfdy["order_number"]
vintage_divide_d7_esfdy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （9）拒量
vintage_df_d7_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & conditions_jl],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_jl = vintage_df_d7_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_d7_jl = vintage_df_d7_jl.divide(df_mon_jl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_jl['订单量'] = df_mon_jl["order_number"]
vintage_divide_d7_jl[['订单量','mob_1','mob_2','mob_3','mob_4']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,4.55%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,31.82%,40.91%,40.91%,36.36%,36.36%
2022-08,56,10.71%,21.43%,26.79%,28.57%,30.36%,33.93%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,20.45%,29.55%,27.27%,25.00%,25.00%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,21.95%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,11.63%,11.63%,20.93%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,1.52%,7.58%,12.12%,19.70%,18.18%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.98%,10.86%,14.48%,15.84%,19.00%,24.43%,24.43%,27.60%,29.86%,30.77%,30.32%,29.41%
2023-02,332,6.93%,10.54%,14.76%,16.87%,19.88%,21.39%,22.89%,24.70%,23.80%,24.10%,23.49%,23.19%
2023-03,316,4.43%,10.76%,13.92%,18.99%,21.84%,23.73%,24.68%,25.63%,28.48%,30.06%,28.48%,27.85%
2023-04,634,3.31%,7.26%,11.99%,14.98%,17.67%,20.50%,21.61%,23.34%,24.61%,25.24%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,4.55%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,31.82%,40.91%,40.91%,36.36%,36.36%
2022-08,56,10.71%,21.43%,26.79%,28.57%,30.36%,33.93%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,20.45%,29.55%,27.27%,25.00%,25.00%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,21.95%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,11.63%,11.63%,20.93%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,1.52%,7.58%,12.12%,19.70%,18.18%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.98%,10.86%,14.48%,15.84%,19.00%,24.43%,24.43%,27.60%,29.86%,30.77%,30.32%,29.41%
2023-02,332,6.93%,10.54%,14.76%,16.87%,19.88%,21.39%,22.89%,24.70%,23.80%,24.10%,23.49%,23.19%
2023-03,316,4.43%,10.76%,13.92%,18.99%,21.84%,23.73%,24.68%,25.63%,28.48%,30.06%,28.48%,27.85%
2023-04,634,3.31%,7.26%,11.99%,14.98%,17.67%,20.50%,21.61%,23.34%,24.61%,25.24%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,201,1.99%,5.47%,8.96%,10.45%,11.44%,16.42%,16.92%,18.91%,19.40%,18.41%,18.91%,18.91%
2023-05,133,nan%,3.76%,8.27%,8.27%,9.77%,9.77%,15.79%,18.05%,20.30%,17.29%,20.30%,19.55%
2023-06,247,2.02%,5.26%,10.53%,13.36%,15.79%,18.22%,22.27%,23.08%,24.70%,27.13%,26.32%,26.32%
2023-07,515,2.52%,6.99%,12.23%,14.76%,16.12%,18.25%,20.00%,22.52%,25.24%,26.41%,24.27%,23.30%
2023-08,500,2.80%,7.60%,9.20%,12.00%,16.40%,17.20%,18.60%,21.40%,23.60%,23.60%,22.40%,22.20%
2023-09,550,2.18%,4.36%,6.73%,9.64%,10.73%,12.00%,15.45%,18.55%,20.36%,21.45%,21.09%,20.36%
2023-10,753,3.19%,4.65%,9.96%,12.88%,15.67%,18.19%,21.65%,25.10%,27.22%,28.69%,28.29%,27.89%
2023-11,954,1.78%,6.08%,8.39%,10.27%,13.63%,16.67%,19.18%,21.17%,22.64%,23.79%,24.21%,22.64%
2023-12,469,1.92%,4.48%,7.89%,11.09%,13.22%,16.42%,20.47%,21.75%,24.31%,25.16%,26.01%,25.16%
2024-01,342,1.75%,4.39%,8.19%,9.94%,12.57%,15.50%,19.01%,19.01%,21.35%,23.10%,23.39%,23.10%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,195,3.08%,8.21%,15.38%,17.44%,19.49%,19.49%,23.08%,25.64%,28.21%,27.69%,30.26%,28.72%
2023-05,141,2.84%,14.18%,18.44%,22.70%,29.79%,26.95%,26.95%,27.66%,31.21%,33.33%,35.46%,33.33%
2023-06,122,4.10%,9.84%,18.85%,27.87%,31.15%,35.25%,37.70%,36.89%,40.98%,40.98%,39.34%,38.52%
2023-07,149,4.70%,14.09%,16.78%,20.13%,24.16%,30.87%,30.20%,31.54%,35.57%,34.23%,32.89%,30.87%
2023-08,202,4.46%,10.40%,15.35%,19.80%,24.75%,27.23%,27.72%,34.65%,37.62%,36.63%,36.63%,35.64%
2023-09,119,2.52%,8.40%,10.08%,17.65%,17.65%,22.69%,25.21%,29.41%,31.93%,31.93%,30.25%,30.25%
2023-10,102,0.98%,9.80%,13.73%,16.67%,21.57%,24.51%,30.39%,29.41%,31.37%,29.41%,30.39%,30.39%
2023-11,194,1.55%,7.73%,9.28%,14.43%,21.13%,22.68%,24.74%,27.84%,29.90%,30.93%,30.93%,31.44%
2023-12,169,0.59%,2.37%,8.88%,12.43%,16.57%,18.34%,20.12%,23.67%,24.26%,28.40%,27.81%,27.22%
2024-01,136,1.47%,4.41%,5.88%,11.76%,16.18%,16.91%,19.12%,23.53%,24.26%,23.53%,25.74%,22.79%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,12,nan%,nan%,nan%,8.33%,16.67%,16.67%,16.67%,25.00%,41.67%,41.67%,25.00%,25.00%
2023-07,19,nan%,5.26%,10.53%,10.53%,21.05%,21.05%,26.32%,21.05%,31.58%,26.32%,26.32%,26.32%
2023-08,15,13.33%,13.33%,13.33%,13.33%,20.00%,20.00%,26.67%,33.33%,33.33%,26.67%,26.67%,26.67%
2023-09,31,9.68%,9.68%,9.68%,16.13%,19.35%,16.13%,19.35%,22.58%,32.26%,35.48%,35.48%,35.48%
2023-10,54,3.70%,7.41%,18.52%,22.22%,24.07%,27.78%,27.78%,35.19%,31.48%,31.48%,27.78%,27.78%
2023-11,143,3.50%,12.59%,16.78%,22.38%,24.48%,30.07%,31.47%,34.27%,37.76%,38.46%,38.46%,37.76%
2023-12,248,2.42%,4.84%,9.27%,14.11%,20.16%,21.37%,24.60%,29.84%,30.24%,31.05%,30.24%,29.84%
2024-01,136,2.94%,7.35%,11.03%,13.97%,16.18%,23.53%,22.79%,25.74%,27.21%,24.26%,25.00%,24.26%
2024-06,5,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,35,2.86%,14.29%,20.00%,22.86%,22.86%,28.57%,31.43%,28.57%,22.86%,22.86%,28.57%,28.57%
2023-05,31,3.23%,6.45%,6.45%,12.90%,6.45%,9.68%,12.90%,12.90%,19.35%,19.35%,19.35%,19.35%
2023-06,18,nan%,nan%,5.56%,5.56%,5.56%,5.56%,5.56%,11.11%,5.56%,11.11%,11.11%,11.11%
2023-07,35,5.71%,20.00%,31.43%,45.71%,40.00%,45.71%,54.29%,54.29%,57.14%,60.00%,54.29%,51.43%
2023-08,46,19.57%,32.61%,36.96%,50.00%,47.83%,50.00%,56.52%,56.52%,65.22%,63.04%,56.52%,54.35%
2023-09,42,9.52%,19.05%,30.95%,38.10%,50.00%,50.00%,50.00%,64.29%,59.52%,57.14%,54.76%,52.38%
2023-10,46,8.70%,21.74%,34.78%,45.65%,45.65%,43.48%,47.83%,47.83%,47.83%,47.83%,47.83%,47.83%
2023-11,81,2.47%,18.52%,20.99%,27.16%,30.86%,32.10%,33.33%,34.57%,34.57%,38.27%,38.27%,34.57%
2023-12,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2024-01,1,nan%,nan%,nan%,100.00%,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,14.29%,26.53%,24.49%,36.73%,40.82%,42.86%,46.94%,48.98%,44.90%,46.94%,51.02%,51.02%
2024-07,111,13.51%,25.23%,34.23%,39.64%,39.64%,43.24%,45.95%,49.55%,50.45%,53.15%,55.86%,55.86%
2024-08,65,27.69%,38.46%,36.92%,29.23%,27.69%,30.77%,32.31%,38.46%,40.00%,40.00%,40.00%,nan%
2024-09,8,25.00%,37.50%,50.00%,62.50%,75.00%,62.50%,75.00%,50.00%,75.00%,75.00%,nan%,nan%
2024-11,3,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,nan%,nan%,nan%,nan%
2024-12,59,11.86%,16.95%,27.12%,28.81%,32.20%,32.20%,30.51%,nan%,nan%,nan%,nan%,nan%
2025-01,51,1.96%,13.73%,17.65%,21.57%,21.57%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,58,6.90%,6.90%,20.69%,18.97%,27.59%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,33.33%,44.44%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,11.11%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,33.33%,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,11.76%,17.65%,11.76%,17.65%,29.41%,23.53%,29.41%,23.53%
2023-02,30,10.00%,10.00%,13.33%,20.00%,16.67%,23.33%,26.67%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,2.27%,11.36%,15.91%,20.45%,25.00%,31.82%,29.55%,29.55%,31.82%,31.82%,31.82%,29.55%
2023-04,74,1.35%,4.05%,5.41%,12.16%,12.16%,12.16%,13.51%,9.46%,10.81%,12.16%,13.51%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4
2024-12,30,nan%,3.33%,6.67%,10.00%
2025-01,31,nan%,nan%,19.35%,22.58%
2025-02,251,5.98%,13.55%,21.12%,23.11%
2025-03,304,4.61%,16.78%,21.05%,26.64%
2025-04,417,6.47%,17.51%,24.46%,nan%
2025-05,368,8.15%,18.48%,nan%,nan%
2025-06,303,6.93%,nan%,nan%,nan%
2025-07,210,nan%,nan%,nan%,nan%
2025-08,21,nan%,nan%,nan%,nan%


## 4_14.vintage 15+ 订单 口径

In [95]:

# note 20241127 df_mon 改为 df_mon2，分母保持一致


# （1）总体
vintage_df_d15 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15 = vintage_df_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15 = vintage_df_d15.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15['订单量'] = df_mon2["order_number"]
vintage_divide_d15[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
# df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='抖音渠道')]
vintage_df_d15 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_all_2],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15 = vintage_df_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15 = vintage_df_d15.divide(df_mon2_1["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15['订单量'] = df_mon2_1["order_number"]
vintage_divide_d15[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）芝麻租物渠道
vintage_df_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_zm],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_zm = vintage_df_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_zm = vintage_df_d15_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d15_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）搜索渠道
vintage_df_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_ss],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_ss = vintage_df_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_ss = vintage_df_d15_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d15_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （5）单人会话
vintage_df_d15_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)&conditions_drhh],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_drhh = vintage_df_d15_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_drhh = vintage_df_d15_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d15_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_df_d15_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)&conditions_sl],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_sl = vintage_df_d15_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_sl = vintage_df_d15_sl.divide(df_mon_sl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_sl['订单量'] = df_mon_sl["order_number"]
vintage_divide_d15_sl[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （7）抖音渠道
vintage_df_d15_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_dy],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_dy = vintage_df_d15_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_dy = vintage_df_d15_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_dy['订单量'] = df_mon_dy["order_number"]
vintage_divide_d15_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （8）二手非抖音
vintage_df_d15_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_esfdy],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_esfdy = vintage_df_d15_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_esfdy = vintage_df_d15_esfdy.divide(df_mon_esfdy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_esfdy['订单量'] = df_mon_esfdy["order_number"]
vintage_divide_d15_esfdy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （9）拒量
vintage_df_d15_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & conditions_jl],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_jl = vintage_df_d15_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_d15_jl = vintage_df_d15_jl.divide(df_mon_jl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_jl['订单量'] = df_mon_jl["order_number"]
vintage_divide_d15_jl[['订单量','mob_1','mob_2','mob_3','mob_4']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,nan%,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,31.82%,31.82%,36.36%,36.36%,36.36%
2022-08,56,5.36%,14.29%,19.64%,23.21%,28.57%,33.93%,37.50%,39.29%,39.29%,41.07%,41.07%,39.29%
2022-09,44,2.27%,9.09%,18.18%,18.18%,27.27%,27.27%,25.00%,25.00%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,19.51%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,4.65%,11.63%,9.30%,11.63%,11.63%,18.60%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,nan%,7.58%,7.58%,19.70%,16.67%,16.67%,22.73%,21.21%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.07%,9.95%,11.76%,15.38%,18.55%,23.98%,23.98%,26.70%,29.41%,28.96%,30.32%,29.41%
2023-02,332,4.22%,8.73%,13.55%,14.16%,18.98%,19.88%,21.08%,23.80%,23.80%,23.80%,23.49%,23.19%
2023-03,316,2.85%,8.54%,11.71%,16.46%,17.72%,22.15%,22.78%,23.73%,27.22%,29.11%,28.48%,27.85%
2023-04,634,2.21%,5.68%,10.09%,13.25%,15.62%,19.09%,20.50%,21.77%,23.66%,23.19%,25.39%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,nan%,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,31.82%,31.82%,36.36%,36.36%,36.36%
2022-08,56,5.36%,14.29%,19.64%,23.21%,28.57%,33.93%,37.50%,39.29%,39.29%,41.07%,41.07%,39.29%
2022-09,44,2.27%,9.09%,18.18%,18.18%,27.27%,27.27%,25.00%,25.00%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,19.51%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,4.65%,11.63%,9.30%,11.63%,11.63%,18.60%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,nan%,7.58%,7.58%,19.70%,16.67%,16.67%,22.73%,21.21%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.07%,9.95%,11.76%,15.38%,18.55%,23.98%,23.98%,26.70%,29.41%,28.96%,30.32%,29.41%
2023-02,332,4.22%,8.73%,13.55%,14.16%,18.98%,19.88%,21.08%,23.80%,23.80%,23.80%,23.49%,23.19%
2023-03,316,2.85%,8.54%,11.71%,16.46%,17.72%,22.15%,22.78%,23.73%,27.22%,29.11%,28.48%,27.85%
2023-04,634,2.21%,5.68%,10.09%,13.25%,15.62%,19.09%,20.50%,21.77%,23.66%,23.19%,25.39%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,201,1.00%,2.99%,6.97%,8.96%,10.45%,14.43%,15.92%,16.92%,19.40%,17.91%,18.91%,18.91%
2023-05,133,nan%,2.26%,8.27%,8.27%,9.77%,9.02%,15.79%,16.54%,18.05%,16.54%,20.30%,19.55%
2023-06,247,1.21%,4.86%,8.91%,12.55%,15.38%,17.41%,21.46%,22.27%,23.89%,25.91%,26.32%,26.32%
2023-07,515,0.78%,4.47%,10.10%,13.01%,14.56%,16.31%,18.83%,21.36%,23.50%,25.44%,24.27%,23.30%
2023-08,500,1.60%,5.00%,7.80%,10.20%,14.40%,15.40%,17.40%,20.00%,22.20%,22.80%,22.40%,22.20%
2023-09,550,1.27%,3.82%,6.18%,8.91%,9.45%,11.09%,14.18%,17.09%,18.91%,20.18%,20.73%,20.36%
2023-10,753,1.73%,3.72%,8.63%,11.69%,13.81%,16.33%,19.79%,23.37%,26.43%,27.09%,27.62%,27.89%
2023-11,954,0.73%,4.72%,7.02%,9.54%,12.79%,15.41%,18.13%,19.71%,21.91%,22.43%,23.69%,22.64%
2023-12,469,1.28%,3.84%,6.61%,9.38%,11.94%,14.29%,18.76%,20.26%,22.39%,23.67%,24.95%,25.16%
2024-01,342,0.88%,3.80%,6.73%,9.36%,11.70%,15.20%,17.54%,18.71%,20.18%,21.64%,23.39%,23.10%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,195,2.05%,7.18%,11.79%,14.36%,15.90%,18.46%,21.54%,23.08%,26.15%,23.59%,28.72%,28.72%
2023-05,141,2.13%,12.06%,14.89%,20.57%,26.95%,25.53%,26.24%,27.66%,29.79%,30.50%,34.75%,33.33%
2023-06,122,1.64%,7.38%,14.75%,23.77%,28.69%,31.97%,36.07%,36.89%,37.70%,39.34%,39.34%,38.52%
2023-07,149,2.68%,10.07%,15.44%,16.11%,22.15%,27.52%,28.86%,30.87%,33.56%,34.23%,32.21%,30.87%
2023-08,202,2.48%,10.40%,14.36%,18.32%,22.77%,24.75%,26.73%,31.68%,36.14%,36.63%,36.63%,35.64%
2023-09,119,1.68%,7.56%,10.08%,13.45%,15.97%,20.17%,24.37%,27.73%,31.09%,31.09%,30.25%,30.25%
2023-10,102,nan%,8.82%,9.80%,14.71%,19.61%,22.55%,29.41%,29.41%,31.37%,29.41%,29.41%,30.39%
2023-11,194,0.52%,4.64%,6.70%,11.86%,18.56%,21.65%,23.71%,24.23%,27.32%,30.41%,30.93%,31.44%
2023-12,169,nan%,1.78%,7.10%,11.83%,13.61%,17.75%,19.53%,22.49%,24.26%,27.22%,27.81%,27.22%
2024-01,136,0.74%,2.94%,4.41%,8.09%,13.97%,16.18%,19.12%,21.32%,23.53%,23.53%,25.74%,22.79%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,12,nan%,nan%,nan%,8.33%,16.67%,16.67%,16.67%,25.00%,41.67%,41.67%,25.00%,25.00%
2023-07,19,nan%,5.26%,5.26%,10.53%,21.05%,21.05%,26.32%,21.05%,31.58%,26.32%,26.32%,26.32%
2023-08,15,nan%,6.67%,13.33%,13.33%,20.00%,20.00%,26.67%,33.33%,33.33%,26.67%,26.67%,26.67%
2023-09,31,3.23%,9.68%,6.45%,9.68%,19.35%,12.90%,19.35%,22.58%,29.03%,35.48%,35.48%,35.48%
2023-10,54,nan%,3.70%,11.11%,18.52%,22.22%,27.78%,27.78%,31.48%,27.78%,27.78%,27.78%,27.78%
2023-11,143,1.40%,9.09%,16.08%,18.18%,23.78%,27.27%,29.37%,32.87%,36.36%,37.06%,38.46%,37.76%
2023-12,248,1.21%,4.44%,6.85%,11.69%,17.74%,19.76%,22.58%,27.42%,28.23%,31.05%,30.24%,29.84%
2024-01,136,2.21%,7.35%,8.09%,13.97%,14.71%,21.32%,22.79%,22.79%,25.74%,24.26%,25.00%,24.26%
2024-06,5,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,35,2.86%,11.43%,17.14%,22.86%,22.86%,25.71%,31.43%,28.57%,22.86%,22.86%,28.57%,28.57%
2023-05,31,3.23%,6.45%,6.45%,12.90%,6.45%,9.68%,12.90%,12.90%,19.35%,19.35%,19.35%,19.35%
2023-06,18,nan%,nan%,5.56%,5.56%,5.56%,5.56%,5.56%,11.11%,5.56%,11.11%,11.11%,11.11%
2023-07,35,nan%,17.14%,31.43%,42.86%,40.00%,45.71%,48.57%,51.43%,51.43%,60.00%,54.29%,51.43%
2023-08,46,15.22%,23.91%,36.96%,47.83%,47.83%,50.00%,52.17%,56.52%,58.70%,63.04%,56.52%,54.35%
2023-09,42,7.14%,16.67%,28.57%,33.33%,38.10%,50.00%,50.00%,54.76%,54.76%,54.76%,54.76%,52.38%
2023-10,46,6.52%,19.57%,34.78%,41.30%,45.65%,41.30%,47.83%,47.83%,47.83%,47.83%,47.83%,47.83%
2023-11,81,2.47%,11.11%,19.75%,23.46%,30.86%,30.86%,33.33%,33.33%,34.57%,37.04%,38.27%,34.57%
2023-12,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2024-01,1,nan%,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,4,nan%,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,2.04%,22.45%,24.49%,30.61%,36.73%,40.82%,44.90%,46.94%,44.90%,46.94%,51.02%,51.02%
2024-07,111,9.01%,25.23%,34.23%,36.94%,39.64%,42.34%,45.05%,47.75%,48.65%,49.55%,55.86%,55.86%
2024-08,65,18.46%,36.92%,36.92%,26.15%,26.15%,30.77%,29.23%,36.92%,38.46%,38.46%,40.00%,nan%
2024-09,8,12.50%,25.00%,37.50%,37.50%,62.50%,50.00%,62.50%,50.00%,62.50%,62.50%,nan%,nan%
2024-11,3,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,nan%,nan%,nan%,nan%
2024-12,59,6.78%,15.25%,23.73%,27.12%,32.20%,32.20%,30.51%,nan%,nan%,nan%,nan%,nan%
2025-01,51,nan%,9.80%,7.84%,15.69%,19.61%,29.41%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,58,nan%,5.17%,13.79%,18.97%,24.14%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,33.33%,33.33%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,11.11%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,11.76%,17.65%,11.76%,17.65%,29.41%,23.53%,29.41%,23.53%
2023-02,30,3.33%,10.00%,13.33%,16.67%,16.67%,20.00%,23.33%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,nan%,6.82%,9.09%,18.18%,20.45%,27.27%,27.27%,25.00%,29.55%,31.82%,31.82%,29.55%
2023-04,74,1.35%,1.35%,4.05%,10.81%,9.46%,10.81%,10.81%,9.46%,9.46%,8.11%,12.16%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4
2024-12,30,nan%,3.33%,3.33%,3.33%
2025-01,31,nan%,nan%,nan%,9.68%
2025-02,251,1.99%,9.96%,17.53%,21.51%
2025-03,304,2.96%,13.49%,18.75%,24.67%
2025-04,417,2.40%,13.67%,22.06%,nan%
2025-05,368,5.71%,15.49%,nan%,nan%
2025-06,303,3.30%,nan%,nan%,nan%
2025-07,210,nan%,nan%,nan%,nan%
2025-08,21,nan%,nan%,nan%,nan%


## 4_15.vintage 31+ 订单 口径

In [96]:

# note 20241127 df_mon 改为 df_mon2，分母保持一致

# （1）总体
vintage_df_d31 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31 = vintage_df_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31 = vintage_df_d31.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31['订单量'] = df_mon2["order_number"]
vintage_divide_d31[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
# df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='抖音渠道')]
vintage_df_d31 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_all_2],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31 = vintage_df_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31 = vintage_df_d31.divide(df_mon2_1["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31['订单量'] = df_mon2_1["order_number"]
vintage_divide_d31[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）芝麻租物渠道
vintage_df_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_zm],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_zm = vintage_df_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_zm = vintage_df_d31_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d31_zm[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）搜索渠道
vintage_df_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_ss],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_ss = vintage_df_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d31_ss = vintage_df_d31_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d31_ss[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （5）单人会话
vintage_df_d31_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)&conditions_drhh],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_drhh = vintage_df_d31_drhh[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_drhh = vintage_df_d31_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d31_drhh[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_df_d31_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)&conditions_sl],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_sl = vintage_df_d31_sl[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_sl = vintage_df_d31_sl.divide(df_mon_sl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_sl['订单量'] = df_mon_sl["order_number"]
vintage_divide_d31_sl[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （7）抖音渠道
vintage_df_d31_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_dy],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_dy = vintage_df_d31_dy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_dy = vintage_df_d31_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_dy['订单量'] = df_mon_dy["order_number"]
vintage_divide_d31_dy[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （8）二手非抖音
vintage_df_d31_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_esfdy],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_esfdy = vintage_df_d31_esfdy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_esfdy = vintage_df_d31_esfdy.divide(df_mon_esfdy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_esfdy['订单量'] = df_mon_esfdy["order_number"]
vintage_divide_d31_esfdy[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （9）拒量
# A2024122615104994
vintage_df_d31_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & conditions_jl],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_jl = vintage_df_d31_jl[['mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_d31_jl = vintage_df_d31_jl.divide(df_mon_jl["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_jl['订单量'] = df_mon_jl["order_number"]
# vintage_divide_d31_jl[['订单量','mob_2','mob_3']]
vintage_divide_d31_jl[['订单量','mob_2','mob_3','mob_4']]

# df_fq_concat[df_fq_concat.order_number == 'A2024122615104994'][['当前期数','应付日期','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2024122615104994'][['overdue_days_mob','MOB']]

MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,31.82%,31.82%,36.36%,36.36%,36.36%
2022-08,56,12.50%,17.86%,21.43%,26.79%,28.57%,32.14%,37.50%,35.71%,41.07%,39.29%,39.29%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,6.06%,15.15%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,12.22%,14.48%,20.36%,20.81%,23.08%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.02%,9.64%,12.65%,16.87%,16.87%,19.58%,21.69%,22.59%,22.89%,23.19%,23.19%
2023-03,316,6.33%,10.13%,14.24%,17.41%,20.25%,20.89%,22.78%,26.58%,27.85%,27.22%,27.85%
2023-04,634,3.15%,6.31%,10.25%,12.78%,15.77%,18.30%,19.72%,22.24%,22.08%,23.97%,25.24%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,31.82%,31.82%,36.36%,36.36%,36.36%
2022-08,56,12.50%,17.86%,21.43%,26.79%,28.57%,32.14%,37.50%,35.71%,41.07%,39.29%,39.29%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,6.06%,15.15%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,12.22%,14.48%,20.36%,20.81%,23.08%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.02%,9.64%,12.65%,16.87%,16.87%,19.58%,21.69%,22.59%,22.89%,23.19%,23.19%
2023-03,316,6.33%,10.13%,14.24%,17.41%,20.25%,20.89%,22.78%,26.58%,27.85%,27.22%,27.85%
2023-04,634,3.15%,6.31%,10.25%,12.78%,15.77%,18.30%,19.72%,22.24%,22.08%,23.97%,25.24%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,201,1.49%,4.48%,7.46%,8.96%,11.44%,13.43%,15.42%,18.41%,17.91%,18.41%,18.91%
2023-05,133,nan%,3.01%,6.77%,7.52%,8.27%,12.03%,13.53%,16.54%,16.54%,18.05%,19.55%
2023-06,247,2.43%,5.67%,10.12%,12.55%,15.38%,18.22%,20.65%,22.27%,24.29%,25.91%,26.32%
2023-07,515,3.11%,7.38%,11.65%,14.17%,14.95%,17.28%,20.00%,22.14%,24.27%,24.08%,23.30%
2023-08,500,3.20%,6.00%,8.20%,11.40%,13.80%,15.80%,18.40%,20.20%,22.40%,22.20%,22.20%
2023-09,550,2.18%,4.36%,5.64%,8.73%,10.00%,12.00%,15.09%,17.45%,19.09%,20.55%,20.00%
2023-10,753,2.26%,5.98%,9.56%,11.69%,14.08%,17.66%,20.85%,24.83%,25.76%,26.69%,27.49%
2023-11,954,1.89%,5.56%,7.65%,9.96%,13.63%,16.35%,18.34%,19.71%,21.28%,22.33%,22.54%
2023-12,469,2.13%,5.33%,7.04%,10.87%,12.58%,15.78%,18.34%,20.90%,23.03%,24.31%,24.95%
2024-01,342,1.46%,4.97%,7.89%,9.65%,13.16%,14.33%,17.25%,19.01%,20.47%,22.51%,23.10%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,195,3.59%,6.67%,12.31%,14.87%,15.90%,18.46%,20.51%,24.62%,23.59%,26.67%,28.72%
2023-05,141,2.13%,11.35%,15.60%,21.99%,23.40%,24.11%,26.24%,25.53%,29.08%,31.91%,33.33%
2023-06,122,2.46%,7.38%,18.03%,24.59%,29.51%,31.97%,34.43%,33.61%,37.70%,38.52%,38.52%
2023-07,149,4.70%,11.41%,15.44%,20.13%,26.17%,26.17%,28.19%,30.87%,32.89%,32.21%,30.87%
2023-08,202,4.95%,10.89%,14.85%,18.81%,22.28%,25.25%,27.23%,34.65%,35.64%,36.14%,35.64%
2023-09,119,3.36%,7.56%,10.08%,14.29%,16.81%,21.85%,23.53%,26.89%,29.41%,30.25%,30.25%
2023-10,102,3.92%,8.82%,13.73%,15.69%,21.57%,25.49%,27.45%,28.43%,29.41%,28.43%,30.39%
2023-11,194,1.55%,5.15%,9.79%,14.95%,19.59%,21.65%,22.68%,25.77%,27.32%,30.93%,31.44%
2023-12,169,nan%,2.96%,10.06%,11.24%,15.98%,17.75%,20.12%,21.89%,24.26%,26.63%,27.22%
2024-01,136,1.47%,3.68%,5.15%,10.29%,13.97%,16.18%,19.85%,23.53%,22.79%,25.00%,22.79%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,12,nan%,nan%,8.33%,8.33%,16.67%,16.67%,16.67%,25.00%,41.67%,25.00%,25.00%
2023-07,19,5.26%,5.26%,5.26%,21.05%,21.05%,26.32%,21.05%,21.05%,26.32%,26.32%,26.32%
2023-08,15,6.67%,13.33%,13.33%,13.33%,20.00%,20.00%,26.67%,26.67%,26.67%,26.67%,26.67%
2023-09,31,9.68%,6.45%,9.68%,16.13%,12.90%,19.35%,19.35%,29.03%,35.48%,35.48%,35.48%
2023-10,54,nan%,7.41%,12.96%,20.37%,24.07%,25.93%,25.93%,25.93%,25.93%,27.78%,27.78%
2023-11,143,4.20%,11.89%,16.78%,22.38%,25.87%,27.97%,31.47%,34.27%,35.66%,38.46%,37.76%
2023-12,248,2.02%,4.03%,8.47%,14.52%,18.55%,20.97%,23.79%,26.21%,29.84%,30.24%,29.84%
2024-01,136,2.94%,6.62%,9.56%,11.76%,13.97%,19.85%,20.59%,23.53%,24.26%,24.26%,24.26%
2024-06,5,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,35,2.86%,14.29%,14.29%,20.00%,20.00%,31.43%,28.57%,22.86%,22.86%,28.57%,28.57%
2023-05,31,3.23%,6.45%,6.45%,3.23%,3.23%,9.68%,12.90%,12.90%,19.35%,19.35%,19.35%
2023-06,18,nan%,nan%,5.56%,5.56%,5.56%,5.56%,5.56%,5.56%,5.56%,11.11%,11.11%
2023-07,35,11.43%,22.86%,37.14%,37.14%,40.00%,45.71%,48.57%,51.43%,57.14%,54.29%,51.43%
2023-08,46,19.57%,30.43%,34.78%,47.83%,45.65%,50.00%,54.35%,58.70%,63.04%,56.52%,54.35%
2023-09,42,11.90%,16.67%,28.57%,33.33%,47.62%,47.62%,47.62%,52.38%,54.76%,54.76%,52.38%
2023-10,46,10.87%,23.91%,34.78%,45.65%,39.13%,45.65%,47.83%,47.83%,47.83%,47.83%,47.83%
2023-11,81,3.70%,13.58%,19.75%,25.93%,30.86%,32.10%,29.63%,30.86%,34.57%,35.80%,34.57%
2023-12,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2024-01,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,16.33%,22.45%,26.53%,30.61%,38.78%,42.86%,46.94%,44.90%,44.90%,46.94%,51.02%
2024-07,111,15.32%,25.23%,31.53%,36.04%,37.84%,42.34%,45.05%,46.85%,46.85%,51.35%,55.86%
2024-08,65,30.77%,36.92%,24.62%,26.15%,27.69%,29.23%,32.31%,36.92%,38.46%,38.46%,nan%
2024-09,8,25.00%,25.00%,37.50%,50.00%,50.00%,50.00%,50.00%,50.00%,62.50%,nan%,nan%
2024-11,3,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,nan%,nan%,nan%,nan%
2024-12,59,15.25%,16.95%,25.42%,27.12%,27.12%,28.81%,nan%,nan%,nan%,nan%,nan%
2025-01,51,3.92%,7.84%,11.76%,15.69%,23.53%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,58,5.17%,10.34%,15.52%,18.97%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,33.33%,33.33%,22.22%,22.22%,22.22%,33.33%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,nan%,11.11%,nan%,11.11%,11.11%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,5.88%,5.88%,17.65%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%
2023-02,30,10.00%,10.00%,13.33%,16.67%,16.67%,20.00%,26.67%,26.67%,33.33%,26.67%,26.67%
2023-03,44,4.55%,6.82%,13.64%,20.45%,27.27%,27.27%,25.00%,29.55%,29.55%,25.00%,29.55%
2023-04,74,1.35%,1.35%,9.46%,8.11%,8.11%,6.76%,8.11%,6.76%,8.11%,8.11%,10.81%


MOB,订单量,mob_2,mob_3,mob_4
2024-12,30,nan%,3.33%,3.33%
2025-01,31,nan%,nan%,9.68%
2025-02,251,5.98%,12.35%,19.52%
2025-03,304,7.24%,16.45%,22.04%
2025-04,417,9.11%,18.94%,nan%
2025-05,368,7.61%,nan%,nan%
2025-06,303,nan%,nan%,nan%
2025-07,210,nan%,nan%,nan%
2025-08,21,nan%,nan%,nan%


## 4_16.vintage 4+ 金额 口径

In [97]:
df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']


In [98]:
# df_mob_total.shape
# df_mob_total[:1]
# df_mob_total[df_mob_total.order_number == 'A20241103155713186'][['order_number','下单日期','paid_money_mob', 'overdue_days_mob', '当前应付租金','MOB','purchase_amount','paid_money_mob','unpaid_money_mob']]
# df_mob_total[df_mob_total.order_number == 'A20250122164125236'][['order_number','下单日期','paid_money_mob', 'overdue_days_mob', '当前应付租金','MOB','purchase_amount','paid_money_mob','unpaid_money_mob']]


In [99]:
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) ].shape
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) & (df_mob_total.purchase_amount > 0)].shape
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) & (df_mob_total.purchase_amount > 0)][:3]
# df_mob_total[df_mob_total.order_number == 'A202207131230485']
# df_mob_total[df_mob_total.order_number == 'A202207131230485']
# df_fq_concat[df_fq_concat.order_number == 'A2024060516202368']
# A2024060516202368
# df_mob_total[:3]

# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) ]['order_number'].nunique()
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0)&(df_mob_total.overdue_days_1==1) ]['order_number'].nunique()


In [100]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# note 20241127 增加条件 (df_mob_total.unpaid_money_mob > 0)   
# note 20250429

# （1）总体
# vintage_money_d1 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_1==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

# vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
# vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

vintage_money_d4 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d4['采购金额'] = df_mon['采购金额']
# vintage_money_d4
vintage_divide_money_d4 = vintage_money_d4.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
vintage_money_d4 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_all_2],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d4['采购金额'] = df_mon['采购金额']
# vintage_money_d4
vintage_divide_money_d4 = vintage_money_d4.divide(df_mon2_1["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4['采购金额'] = df_mon2_1["purchase_amount"]
vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （3）芝麻租物
vintage_money_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_zm],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_zm = vintage_money_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',
vintage_divide_money_d4_zm = vintage_money_d4_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d4_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （4）搜索渠道
vintage_money_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_ss],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_ss = vintage_money_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_ss = vintage_money_d4_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d4_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （5）单人会话
vintage_money_d4_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_drhh],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_drhh = vintage_money_d4_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d4_drhh = vintage_money_d4_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d4_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_money_d4_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_sl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_sl = vintage_money_d4_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d4_sl = vintage_money_d4_sl.divide(df_mon_sl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_sl['采购金额'] = df_mon_sl["purchase_amount"]
vintage_divide_money_d4_sl[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （7）抖音
vintage_money_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_dy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_dy = vintage_money_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',,'mob_2',
vintage_divide_money_d4_dy = vintage_money_d4_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d4_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （8）二手非抖音
vintage_money_d4_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_esfdy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_esfdy = vintage_money_d4_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d4_esfdy = vintage_money_d4_esfdy.divide(df_mon_esfdy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_esfdy['采购金额'] = df_mon_esfdy["purchase_amount"]
vintage_divide_money_d4_esfdy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （9）拒量
vintage_money_d4_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_jl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_jl = vintage_money_d4_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_money_d4_jl = vintage_money_d4_jl.divide(df_mon_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_jl['采购金额'] = df_mon_jl["purchase_amount"]
vintage_divide_money_d4_jl[['采购金额','mob_1','mob_2','mob_3','mob_4']]


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,28.38%,29.41%,28.84%,28.84%,28.84%
2022-08,358451.00,12.39%,19.81%,24.38%,24.76%,25.57%,27.19%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,18.43%,21.42%,19.59%,18.61%,18.87%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,7.75%,12.33%,18.57%,16.13%,14.42%,16.43%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,11.72%,10.45%,16.50%,15.52%,12.47%,13.24%,12.93%,12.12%,12.12%
2022-12,566276.59,2.36%,6.21%,9.10%,14.62%,12.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.38%,8.36%,9.92%,10.62%,13.12%,14.40%,14.05%,14.56%,14.77%,14.52%,14.36%,14.23%
2023-02,2181552.82,6.80%,10.86%,13.71%,14.54%,16.71%,16.95%,16.84%,17.41%,16.31%,16.21%,15.71%,15.53%
2023-03,2083021.00,5.91%,10.37%,12.55%,16.12%,17.16%,18.82%,18.50%,17.96%,18.21%,18.30%,17.70%,17.60%
2023-04,4317651.50,3.49%,7.37%,11.54%,13.16%,14.83%,16.06%,16.53%,16.77%,16.81%,16.53%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,28.38%,29.41%,28.84%,28.84%,28.84%
2022-08,358451.00,12.39%,19.81%,24.38%,24.76%,25.57%,27.19%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,18.43%,21.42%,19.59%,18.61%,18.87%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,7.75%,12.33%,18.57%,16.13%,14.42%,16.43%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,11.72%,10.45%,16.50%,15.52%,12.47%,13.24%,12.93%,12.12%,12.12%
2022-12,566276.59,2.36%,6.21%,9.10%,14.62%,12.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.38%,8.36%,9.92%,10.62%,13.12%,14.40%,14.05%,14.56%,14.77%,14.52%,14.36%,14.23%
2023-02,2181552.82,6.80%,10.86%,13.71%,14.54%,16.71%,16.95%,16.84%,17.41%,16.31%,16.21%,15.71%,15.53%
2023-03,2083021.00,5.91%,10.37%,12.55%,16.12%,17.16%,18.82%,18.50%,17.96%,18.21%,18.30%,17.70%,17.60%
2023-04,4317651.50,3.49%,7.37%,11.54%,13.16%,14.83%,16.06%,16.53%,16.77%,16.81%,16.53%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1355156.00,1.98%,5.44%,8.21%,8.92%,9.85%,13.31%,13.76%,14.43%,14.63%,13.80%,14.01%,13.97%
2023-05,844546.00,nan%,3.68%,7.24%,6.53%,8.80%,7.98%,11.97%,13.24%,13.92%,12.70%,13.44%,12.98%
2023-06,1607377.00,3.17%,6.14%,10.52%,13.01%,14.12%,15.43%,17.06%,18.12%,17.74%,19.03%,18.75%,18.58%
2023-07,3594170.00,2.80%,7.99%,12.25%,13.71%,14.83%,15.93%,16.30%,17.51%,18.67%,18.92%,17.30%,16.76%
2023-08,3504454.00,2.82%,7.16%,8.17%,10.10%,12.75%,12.76%,13.80%,15.14%,15.99%,15.61%,14.83%,14.67%
2023-09,3988756.12,2.58%,4.70%,6.64%,9.01%,8.64%,9.77%,11.43%,12.96%,13.82%,13.87%,13.32%,13.01%
2023-10,5723013.25,3.16%,5.09%,8.74%,10.59%,12.44%,13.49%,15.21%,16.68%,17.06%,17.32%,16.92%,16.65%
2023-11,7234215.00,1.96%,5.94%,7.57%,8.80%,10.94%,12.53%,13.44%,14.15%,14.36%,14.60%,14.48%,13.96%
2023-12,3352399.00,1.97%,4.48%,6.81%,9.09%,10.32%,12.64%,13.84%,14.09%,14.78%,15.19%,15.09%,14.57%
2024-01,2295409.00,1.60%,3.88%,6.72%,8.70%,9.87%,11.59%,12.77%,12.40%,13.11%,13.63%,13.04%,12.97%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1305173.00,3.86%,9.43%,14.63%,15.36%,16.72%,15.53%,17.76%,17.21%,17.71%,17.28%,17.10%,16.71%
2023-05,925666.00,3.17%,12.81%,16.97%,19.19%,22.75%,20.72%,19.75%,19.46%,18.50%,19.16%,18.86%,18.43%
2023-06,830115.00,4.20%,9.09%,18.08%,22.90%,24.45%,25.86%,26.57%,25.42%,25.40%,25.33%,23.85%,23.77%
2023-07,1062805.00,4.55%,12.69%,14.67%,15.28%,18.25%,21.39%,20.32%,20.08%,20.62%,19.65%,18.24%,17.78%
2023-08,1418637.00,5.16%,9.05%,13.36%,16.12%,18.86%,20.15%,19.65%,21.73%,22.22%,21.91%,21.40%,20.90%
2023-09,910558.00,2.11%,7.29%,8.49%,13.62%,14.83%,15.19%,16.87%,17.01%,18.30%,17.85%,16.76%,16.76%
2023-10,873717.50,0.90%,8.05%,11.84%,13.86%,17.30%,19.06%,21.51%,20.61%,21.10%,20.20%,20.42%,20.42%
2023-11,1579325.00,1.99%,7.96%,8.96%,12.65%,14.91%,15.27%,15.56%,16.66%,16.33%,16.22%,15.98%,15.66%
2023-12,1373729.75,1.21%,2.01%,7.36%,8.79%,11.27%,12.06%,12.97%,14.14%,13.50%,13.22%,13.06%,12.63%
2024-01,1021172.00,1.29%,3.43%,4.33%,8.01%,9.96%,10.76%,11.24%,12.60%,12.85%,12.44%,12.66%,12.11%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,6840.00,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,79168.00,nan%,nan%,nan%,8.13%,10.55%,10.55%,10.55%,13.23%,20.22%,20.22%,13.23%,13.23%
2023-07,149045.00,4.78%,4.78%,11.68%,7.00%,13.71%,16.45%,16.45%,13.71%,16.92%,15.31%,15.31%,15.31%
2023-08,126234.00,11.33%,11.88%,11.88%,11.88%,15.15%,15.15%,18.45%,21.23%,20.76%,18.45%,17.51%,17.51%
2023-09,261610.00,8.46%,8.46%,7.92%,10.99%,12.97%,11.70%,13.13%,15.53%,16.44%,17.58%,17.58%,17.36%
2023-10,474985.00,4.53%,7.16%,15.43%,16.69%,18.61%,20.28%,19.06%,22.29%,19.90%,18.84%,17.82%,17.82%
2023-11,1226260.00,3.75%,11.95%,14.43%,17.76%,18.31%,20.92%,20.78%,22.63%,22.49%,21.91%,21.38%,21.18%
2023-12,2049806.25,2.03%,5.02%,7.70%,10.45%,14.09%,14.49%,15.03%,16.70%,16.68%,16.16%,15.74%,15.71%
2024-01,1104144.00,2.48%,5.64%,8.21%,9.95%,11.36%,13.77%,13.55%,14.42%,14.27%,13.50%,13.40%,13.30%
2024-06,39138.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,244612.0,2.32%,11.26%,18.30%,19.55%,18.72%,20.60%,19.92%,20.55%,15.11%,13.80%,14.50%,14.50%
2023-05,208404.0,3.70%,7.12%,7.12%,12.06%,5.39%,7.20%,9.13%,9.13%,12.17%,12.17%,11.70%,11.70%
2023-06,132717.0,nan%,nan%,3.60%,3.60%,3.60%,3.60%,3.60%,6.23%,3.60%,3.60%,3.60%,3.60%
2023-07,277310.0,7.65%,12.88%,20.86%,24.59%,20.54%,22.34%,22.82%,22.55%,22.55%,20.92%,20.32%,20.32%
2023-08,378466.0,13.46%,18.66%,21.09%,22.36%,21.22%,19.24%,18.77%,18.28%,18.28%,17.99%,16.92%,16.84%
2023-09,356086.0,6.05%,11.20%,16.05%,18.28%,20.06%,18.42%,17.49%,17.87%,15.17%,15.84%,14.83%,14.83%
2023-10,373112.0,5.93%,15.97%,26.20%,28.36%,29.29%,24.46%,27.35%,26.56%,27.09%,26.56%,26.56%,26.56%
2023-11,710870.0,2.53%,12.15%,14.51%,15.33%,17.11%,17.47%,16.52%,15.65%,15.40%,15.69%,15.69%,15.40%
2023-12,9010.0,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%
2024-01,7750.0,nan%,nan%,nan%,24.52%,11.95%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%,28.83%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,14.35%,18.77%,19.06%,25.17%,25.84%,27.42%,28.84%,28.74%,27.62%,27.52%,28.34%,27.70%
2024-07,448562.0,13.58%,20.41%,25.99%,28.96%,28.62%,29.82%,31.12%,31.78%,31.75%,31.68%,32.08%,32.05%
2024-08,271022.0,17.48%,26.44%,25.52%,26.51%,24.67%,26.66%,26.56%,29.47%,29.24%,29.10%,28.96%,nan%
2024-09,30173.0,15.33%,24.41%,34.84%,40.23%,46.48%,38.32%,43.19%,33.35%,41.07%,39.89%,nan%,nan%
2024-11,9865.0,nan%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,nan%,nan%,nan%,nan%
2024-12,105399.0,28.94%,30.28%,40.27%,40.93%,47.28%,45.11%,44.44%,nan%,nan%,nan%,nan%,nan%
2025-01,95598.0,4.78%,13.42%,20.85%,22.57%,22.68%,25.88%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,119574.0,13.35%,8.76%,29.41%,23.26%,34.54%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,8.53%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,29.68%,28.30%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,9.68%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,9.70%,nan%,5.54%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,11.74%,11.56%,10.69%,12.43%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,12.57%,12.39%,14.27%,20.30%,15.78%,21.39%,22.39%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,4.74%,9.50%,12.07%,17.02%,19.04%,23.24%,21.70%,21.63%,21.66%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,3.09%,5.01%,9.63%,10.68%,9.73%,9.16%,6.79%,6.44%,6.62%,6.44%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4
2024-12,236025.0,nan%,3.92%,6.69%,9.34%
2025-01,269323.0,6.27%,nan%,13.17%,13.12%
2025-02,2132295.0,6.33%,11.72%,16.10%,16.82%
2025-03,2499753.0,4.95%,13.62%,15.87%,18.53%
2025-04,3521285.0,7.66%,15.45%,18.58%,nan%
2025-05,2961684.0,7.15%,14.07%,nan%,nan%
2025-06,2380146.0,6.40%,nan%,nan%,nan%
2025-07,1674599.0,nan%,nan%,nan%,nan%
2025-08,149441.0,nan%,nan%,nan%,nan%


## 4_17.vintage 7+ 金额 口径

In [101]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体


vintage_money_d7 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7['采购金额'] = df_mon['采购金额']
# vintage_money_d7
vintage_divide_money_d7 = vintage_money_d7.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
vintage_money_d7 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_all_2],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7['采购金额'] = df_mon['采购金额']
# vintage_money_d7
vintage_divide_money_d7 = vintage_money_d7.divide(df_mon2_1["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7['采购金额'] = df_mon2_1["purchase_amount"]
vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （3）芝麻租物
vintage_money_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_zm],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_zm = vintage_money_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',
vintage_divide_money_d7_zm = vintage_money_d7_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d7_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （4）搜索渠道
vintage_money_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_ss],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_ss = vintage_money_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_ss = vintage_money_d7_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d7_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （5）单人会话
vintage_money_d7_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_drhh],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_drhh = vintage_money_d7_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d7_drhh = vintage_money_d7_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d7_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_money_d7_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_sl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_sl = vintage_money_d7_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d7_sl = vintage_money_d7_sl.divide(df_mon_sl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_sl['采购金额'] = df_mon_sl["purchase_amount"]
vintage_divide_money_d7_sl[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （7）抖音
vintage_money_d7_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_dy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_dy = vintage_money_d7_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',,'mob_2',
vintage_divide_money_d7_dy = vintage_money_d7_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d7_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #,'mob_7','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （8）二手非抖音
vintage_money_d7_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_esfdy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_esfdy = vintage_money_d7_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d7_esfdy = vintage_money_d7_esfdy.divide(df_mon_esfdy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_esfdy['采购金额'] = df_mon_esfdy["purchase_amount"]
vintage_divide_money_d7_esfdy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （9）拒量
vintage_money_d7_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_jl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_jl = vintage_money_d7_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_money_d7_jl = vintage_money_d7_jl.divide(df_mon_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_jl['采购金额'] = df_mon_jl["purchase_amount"]
vintage_divide_money_d7_jl[['采购金额','mob_1','mob_2','mob_3','mob_4']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,3.02%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,28.38%,29.41%,28.84%,28.84%,28.84%
2022-08,358451.00,9.77%,19.81%,24.38%,24.76%,25.57%,25.99%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,16.55%,21.42%,19.59%,18.61%,17.90%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,18.57%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,9.44%,9.44%,14.48%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,1.14%,6.21%,8.10%,14.62%,11.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.97%,7.97%,9.92%,10.62%,12.05%,14.29%,13.41%,14.27%,14.77%,14.52%,14.36%,14.23%
2023-02,2181552.82,6.41%,10.07%,13.19%,14.45%,16.30%,16.47%,16.79%,17.24%,16.16%,16.10%,15.71%,15.53%
2023-03,2083021.00,4.06%,9.54%,11.16%,15.50%,16.61%,17.60%,17.68%,17.58%,18.21%,18.30%,17.70%,17.60%
2023-04,4317651.50,3.19%,6.62%,10.75%,12.69%,14.42%,15.89%,16.15%,16.55%,16.69%,16.43%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,3.02%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,28.38%,29.41%,28.84%,28.84%,28.84%
2022-08,358451.00,9.77%,19.81%,24.38%,24.76%,25.57%,25.99%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,16.55%,21.42%,19.59%,18.61%,17.90%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,18.57%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,9.44%,9.44%,14.48%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,1.14%,6.21%,8.10%,14.62%,11.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.97%,7.97%,9.92%,10.62%,12.05%,14.29%,13.41%,14.27%,14.77%,14.52%,14.36%,14.23%
2023-02,2181552.82,6.41%,10.07%,13.19%,14.45%,16.30%,16.47%,16.79%,17.24%,16.16%,16.10%,15.71%,15.53%
2023-03,2083021.00,4.06%,9.54%,11.16%,15.50%,16.61%,17.60%,17.68%,17.58%,18.21%,18.30%,17.70%,17.60%
2023-04,4317651.50,3.19%,6.62%,10.75%,12.69%,14.42%,15.89%,16.15%,16.55%,16.69%,16.43%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1355156.00,1.98%,4.93%,7.72%,8.92%,9.85%,13.31%,13.38%,14.43%,14.63%,13.80%,14.01%,13.97%
2023-05,844546.00,nan%,3.68%,7.17%,6.53%,7.87%,7.98%,11.97%,12.81%,13.92%,12.40%,13.24%,12.98%
2023-06,1607377.00,2.21%,4.87%,9.74%,12.05%,13.53%,14.84%,17.06%,17.19%,17.74%,18.87%,18.59%,18.58%
2023-07,3594170.00,2.41%,6.66%,11.08%,13.03%,14.04%,15.13%,15.70%,17.05%,18.30%,18.44%,17.30%,16.76%
2023-08,3504454.00,2.52%,6.48%,7.79%,9.58%,12.58%,12.76%,13.50%,15.01%,15.69%,15.54%,14.83%,14.67%
2023-09,3988756.12,2.21%,4.00%,6.16%,7.86%,8.50%,9.23%,11.19%,12.76%,13.34%,13.69%,13.32%,13.01%
2023-10,5723013.25,3.04%,4.31%,8.47%,10.51%,12.24%,13.31%,15.08%,16.50%,17.06%,17.09%,16.92%,16.65%
2023-11,7234215.00,1.59%,5.49%,7.37%,8.38%,10.46%,11.99%,13.14%,13.93%,14.22%,14.43%,14.48%,13.96%
2023-12,3352399.00,1.74%,3.80%,6.34%,8.70%,9.92%,11.76%,13.58%,13.87%,14.78%,15.13%,15.05%,14.57%
2024-01,2295409.00,1.60%,3.68%,6.42%,7.53%,9.40%,10.96%,12.53%,12.26%,12.90%,13.27%,12.99%,12.97%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1305173.00,2.85%,7.47%,13.76%,14.47%,15.73%,15.04%,16.90%,16.99%,17.71%,17.10%,17.10%,16.71%
2023-05,925666.00,3.17%,12.12%,15.88%,17.98%,22.23%,20.22%,19.30%,19.05%,18.50%,18.95%,18.86%,18.43%
2023-06,830115.00,4.20%,9.09%,16.10%,22.17%,23.20%,25.28%,26.06%,24.97%,25.40%,25.03%,23.85%,23.77%
2023-07,1062805.00,3.88%,11.67%,13.45%,15.28%,17.39%,21.39%,19.87%,19.30%,20.62%,19.65%,18.24%,17.78%
2023-08,1418637.00,4.15%,8.56%,12.62%,15.69%,18.69%,19.75%,19.65%,21.73%,22.22%,21.56%,21.40%,20.90%
2023-09,910558.00,2.11%,7.29%,8.49%,13.62%,13.62%,15.19%,16.36%,17.01%,18.30%,17.85%,16.76%,16.76%
2023-10,873717.50,0.90%,8.05%,11.11%,13.36%,16.63%,18.41%,21.51%,20.61%,21.10%,20.20%,20.42%,20.42%
2023-11,1579325.00,1.50%,6.61%,7.41%,10.84%,13.87%,14.24%,15.01%,15.80%,15.95%,15.91%,15.75%,15.66%
2023-12,1373729.75,0.61%,2.01%,6.50%,8.79%,10.92%,11.83%,12.44%,13.66%,12.77%,12.96%,12.96%,12.63%
2024-01,1021172.00,1.29%,3.43%,4.33%,8.01%,9.96%,10.37%,11.24%,12.60%,12.59%,12.31%,12.66%,12.11%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,6840.00,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,79168.00,nan%,nan%,nan%,8.13%,10.55%,10.55%,10.55%,13.23%,20.22%,20.22%,13.23%,13.23%
2023-07,149045.00,nan%,4.78%,8.52%,7.00%,13.71%,13.71%,16.45%,13.71%,16.92%,15.31%,15.31%,15.31%
2023-08,126234.00,11.33%,11.88%,11.88%,11.88%,15.15%,15.15%,18.45%,21.23%,20.76%,18.45%,17.51%,17.51%
2023-09,261610.00,8.46%,8.46%,7.92%,10.99%,12.97%,10.06%,11.70%,12.89%,16.44%,17.58%,17.58%,17.36%
2023-10,474985.00,3.08%,5.84%,14.23%,16.69%,17.46%,19.47%,19.06%,21.65%,19.38%,18.84%,17.82%,17.82%
2023-11,1226260.00,3.15%,10.35%,13.39%,16.91%,17.97%,20.92%,20.78%,21.82%,22.25%,21.76%,21.38%,21.18%
2023-12,2049806.25,2.03%,3.85%,6.83%,10.02%,13.37%,13.57%,14.83%,16.53%,16.20%,16.09%,15.74%,15.71%
2024-01,1104144.00,2.48%,5.64%,8.21%,9.95%,10.92%,13.77%,13.55%,14.16%,14.27%,13.32%,13.40%,13.30%
2024-06,39138.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,244612.0,2.32%,11.26%,16.55%,18.24%,16.67%,20.17%,19.92%,17.85%,12.85%,12.85%,14.50%,14.50%
2023-05,208404.0,3.70%,7.12%,7.12%,12.06%,5.39%,7.20%,9.13%,9.13%,12.17%,12.17%,11.70%,11.70%
2023-06,132717.0,nan%,nan%,3.60%,3.60%,3.60%,3.60%,3.60%,6.23%,3.60%,3.60%,3.60%,3.60%
2023-07,277310.0,4.03%,12.88%,18.79%,24.59%,20.54%,22.34%,22.82%,22.55%,22.55%,20.92%,20.32%,20.32%
2023-08,378466.0,12.05%,18.66%,20.30%,22.36%,21.03%,19.19%,18.77%,18.28%,18.28%,17.99%,16.92%,16.84%
2023-09,356086.0,6.05%,11.20%,14.96%,18.28%,18.51%,18.42%,17.49%,17.87%,15.17%,15.84%,14.83%,14.83%
2023-10,373112.0,5.93%,14.48%,24.15%,28.36%,28.21%,24.46%,27.35%,26.56%,26.56%,26.56%,26.56%,26.56%
2023-11,710870.0,1.87%,12.15%,13.08%,15.33%,16.27%,16.38%,16.52%,15.09%,15.40%,15.69%,15.69%,15.40%
2023-12,9010.0,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%
2024-01,7750.0,nan%,nan%,nan%,24.52%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,9.15%,18.77%,17.93%,24.14%,25.84%,26.60%,28.13%,28.74%,27.11%,27.52%,28.34%,27.70%
2024-07,448562.0,10.76%,20.41%,25.99%,28.96%,28.62%,29.82%,31.12%,31.78%,31.75%,31.68%,32.08%,32.05%
2024-08,271022.0,15.54%,25.42%,23.22%,25.07%,24.67%,26.16%,26.56%,29.09%,29.24%,29.10%,28.96%,nan%
2024-09,30173.0,15.33%,24.41%,34.84%,40.23%,46.48%,38.32%,43.19%,33.35%,41.07%,39.89%,nan%,nan%
2024-11,9865.0,nan%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,nan%,nan%,nan%,nan%
2024-12,105399.0,23.06%,30.28%,40.27%,40.93%,47.28%,45.11%,44.44%,nan%,nan%,nan%,nan%,nan%
2025-01,95598.0,1.68%,13.42%,19.83%,22.57%,22.06%,25.88%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,119574.0,8.56%,7.76%,27.33%,21.42%,32.94%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,8.53%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,24.07%,28.30%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,7.87%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,9.70%,nan%,5.54%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,7.36%,11.56%,7.18%,9.36%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,10.68%,10.68%,12.75%,18.95%,15.78%,20.40%,21.58%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,1.90%,9.50%,12.07%,17.02%,19.04%,23.24%,21.70%,21.55%,21.66%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,3.09%,4.42%,9.63%,10.68%,9.73%,9.16%,6.79%,6.44%,6.62%,6.44%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4
2024-12,236025.0,nan%,3.92%,6.69%,9.34%
2025-01,269323.0,nan%,nan%,13.17%,13.12%
2025-02,2132295.0,4.96%,10.61%,15.10%,15.91%
2025-03,2499753.0,3.65%,12.89%,15.27%,17.83%
2025-04,3521285.0,5.29%,13.21%,17.24%,nan%
2025-05,2961684.0,6.49%,13.55%,nan%,nan%
2025-06,2380146.0,5.56%,nan%,nan%,nan%
2025-07,1674599.0,nan%,nan%,nan%,nan%
2025-08,149441.0,nan%,nan%,nan%,nan%


## 4_18.vintage 15+ 金额 口径

In [102]:


# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体


# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体


vintage_money_d15 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d15['采购金额'] = df_mon['采购金额']
# vintage_money_d15
vintage_divide_money_d15 = vintage_money_d15.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
vintage_money_d15 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_all_2],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d15['采购金额'] = df_mon['采购金额']
# vintage_money_d15
vintage_divide_money_d15 = vintage_money_d15.divide(df_mon2_1["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15['采购金额'] = df_mon2_1["purchase_amount"]
vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （3）芝麻租物
vintage_money_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_zm],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_zm = vintage_money_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',
vintage_divide_money_d15_zm = vintage_money_d15_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d15_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （4）搜索渠道
vintage_money_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_ss],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_ss = vintage_money_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_ss = vintage_money_d15_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d15_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （5）单人会话
vintage_money_d15_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_drhh],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_drhh = vintage_money_d15_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d15_drhh = vintage_money_d15_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d15_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_money_d15_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_sl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_sl = vintage_money_d15_sl[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d15_sl = vintage_money_d15_sl.divide(df_mon_sl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_sl['采购金额'] = df_mon_sl["purchase_amount"]
vintage_divide_money_d15_sl[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （7）抖音
vintage_money_d15_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_dy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_dy = vintage_money_d15_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',,'mob_2',
vintage_divide_money_d15_dy = vintage_money_d15_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d15_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #,'mob_7','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （8）二手非抖音
vintage_money_d15_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_esfdy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_esfdy = vintage_money_d15_esfdy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d15_esfdy = vintage_money_d15_esfdy.divide(df_mon_esfdy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_esfdy['采购金额'] = df_mon_esfdy["purchase_amount"]
vintage_divide_money_d15_esfdy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （9）拒量
vintage_money_d15_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_jl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_jl = vintage_money_d15_jl[['mob_1','mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_money_d15_jl = vintage_money_d15_jl.divide(df_mon_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_jl['采购金额'] = df_mon_jl["purchase_amount"]
vintage_divide_money_d15_jl[['采购金额','mob_1','mob_2','mob_3','mob_4']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,28.38%,28.38%,28.84%,28.84%,28.84%
2022-08,358451.00,5.45%,13.09%,18.50%,21.03%,24.15%,25.99%,26.45%,27.19%,26.35%,26.93%,27.46%,26.76%
2022-09,277982.75,2.27%,10.01%,15.15%,14.79%,19.98%,19.59%,18.61%,17.90%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,16.70%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,4.44%,9.70%,8.05%,9.44%,9.44%,12.78%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,nan%,6.21%,6.12%,14.62%,11.84%,11.75%,14.92%,14.37%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.35%,7.29%,8.22%,10.33%,11.78%,14.03%,13.17%,14.07%,14.61%,14.27%,14.36%,14.23%
2023-02,2181552.82,3.97%,8.38%,12.27%,12.50%,15.80%,15.49%,15.94%,16.82%,16.16%,16.00%,15.71%,15.53%
2023-03,2083021.00,2.85%,7.78%,9.69%,13.25%,14.33%,16.73%,16.89%,17.01%,17.61%,18.02%,17.70%,17.60%
2023-04,4317651.50,2.18%,5.31%,9.31%,11.40%,13.08%,14.99%,15.49%,15.76%,16.42%,15.80%,16.21%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,28.38%,28.38%,28.84%,28.84%,28.84%
2022-08,358451.00,5.45%,13.09%,18.50%,21.03%,24.15%,25.99%,26.45%,27.19%,26.35%,26.93%,27.46%,26.76%
2022-09,277982.75,2.27%,10.01%,15.15%,14.79%,19.98%,19.59%,18.61%,17.90%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,16.70%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,4.44%,9.70%,8.05%,9.44%,9.44%,12.78%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,nan%,6.21%,6.12%,14.62%,11.84%,11.75%,14.92%,14.37%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.35%,7.29%,8.22%,10.33%,11.78%,14.03%,13.17%,14.07%,14.61%,14.27%,14.36%,14.23%
2023-02,2181552.82,3.97%,8.38%,12.27%,12.50%,15.80%,15.49%,15.94%,16.82%,16.16%,16.00%,15.71%,15.53%
2023-03,2083021.00,2.85%,7.78%,9.69%,13.25%,14.33%,16.73%,16.89%,17.01%,17.61%,18.02%,17.70%,17.60%
2023-04,4317651.50,2.18%,5.31%,9.31%,11.40%,13.08%,14.99%,15.49%,15.76%,16.42%,15.80%,16.21%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1355156.00,0.98%,3.04%,6.47%,7.58%,9.14%,11.89%,12.71%,13.12%,14.63%,13.64%,14.01%,13.97%
2023-05,844546.00,nan%,2.34%,7.17%,6.53%,7.87%,7.73%,11.97%,12.22%,13.02%,12.09%,13.24%,12.98%
2023-06,1607377.00,1.36%,4.50%,8.33%,11.36%,13.35%,14.51%,16.56%,16.79%,17.46%,18.23%,18.59%,18.58%
2023-07,3594170.00,0.64%,4.31%,9.25%,11.62%,12.93%,13.86%,14.93%,16.32%,17.36%,17.99%,17.30%,16.76%
2023-08,3504454.00,1.42%,4.45%,6.66%,8.34%,11.32%,11.65%,12.81%,14.23%,15.09%,15.27%,14.83%,14.67%
2023-09,3988756.12,1.30%,3.51%,5.71%,7.29%,7.63%,8.67%,10.37%,11.89%,12.59%,13.15%,13.26%,13.01%
2023-10,5723013.25,1.79%,3.46%,7.41%,9.55%,11.08%,12.30%,13.95%,15.66%,16.72%,16.52%,16.63%,16.65%
2023-11,7234215.00,0.66%,4.30%,6.39%,7.88%,9.89%,11.34%,12.60%,13.30%,13.98%,14.04%,14.36%,13.96%
2023-12,3352399.00,1.23%,3.21%,5.36%,7.49%,9.22%,10.59%,12.85%,13.36%,14.11%,14.65%,14.78%,14.57%
2024-01,2295409.00,0.60%,3.08%,5.58%,7.30%,8.77%,10.84%,11.91%,12.24%,12.60%,12.85%,12.99%,12.97%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1305173.00,1.94%,6.67%,10.81%,12.48%,13.41%,14.43%,16.07%,16.02%,17.16%,15.92%,16.91%,16.71%
2023-05,925666.00,2.34%,10.44%,12.78%,16.47%,20.54%,19.18%,18.85%,19.05%,17.90%,18.28%,18.70%,18.43%
2023-06,830115.00,1.63%,6.86%,12.87%,19.56%,21.86%,23.53%,25.13%,24.97%,24.09%,24.45%,23.85%,23.77%
2023-07,1062805.00,2.43%,8.69%,12.23%,12.81%,16.24%,19.67%,19.42%,18.97%,20.07%,19.65%,18.08%,17.78%
2023-08,1418637.00,2.05%,8.56%,11.75%,14.57%,17.63%,18.34%,19.33%,20.99%,21.90%,21.56%,21.40%,20.90%
2023-09,910558.00,1.66%,6.49%,8.49%,10.68%,12.44%,14.07%,15.97%,16.28%,17.89%,17.50%,16.76%,16.76%
2023-10,873717.50,nan%,7.09%,7.91%,11.83%,15.28%,17.20%,20.98%,20.61%,21.10%,20.20%,20.16%,20.42%
2023-11,1579325.00,0.52%,4.20%,5.55%,9.12%,12.79%,13.80%,14.47%,14.54%,15.51%,15.79%,15.75%,15.66%
2023-12,1373729.75,nan%,1.62%,5.51%,8.45%,9.27%,11.51%,12.27%,13.39%,12.77%,12.80%,12.96%,12.63%
2024-01,1021172.00,0.65%,2.44%,3.55%,5.73%,8.67%,9.90%,11.24%,12.07%,12.47%,12.31%,12.66%,12.11%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,6840.00,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,79168.00,nan%,nan%,nan%,8.13%,10.55%,10.55%,10.55%,13.23%,20.22%,20.22%,13.23%,13.23%
2023-07,149045.00,nan%,4.78%,4.78%,7.00%,13.71%,13.71%,16.45%,13.71%,16.92%,15.31%,15.31%,15.31%
2023-08,126234.00,nan%,6.67%,11.88%,11.88%,15.15%,15.15%,18.45%,21.23%,20.76%,18.45%,17.51%,17.51%
2023-09,261610.00,2.81%,8.46%,5.46%,7.92%,12.97%,8.18%,11.70%,12.89%,15.53%,17.58%,17.58%,17.36%
2023-10,474985.00,nan%,3.08%,8.75%,14.01%,16.40%,19.47%,19.06%,20.16%,18.10%,17.81%,17.82%,17.82%
2023-11,1226260.00,1.38%,7.61%,12.90%,14.44%,17.60%,19.57%,19.86%,21.28%,21.71%,21.23%,21.38%,21.18%
2023-12,2049806.25,1.09%,3.52%,5.14%,8.50%,12.07%,12.75%,14.00%,15.82%,15.73%,16.09%,15.74%,15.71%
2024-01,1104144.00,1.88%,5.64%,6.23%,9.95%,10.19%,12.93%,13.55%,13.13%,13.86%,13.32%,13.40%,13.30%
2024-06,39138.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,244612.0,2.32%,8.94%,13.77%,18.24%,16.67%,18.33%,19.92%,17.85%,12.85%,12.85%,14.50%,14.50%
2023-05,208404.0,3.70%,7.12%,7.12%,12.06%,5.39%,7.20%,9.13%,9.13%,12.17%,12.17%,11.70%,11.70%
2023-06,132717.0,nan%,nan%,3.60%,3.60%,3.60%,3.60%,3.60%,6.23%,3.60%,3.60%,3.60%,3.60%
2023-07,277310.0,nan%,11.44%,18.79%,23.16%,20.54%,22.34%,22.55%,22.55%,22.55%,20.92%,20.32%,20.32%
2023-08,378466.0,9.34%,14.37%,20.30%,21.85%,21.03%,19.19%,18.77%,18.28%,18.28%,17.99%,16.92%,16.84%
2023-09,356086.0,4.50%,9.95%,14.09%,15.87%,15.85%,18.42%,17.49%,16.93%,15.17%,15.17%,14.83%,14.83%
2023-10,373112.0,4.45%,13.25%,24.15%,26.96%,28.21%,23.66%,27.35%,26.56%,26.56%,26.56%,26.56%,26.56%
2023-11,710870.0,1.87%,7.17%,12.55%,14.20%,16.27%,15.93%,16.52%,14.95%,15.40%,15.69%,15.69%,15.40%
2023-12,9010.0,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%
2024-01,7750.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,15930.0,nan%,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,1.12%,15.62%,17.93%,21.05%,23.98%,25.49%,27.42%,28.13%,27.11%,27.52%,28.34%,27.70%
2024-07,448562.0,6.63%,20.41%,25.99%,27.44%,28.62%,29.41%,30.76%,31.23%,31.33%,31.01%,32.08%,32.05%
2024-08,271022.0,7.41%,23.75%,23.22%,23.60%,23.91%,26.16%,24.69%,28.50%,28.92%,28.69%,28.96%,nan%
2024-09,30173.0,7.93%,15.33%,26.51%,26.51%,40.92%,33.35%,38.82%,33.35%,37.30%,37.30%,nan%,nan%
2024-11,9865.0,nan%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,nan%,nan%,nan%,nan%
2024-12,105399.0,13.03%,28.94%,34.45%,39.82%,47.28%,45.11%,44.44%,nan%,nan%,nan%,nan%,nan%
2025-01,95598.0,nan%,12.19%,10.96%,19.55%,21.25%,24.72%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,119574.0,nan%,6.62%,17.67%,21.42%,28.90%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,24.07%,23.14%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,7.87%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,7.36%,11.56%,7.18%,9.36%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,3.99%,10.68%,12.75%,15.78%,15.78%,17.88%,19.38%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,nan%,6.55%,7.79%,14.54%,16.88%,21.21%,19.84%,18.77%,20.74%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,1.24%,4.27%,8.50%,8.58%,9.05%,7.29%,6.79%,5.89%,5.06%,6.12%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4
2024-12,236025.0,nan%,3.92%,3.92%,3.92%
2025-01,269323.0,nan%,nan%,nan%,6.67%
2025-02,2132295.0,1.59%,8.08%,13.09%,15.21%
2025-03,2499753.0,2.46%,10.63%,14.03%,17.08%
2025-04,3521285.0,1.93%,10.68%,15.98%,nan%
2025-05,2961684.0,4.64%,11.55%,nan%,nan%
2025-06,2380146.0,2.63%,nan%,nan%,nan%
2025-07,1674599.0,nan%,nan%,nan%,nan%
2025-08,149441.0,nan%,nan%,nan%,nan%


## 4_19.vintage 31+ 金额 口径

In [103]:


# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_money_d31 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d31['采购金额'] = df_mon['采购金额']
# vintage_money_d31
vintage_divide_money_d31 = vintage_money_d31.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （2）总体非抖音非拒量非号卡
vintage_money_d31 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_all_2],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d31['采购金额'] = df_mon['采购金额']
# vintage_money_d31
vintage_divide_money_d31 = vintage_money_d31.divide(df_mon2_1["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31['采购金额'] = df_mon2_1["purchase_amount"]
vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （3）芝麻租物
vintage_money_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_zm],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_zm = vintage_money_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_zm = vintage_money_d31_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d31_zm[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （4）搜索渠道
vintage_money_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_ss],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_ss = vintage_money_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_ss = vintage_money_d31_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d31_ss[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （5）单人会话
vintage_money_d31_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_drhh],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_drhh = vintage_money_d31_drhh[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_drhh = vintage_money_d31_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d31_drhh[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （6）S量
vintage_money_d31_sl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_sl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_sl = vintage_money_d31_sl[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_sl = vintage_money_d31_sl.divide(df_mon_sl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_sl['采购金额'] = df_mon_sl["purchase_amount"]
vintage_divide_money_d31_sl[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （7）抖音
vintage_money_d31_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & conditions_dy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_dy = vintage_money_d31_dy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',,'mob_2',
vintage_divide_money_d31_dy = vintage_money_d31_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d31_dy[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #,'mob_7','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （8）二手非抖音
vintage_money_d31_esfdy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_esfdy],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_esfdy = vintage_money_d31_esfdy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_esfdy = vintage_money_d31_esfdy.divide(df_mon_esfdy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_esfdy['采购金额'] = df_mon_esfdy["purchase_amount"]
vintage_divide_money_d31_esfdy[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]


# （9）拒量
vintage_money_d31_jl = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&conditions_jl],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_jl = vintage_money_d31_jl[['mob_2','mob_3','mob_4']]  #'下单月份','mob_0',
vintage_divide_money_d31_jl = vintage_money_d31_jl.divide(df_mon_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_jl['采购金额'] = df_mon_jl["purchase_amount"]
# vintage_divide_money_d31_jl[['采购金额','mob_2','mob_3']]
vintage_divide_money_d31_jl[['采购金额','mob_2','mob_3','mob_4']]


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,28.38%,28.38%,28.84%,28.84%,28.84%
2022-08,358451.00,11.16%,16.74%,20.26%,23.27%,23.85%,25.22%,26.45%,25.51%,26.93%,26.93%,26.76%
2022-09,277982.75,7.73%,12.04%,14.42%,18.34%,18.61%,18.61%,17.11%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,5.47%,5.30%,7.75%,12.33%,14.00%,10.83%,14.42%,14.00%,15.67%,15.27%,15.04%
2022-11,344258.81,8.08%,8.05%,9.44%,9.44%,10.45%,12.51%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,3.85%,5.20%,11.13%,11.84%,11.75%,13.89%,14.37%,15.74%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.21%,6.53%,8.33%,9.55%,12.84%,12.41%,13.07%,14.20%,14.09%,14.29%,14.23%
2023-02,2181552.82,5.71%,9.35%,11.65%,14.48%,14.35%,15.33%,16.06%,16.01%,15.58%,15.63%,15.53%
2023-03,2083021.00,5.95%,8.84%,11.80%,14.21%,15.66%,15.84%,16.89%,17.37%,17.71%,17.21%,17.60%
2023-04,4317651.50,3.00%,5.96%,8.99%,11.16%,13.00%,14.27%,15.00%,15.87%,15.53%,15.95%,16.07%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,28.38%,28.38%,28.84%,28.84%,28.84%
2022-08,358451.00,11.16%,16.74%,20.26%,23.27%,23.85%,25.22%,26.45%,25.51%,26.93%,26.93%,26.76%
2022-09,277982.75,7.73%,12.04%,14.42%,18.34%,18.61%,18.61%,17.11%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,5.47%,5.30%,7.75%,12.33%,14.00%,10.83%,14.42%,14.00%,15.67%,15.27%,15.04%
2022-11,344258.81,8.08%,8.05%,9.44%,9.44%,10.45%,12.51%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,3.85%,5.20%,11.13%,11.84%,11.75%,13.89%,14.37%,15.74%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.21%,6.53%,8.33%,9.55%,12.84%,12.41%,13.07%,14.20%,14.09%,14.29%,14.23%
2023-02,2181552.82,5.71%,9.35%,11.65%,14.48%,14.35%,15.33%,16.06%,16.01%,15.58%,15.63%,15.53%
2023-03,2083021.00,5.95%,8.84%,11.80%,14.21%,15.66%,15.84%,16.89%,17.37%,17.71%,17.21%,17.60%
2023-04,4317651.50,3.00%,5.96%,8.99%,11.16%,13.00%,14.27%,15.00%,15.87%,15.53%,15.95%,16.07%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1355156.00,1.49%,4.53%,6.40%,8.12%,9.74%,11.03%,12.15%,14.06%,13.64%,13.77%,13.97%
2023-05,844546.00,nan%,2.89%,5.42%,6.08%,7.15%,9.34%,10.49%,12.34%,12.09%,12.25%,12.98%
2023-06,1607377.00,2.68%,5.31%,9.50%,11.33%,13.05%,14.58%,15.82%,16.72%,17.53%,18.36%,18.58%
2023-07,3594170.00,3.06%,7.02%,10.66%,12.61%,12.99%,14.13%,15.55%,16.59%,17.48%,17.20%,16.76%
2023-08,3504454.00,3.02%,5.05%,7.01%,9.23%,10.67%,11.89%,13.32%,14.24%,15.10%,14.81%,14.67%
2023-09,3988756.12,2.23%,4.07%,5.14%,7.23%,8.13%,9.12%,10.87%,11.99%,12.60%,13.20%,12.93%
2023-10,5723013.25,2.23%,5.39%,7.98%,9.62%,11.17%,12.82%,14.39%,16.04%,16.17%,16.21%,16.55%
2023-11,7234215.00,1.79%,5.11%,6.60%,8.15%,10.34%,11.78%,12.70%,13.20%,13.62%,13.94%,13.93%
2023-12,3352399.00,1.97%,4.41%,5.69%,8.47%,9.58%,11.07%,12.39%,13.45%,14.43%,14.59%,14.55%
2024-01,2295409.00,1.28%,4.20%,6.14%,7.47%,9.79%,10.35%,11.70%,12.22%,12.58%,12.77%,12.97%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,1305173.00,3.34%,6.04%,11.12%,13.03%,13.22%,14.57%,15.29%,16.57%,15.92%,16.64%,16.71%
2023-05,925666.00,2.39%,9.73%,13.41%,17.67%,17.71%,17.88%,18.39%,16.95%,18.02%,18.09%,18.43%
2023-06,830115.00,2.57%,6.91%,15.36%,19.27%,22.34%,23.52%,23.77%,23.14%,24.00%,23.85%,23.77%
2023-07,1062805.00,4.30%,9.34%,12.33%,15.25%,18.86%,18.59%,18.58%,19.48%,19.23%,18.08%,17.78%
2023-08,1418637.00,4.53%,9.09%,12.14%,15.00%,16.88%,18.48%,19.19%,21.11%,21.36%,21.40%,20.90%
2023-09,910558.00,2.92%,6.59%,8.49%,11.33%,12.59%,14.61%,15.19%,16.55%,17.25%,16.76%,16.76%
2023-10,873717.50,3.44%,7.09%,11.02%,12.61%,16.57%,18.77%,19.72%,19.99%,20.20%,19.91%,20.42%
2023-11,1579325.00,1.50%,4.26%,7.72%,10.85%,12.69%,13.68%,13.76%,14.99%,15.05%,15.75%,15.66%
2023-12,1373729.75,nan%,2.39%,7.35%,7.99%,10.55%,11.57%,12.59%,12.16%,12.45%,12.83%,12.63%
2024-01,1021172.00,1.29%,3.09%,3.81%,6.82%,8.84%,10.08%,11.56%,12.47%,12.10%,12.53%,12.11%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,6840.00,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,79168.00,nan%,nan%,8.13%,8.13%,10.55%,10.55%,10.55%,13.23%,20.22%,13.23%,13.23%
2023-07,149045.00,4.78%,4.78%,4.78%,13.71%,13.71%,16.45%,13.71%,13.71%,15.31%,15.31%,15.31%
2023-08,126234.00,6.67%,11.88%,11.88%,11.88%,15.15%,15.15%,18.45%,18.45%,18.45%,17.51%,17.51%
2023-09,261610.00,8.46%,5.46%,7.92%,10.77%,8.18%,11.70%,11.70%,15.53%,17.58%,17.58%,17.36%
2023-10,474985.00,nan%,5.91%,9.68%,15.21%,17.51%,18.18%,17.88%,17.41%,17.30%,17.82%,17.82%
2023-11,1226260.00,3.70%,9.91%,13.34%,16.87%,18.69%,19.36%,20.59%,21.22%,20.95%,21.38%,21.18%
2023-12,2049806.25,1.74%,3.17%,6.33%,10.18%,12.12%,13.24%,14.45%,15.18%,15.92%,15.74%,15.71%
2024-01,1104144.00,2.48%,5.16%,7.22%,8.53%,9.73%,12.42%,12.57%,13.38%,13.32%,13.27%,13.30%
2024-06,39138.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2023-04,244612.0,2.32%,11.26%,11.26%,14.83%,15.35%,19.92%,17.85%,12.85%,12.85%,14.50%,14.50%
2023-05,208404.0,3.70%,7.12%,7.12%,3.70%,3.70%,7.20%,9.13%,9.13%,12.17%,11.70%,11.70%
2023-06,132717.0,nan%,nan%,3.60%,3.60%,3.60%,3.60%,3.60%,3.60%,3.60%,3.60%,3.60%
2023-07,277310.0,7.31%,14.30%,20.86%,18.82%,20.54%,22.34%,22.55%,22.55%,20.92%,20.32%,20.32%
2023-08,378466.0,12.26%,17.04%,18.94%,21.03%,19.19%,18.77%,18.28%,18.28%,17.99%,16.92%,16.84%
2023-09,356086.0,7.62%,9.67%,14.26%,15.25%,18.39%,17.49%,16.93%,15.17%,15.17%,14.83%,14.83%
2023-10,373112.0,7.53%,16.74%,24.60%,28.21%,23.58%,26.71%,26.56%,26.56%,26.56%,26.56%,26.56%
2023-11,710870.0,2.53%,8.63%,12.35%,14.41%,15.93%,16.28%,14.85%,15.40%,15.40%,15.69%,15.40%
2023-12,9010.0,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%,87.98%
2024-01,7750.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,11.88%,16.50%,19.07%,20.24%,24.82%,26.60%,28.13%,27.11%,27.11%,27.52%,27.70%
2024-07,448562.0,12.86%,20.26%,24.26%,26.74%,27.45%,29.56%,30.41%,30.63%,30.51%,31.27%,32.05%
2024-08,271022.0,18.03%,23.22%,22.73%,23.91%,24.67%,24.69%,26.56%,28.38%,28.69%,28.63%,nan%
2024-09,30173.0,15.33%,15.33%,26.51%,34.08%,33.35%,33.35%,33.35%,33.35%,37.30%,nan%,nan%
2024-11,9865.0,nan%,70.42%,70.42%,70.42%,70.42%,70.42%,70.42%,nan%,nan%,nan%,nan%
2024-12,105399.0,28.94%,30.01%,39.03%,40.15%,39.39%,43.85%,nan%,nan%,nan%,nan%,nan%
2025-01,95598.0,3.35%,10.96%,17.37%,19.37%,22.68%,nan%,nan%,nan%,nan%,nan%,nan%
2025-02,119574.0,6.62%,13.33%,19.53%,21.13%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,26.79%,26.24%,19.54%,19.54%,19.54%,23.14%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,nan%,7.87%,nan%,6.63%,6.01%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.28%,4.28%,4.28%,11.56%,7.18%,7.18%,9.88%,10.38%,9.71%,9.71%
2023-02,147754.00,10.68%,9.11%,12.75%,15.78%,15.78%,17.60%,21.58%,19.92%,20.94%,17.77%,17.77%
2023-03,226883.00,4.74%,6.23%,11.87%,16.88%,21.21%,19.84%,18.77%,20.74%,20.54%,17.99%,19.92%
2023-04,336845.50,1.24%,1.68%,7.24%,7.39%,7.39%,5.54%,6.07%,4.38%,5.06%,5.06%,5.55%


MOB,采购金额,mob_2,mob_3,mob_4
2024-12,236025.0,nan%,3.92%,3.92%
2025-01,269323.0,nan%,nan%,6.67%
2025-02,2132295.0,5.22%,9.97%,14.14%
2025-03,2499753.0,6.01%,12.59%,15.82%
2025-04,3521285.0,7.47%,14.13%,nan%
2025-05,2961684.0,5.90%,nan%,nan%
2025-06,2380146.0,nan%,nan%,nan%
2025-07,1674599.0,nan%,nan%,nan%
2025-08,149441.0,nan%,nan%,nan%


## 4_20.拒量

In [104]:
# df_mob_total.shape
# df_mob_total_tmp = df_mob_total.merge(df_reject_list[['order_number','是否拒量']], on = 'order_number', how = 'left')
# df_mob_total_tmp.shape


# df_mon2_no_jl = df_fq_concat[(df_fq_concat.当前期数==1)&(df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.是否拒量 != 1)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon2_no_jl 

# df_mon2_jl = df_fq_concat[(df_fq_concat.当前期数==1)&(df_fq_concat.是否拒量 == 1)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon2_jl

In [105]:
# # （1）总体-不含拒量-mob7+
# vintage_money_d7 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_7==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 != 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# # vintage_money_d7
# vintage_divide_money_d7 = vintage_money_d7.divide(df_mon2_no_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d7['采购金额'] = df_mon2_no_jl["purchase_amount"]
# vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [106]:
# # （2）总体-不含拒量-mob15+
# vintage_money_d15 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_15==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 != 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_divide_money_d15 = vintage_money_d15.divide(df_mon2_no_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d15['采购金额'] = df_mon2_no_jl["purchase_amount"]
# vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [107]:
# # （3）总体-不含拒量-mob30+
# vintage_money_d31 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_31==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 != 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_divide_money_d31 = vintage_money_d31.divide(df_mon2_no_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d31['采购金额'] = df_mon2_no_jl["purchase_amount"]
# vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [108]:
# # （4）拒量-mob7+
# vintage_money_d7 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_7==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 == 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3']]  #'下单月份','mob_0',
# # vintage_money_d7
# vintage_divide_money_d7 = vintage_money_d7.divide(df_mon2_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d7['采购金额'] = df_mon2_jl["purchase_amount"]
# vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3']]

In [109]:
# # （5）拒量-mob15+
# vintage_money_d15 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_15==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 == 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3']]  #'下单月份','mob_0',
# vintage_divide_money_d15 = vintage_money_d15.divide(df_mon2_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d15['采购金额'] = df_mon2_jl["purchase_amount"]
# vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3']]

In [110]:
# （6）拒量-mob30+
# vintage_money_d31 = pd.pivot_table(df_mob_total_tmp[(df_mob_total_tmp.overdue_days_31==1)& (df_mob_total_tmp.unpaid_money_mob > 0)  & (df_mob_total_tmp.归属渠道!='抖音渠道') & (df_mob_total_tmp.是否拒量 == 1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d31 = vintage_money_d31[['mob_2']]  #'下单月份','mob_0',
# vintage_divide_money_d31 = vintage_money_d31.divide(df_mon2_jl["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d31['采购金额'] = df_mon2_jl["purchase_amount"]
# vintage_divide_money_d31[['采购金额','mob_2']]

# 5.续租订单

<!-- 注意 20241205 20241206 修改非续租的内容 -->
<!-- code未修正 -->

## 5_11.数据处理

In [111]:
# （1）续租数据
df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
df_relet.shape

(68284, 172)

In [112]:
# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


In [113]:
# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)
df_mob_relet[:2]

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,MOB
0,A202207050851373,2022-07-05,354.0,0,354.0,mob_12
1,A2022072220482114,2022-07-22,0.0,9,495.0,mob_12


In [114]:
# （4）
df_mob_relet['overdue_days_1'] = np.where(df_mob_relet['overdue_days_mob']>= 1,1,0)
df_mob_relet['overdue_days_4'] = np.where(df_mob_relet['overdue_days_mob']>= 4,1,0)
df_mob_relet['overdue_days_7'] = np.where(df_mob_relet['overdue_days_mob']>= 7,1,0)
df_mob_relet['overdue_days_15'] = np.where(df_mob_relet['overdue_days_mob']>= 15,1,0)
df_mob_relet['overdue_days_31'] = np.where(df_mob_relet['overdue_days_mob']>= 31,1,0)
df_mob_relet['下单月份'] = df_mob_relet['下单日期'].dt.strftime('%Y-%m')  

In [115]:
# （5）
# note 20241210
# 非续租跨月的有mob0_mob12，正常的情况下只有mob0-mob11，故用实付金额（已把优惠金额计算在内），首租还完了才可以续租
# 正常续租&逾期续租，逾期续租的话，续租时间不确定，可能会导致分母变化，分母分子统一用续租的，对首租的逾期状态不延伸，首租逾期的不统计在续租范围内（如续租期间还首租的），因为续租的时候肯定是首租已经还清了的
# df_mob_total['采购金额'] 
df_mob_relet = df_mob_relet.merge(dfzh1[['order_number','search_time','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_sz_sh = df_xz_concat[df_xz_concat.is_relet == 0].groupby(by=['order_number']).agg({'实付金额':'sum'}).reset_index()
df_mob_relet.shape
# df_mob_relet[:2]

df_relet_mob10 = df_xz_concat[((df_xz_concat[f'mob_date_10'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_10']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob10':'sum'}).reset_index()

df_relet_mob11 = df_xz_concat[((df_xz_concat[f'mob_date_11'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_11']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob11':'sum'}).reset_index()

df_relet_mob12 = df_xz_concat[((df_xz_concat[f'mob_date_12'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_12']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob12':'sum'}).reset_index()

df_mob_relet = df_mob_relet.merge(df_sz_sh,on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob11[['order_number','paid_money_mob11']],on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob12[['order_number','paid_money_mob12']],on = 'order_number',how='left')
df_mob_relet[['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB','purchase_amount','paid_money_mob10','paid_money_mob11','paid_money_mob12','实付金额']][:2]

(46092, 15)

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,purchase_amount,paid_money_mob10,paid_money_mob11,paid_money_mob12,实付金额
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,5150.0,3937.45,4295.4,4649.4,4295.4
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,5890.0,3963.30,4323.6,4323.6,4323.6


In [116]:
# （6）数据检查  首租还完了才可以续租
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283  18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042  11630  2022-07-28 首次应付日期和下单日期一致 续租表现期已满
# A2022072220482114   2022-07-22 续租11期
# A20231008165443277  续租首月就逾期
# A20231008003444120 续租首月就逾期，首租的末期在续租的第一期还

# A2024072414134141
order_number_param = 'A2023070210395232'
var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','当前应付租金','实付金额','purchase_amount'] + var1]
df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB','purchase_amount','paid_money_mob10','paid_money_mob11','paid_money_mob12','实付金额']]

,order_number,下单日期,应付日期,实付日期new,当前应付租金,实付金额,purchase_amount,mob_date_0,mob_date_1,mob_date_2,mob_date_3,mob_date_4,mob_date_11,mob_date_12
42080,A2023070210395232,2023-07-02,2023-07-05,2023-07-02,1.00,1.00,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42081,A2023070210395232,2023-07-02,2023-08-05,2023-08-07,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42082,A2023070210395232,2023-07-02,2023-09-05,2023-09-15,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42083,A2023070210395232,2023-07-02,2023-10-05,2023-11-03,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42084,A2023070210395232,2023-07-02,2023-11-05,2023-12-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42085,A2023070210395232,2023-07-02,2023-12-05,2023-12-25,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42086,A2023070210395232,2023-07-02,2024-01-05,2024-02-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42087,A2023070210395232,2023-07-02,2024-02-05,2024-03-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42088,A2023070210395232,2023-07-02,2024-03-05,2024-03-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42089,A2023070210395232,2023-07-02,2024-04-05,2024-04-12,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31


,order_number,order_id,下单日期,状态,应付日期,当前应付租金,实付日期new,实付金额,逾期天数,mob_date_0,mob_date_11,mob_date_12,mob_date_13,mob_date_21,mob_date_23
42092,A2023070210395232,432132,2023-07-02,已逾期支付,2024-07-04,583.88,2024-07-10,583.88,6,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42093,A2023070210395232,432132,2023-07-02,已逾期支付,2024-08-04,583.88,2024-08-17,583.88,13,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42094,A2023070210395232,432132,2023-07-02,已逾期支付,2024-09-04,583.88,2024-10-02,583.88,28,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42095,A2023070210395232,432132,2023-07-02,已逾期支付,2024-10-04,583.88,2024-10-20,583.88,16,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42096,A2023070210395232,432132,2023-07-02,已逾期支付,2024-11-04,583.88,2024-11-16,583.88,12,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42097,A2023070210395232,432132,2023-07-02,已逾期支付,2024-12-04,583.88,2024-12-13,583.88,9,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42098,A2023070210395232,432132,2023-07-02,已逾期支付,2025-01-04,583.88,2025-01-18,583.88,14,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42099,A2023070210395232,432132,2023-07-02,已逾期支付,2025-02-04,583.88,2025-02-18,583.88,14,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42100,A2023070210395232,432132,2023-07-02,已逾期支付,2025-03-04,583.88,2025-03-16,583.88,12,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42101,A2023070210395232,432132,2023-07-02,已逾期支付,2025-04-04,583.88,2025-04-15,583.88,11,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30


,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,purchase_amount,paid_money_mob10,paid_money_mob11,paid_money_mob12,实付金额
747,A2023070210395232,2023-07-02,583.88,583.88,0,mob_12,7240.0,4859.2,5400.4,6524.08,5940.2
6958,A2023070210395232,2023-07-02,1167.76,1167.76,0,mob_13,7240.0,4859.2,5400.4,6524.08,5940.2
12757,A2023070210395232,2023-07-02,1751.64,1167.76,26,mob_14,7240.0,4859.2,5400.4,6524.08,5940.2
17896,A2023070210395232,2023-07-02,2335.52,2335.52,0,mob_15,7240.0,4859.2,5400.4,6524.08,5940.2
22581,A2023070210395232,2023-07-02,2919.40,2919.40,0,mob_16,7240.0,4859.2,5400.4,6524.08,5940.2
26650,A2023070210395232,2023-07-02,3503.28,3503.28,0,mob_17,7240.0,4859.2,5400.4,6524.08,5940.2
30431,A2023070210395232,2023-07-02,4087.16,4087.16,0,mob_18,7240.0,4859.2,5400.4,6524.08,5940.2
34050,A2023070210395232,2023-07-02,4671.04,4671.04,0,mob_19,7240.0,4859.2,5400.4,6524.08,5940.2
37349,A2023070210395232,2023-07-02,5254.92,5254.92,0,mob_20,7240.0,4859.2,5400.4,6524.08,5940.2
40243,A2023070210395232,2023-07-02,5838.80,5838.80,0,mob_21,7240.0,4859.2,5400.4,6524.08,5940.2


## 5_12.每月对应的续租用户及采购金额

In [117]:
# （1）
# 废弃这个代码
# df_relet1 = df_relet[df_relet.当前期数==13].merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
# df_relet_mon = df_relet1.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum','paid_money_mob10_y':'sum'})
# df_relet_mon['relet_principal'] = df_relet_mon['purchase_amount'] - df_relet_mon['paid_money_mob10_y']
# df_relet_mon

# # A2024072414134141 的 relet_principal 不对

In [118]:
# （2）数据检查
# note 20241210 修改
# 续租有些mob12开始，有些mob13开始

# 数据检查，跑数日期20241210，续租的第一期还款日期>=202412的未统计在内
# tmp = df_relet[df_relet.当前期数==13][['order_number','下单日期','search_time']]
# tmp1 = df_mob_relet.drop_duplicates(subset = 'order_number')[['order_number']]
# tmp1['label'] = 1
# tmp.shape
# tmp1.shape

# tmp2 = tmp.merge(tmp1, on = 'order_number', how = 'left')
# tmp2[tmp2.label.isna()].shape
# # A2023112013061723
# # A20231122095801122
# # A2024040211571596
# # A2024072414134141
# tmp2[tmp2.label.isna()].下单日期.min()
# tmp2[tmp2.label.isna()].下单日期.max()
# tmp2[tmp2.label.isna()].search_time.value_counts()
# tmp2[tmp2.label.isna()].tail()

# df_mob_relet[df_mob_relet.order_number == 'A2023120500472233'][['order_number','overdue_days_mob']]


In [119]:
# （3）拆分续租期数
relet_tmp = df_xz_concat[df_xz_concat.is_relet>=1][['order_number','应付日期']]
relet_tmp = relet_tmp.drop_duplicates(subset = ['order_number','应付日期'])
relet_tmp_1 = relet_tmp.groupby(['order_number']).agg({'应付日期': 'count'}).reset_index()
relet_tmp_1.shape
relet_tmp_1 = relet_tmp_1.rename(columns = {'应付日期': 'relet_periods'})
relet_tmp_1.tail(2)
relet_tmp_1.relet_periods.min()
relet_tmp_1.relet_periods.max()
relet_tmp_1[relet_tmp_1.relet_periods <= 6].shape
relet_tmp_1[relet_tmp_1.relet_periods > 6].shape

# 以下是数据检查，租后监控表的 续租期数 不能用，用资方分期表加工续租期数
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number, tprm.order_id, tprm.relet_periods
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where tprm.order_number = 'A202404291003438'
# '''
# query(sql)

# A202207050851373   资方分期表显示12期，租后监控表显示 续租期数是11
# A2022072220482114  资方分期表显示11期，租后监控表显示 续租期数是11
# A2024040211571596  续租提前买断，6期，租后监控表显示 续租期数是0
# A202404291003438   资方分期表为 8 期，租后监控表 3期
# A2024072414134141  资方分期表为 6 期，租后监控表 0期
# df_xz_concat[(df_xz_concat.order_number == 'A202404291003438')][['order_number','下单日期','应付日期','实付日期new','实付金额','is_relet']]
# df_xz_concat[(df_xz_concat.is_relet>=1)][['order_number']].tail(20)
# relet_tmp_1[relet_tmp_1.order_number == 'A2024072414134141']

df_mob_relet = df_mob_relet.merge(relet_tmp_1, on = 'order_number', how = 'left')
df_mob_relet.shape

(6776, 2)

,order_number,relet_periods
6774,A2024081417192266,6
6775,A20250213163349247,6


np.int64(3)

np.int64(24)

(1097, 2)

(5679, 2)

(46092, 20)

In [120]:
# （4）
df_relet_mon_tmp = df_mob_relet.sort_values(by = 'MOB', ascending = True)
df_relet_mon_tmp = df_relet_mon_tmp.drop_duplicates(subset = 'order_number')
df_relet_mon_tmp[df_relet_mon_tmp.实付金额 > df_relet_mon_tmp.purchase_amount].shape
df_relet_mon_tmp.shape
# 部份分布在mob_13
df_relet_mon_tmp['MOB'].value_counts()

# relet_principal：采购金额-首租已还租金=续租本金
# 续租总体
df_relet_mon = df_relet_mon_tmp.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon['relet_principal'] = np.where((df_relet_mon['purchase_amount'] - df_relet_mon['实付金额'])>0,df_relet_mon['purchase_amount'] - df_relet_mon['实付金额'],0)
df_relet_mon = df_relet_mon.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon

# 续租6期及以内
df_relet_mon_6= df_relet_mon_tmp[df_relet_mon_tmp.relet_periods <= 6].groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon_6['relet_principal'] = np.where((df_relet_mon_6['purchase_amount'] - df_relet_mon_6['实付金额'])>0,df_relet_mon_6['purchase_amount'] - df_relet_mon_6['实付金额'],0)
df_relet_mon_6 = df_relet_mon_6.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon_6

# 续租6期以上
df_relet_mon_12= df_relet_mon_tmp[df_relet_mon_tmp.relet_periods > 6].groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon_12['relet_principal'] = np.where((df_relet_mon_12['purchase_amount'] - df_relet_mon_12['实付金额'])>0,df_relet_mon_12['purchase_amount'] - df_relet_mon_12['实付金额'],0)
df_relet_mon_12 = df_relet_mon_12.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon_12

(117, 20)

(6598, 20)

MOB
mob_12    6152
mob_13     446
Name: count, dtype: int64

,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2022-07,5,22561.42,29187.00,6625.58
2022-08,15,72075.07,98008.00,25932.93
2022-09,14,76425.12,92730.85,16305.73
2022-10,15,71732.50,87371.00,15638.50
2022-11,17,92272.70,132745.00,40472.30
2022-12,21,129087.10,177135.00,48047.90
2023-01,61,304912.80,391326.00,86413.20
2023-02,72,356934.95,457244.00,100309.05
2023-03,69,336251.37,472351.00,136099.63


,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2023-01,59,297922.00,380406.0,82484.00
2023-02,70,347883.00,444794.0,96911.00
2023-03,68,332402.59,464309.0,131906.41
2023-04,109,524505.81,742049.0,217543.19
2023-05,15,70356.82,96510.0,26153.18
2023-07,1,5874.20,7420.0,1545.80
2024-01,10,57699.40,71620.0,13920.60
2024-02,27,151619.32,188460.0,36840.68
2024-03,107,623501.76,792520.0,169018.24


,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2022-07,5,22561.42,29187.00,6625.58
2022-08,15,72075.07,98008.00,25932.93
2022-09,14,76425.12,92730.85,16305.73
2022-10,15,71732.50,87371.00,15638.50
2022-11,17,92272.70,132745.00,40472.30
2022-12,21,129087.10,177135.00,48047.90
2023-01,2,6990.80,10920.00,3929.20
2023-02,2,9051.95,12450.00,3398.05
2023-03,1,3848.78,8042.00,4193.22


## 5_13.续租订单 vintage 4+ 订单口径

In [121]:
# （1）续租总体
vintage_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
vintage_relet_d4 
 #.reset_index()  #,margins=True

vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
# vintage_relet_d4 

# vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
# vintage_relet_d4 

# vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,NaN,2.0,1.0
2022-08,2.0,2.0,3.0,4.0,4.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-09,5.0,2.0,3.0,4.0,5.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,2.0,3.0,4.0,5.0,4.0,5.0,5.0,4.0,3.0,3.0,3.0
2022-11,NaN,1.0,2.0,3.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,4.0,4.0
2022-12,2.0,2.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2023-01,3.0,8.0,14.0,15.0,17.0,18.0,17.0,13.0,13.0,13.0,14.0,14.0,14.0
2023-02,7.0,13.0,15.0,20.0,22.0,26.0,20.0,18.0,18.0,18.0,18.0,18.0,18.0
2023-03,6.0,9.0,13.0,11.0,17.0,20.0,16.0,16.0,15.0,15.0,15.0,15.0,15.0


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,13.11%,22.95%,24.59%,27.87%,29.51%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,9.72%,18.06%,20.83%,27.78%,30.56%,36.11%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,8.70%,13.04%,18.84%,15.94%,24.64%,28.99%,23.19%,23.19%,21.74%,21.74%,21.74%,21.74%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2023-01,59,5.08%,13.56%,23.73%,25.42%,27.12%,28.81%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,8.57%,17.14%,20.00%,27.14%,28.57%,34.29%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,8.82%,13.24%,19.12%,16.18%,25.00%,29.41%,23.53%,23.53%,22.06%,22.06%,22.06%,22.06%
2023-04,109,9.17%,17.43%,22.02%,28.44%,33.94%,34.86%,31.19%,29.36%,28.44%,27.52%,27.52%,27.52%
2023-05,15,6.67%,20.00%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%
2024-01,10,nan%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,nan%,nan%,nan%,nan%,nan%
2024-02,27,nan%,11.11%,11.11%,11.11%,11.11%,22.22%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,107,5.61%,8.41%,9.35%,14.02%,20.56%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,17.86%,14.29%,17.86%,17.86%,14.29%,28.57%,28.57%,28.57%,32.14%,35.71%,35.71%,39.29%


## 5_14.续租订单 vintage 7+ 订单口径

In [122]:
# （1）续租总体
vintage_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum') 
# vintage_relet_d7 

# vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum') 
# vintage_relet_d7 

# vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,13.11%,21.31%,24.59%,27.87%,27.87%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,8.33%,16.67%,18.06%,25.00%,26.39%,33.33%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,7.25%,11.59%,17.39%,14.49%,23.19%,28.99%,23.19%,23.19%,21.74%,21.74%,21.74%,21.74%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,5.08%,13.56%,22.03%,25.42%,27.12%,27.12%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,7.14%,15.71%,17.14%,24.29%,24.29%,31.43%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,7.35%,11.76%,17.65%,14.71%,23.53%,29.41%,23.53%,23.53%,22.06%,22.06%,22.06%,22.06%
2023-04,109,8.26%,17.43%,21.10%,28.44%,33.94%,33.94%,31.19%,29.36%,28.44%,27.52%,27.52%,27.52%
2023-05,15,6.67%,20.00%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%
2024-01,10,nan%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,nan%,nan%,nan%,nan%,nan%
2024-02,27,nan%,7.41%,7.41%,7.41%,7.41%,22.22%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,107,4.67%,5.61%,8.41%,11.21%,19.63%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,303,3.96%,6.93%,11.55%,15.51%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,17.86%,14.29%,17.86%,17.86%,14.29%,28.57%,28.57%,28.57%,32.14%,35.71%,35.71%,39.29%


## 5_15.续租订单 vintage 15+ 订单口径

In [123]:
# （1）续租总体
vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')
vintage_relet_d15
# vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum') 
# vintage_relet_d15 

# vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum') 
# vintage_relet_d15 

# vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,NaN,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN
2022-08,1.0,1.0,3.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0
2022-09,NaN,1.0,2.0,2.0,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,NaN,3.0,2.0,5.0,4.0,5.0,4.0,3.0,3.0,3.0,3.0
2022-11,NaN,NaN,2.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-12,NaN,2.0,2.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0
2023-01,1.0,6.0,11.0,15.0,16.0,15.0,17.0,13.0,13.0,13.0,14.0,14.0,14.0
2023-02,4.0,9.0,13.0,16.0,18.0,23.0,20.0,18.0,18.0,18.0,18.0,18.0,18.0
2023-03,3.0,5.0,9.0,8.0,13.0,18.0,16.0,16.0,15.0,15.0,15.0,15.0,15.0


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,1.64%,9.84%,18.03%,24.59%,26.23%,24.59%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,5.56%,12.50%,18.06%,22.22%,25.00%,31.94%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,4.35%,7.25%,13.04%,11.59%,18.84%,26.09%,23.19%,23.19%,21.74%,21.74%,21.74%,21.74%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,1.69%,10.17%,18.64%,25.42%,27.12%,25.42%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,4.29%,11.43%,17.14%,21.43%,22.86%,30.00%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,4.41%,7.35%,13.24%,11.76%,19.12%,26.47%,23.53%,23.53%,22.06%,22.06%,22.06%,22.06%
2023-04,109,4.59%,15.60%,18.35%,20.18%,28.44%,32.11%,31.19%,29.36%,28.44%,27.52%,27.52%,27.52%
2023-05,15,6.67%,13.33%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%
2023-07,1,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%
2024-01,10,nan%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,nan%,nan%,nan%,nan%,nan%
2024-02,27,nan%,3.70%,7.41%,7.41%,7.41%,14.81%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,107,0.93%,3.74%,4.67%,8.41%,14.95%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,303,1.98%,5.28%,10.23%,12.87%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,10.71%,10.71%,17.86%,14.29%,14.29%,17.86%,28.57%,28.57%,32.14%,32.14%,35.71%,39.29%


## 5_16.续租订单 vintage 31+ 订单口径

In [124]:
# （1）
vintage_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
vintage_relet_d31

 #.reset_index()  #,margins=True

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23''mob_11',
# vintage_df_d4
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
# vintage_relet_d31 

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon_6["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
# vintage_relet_d31 

# vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,
2022-07,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN
2022-08,1.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0
2022-09,NaN,2.0,2.0,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,1.0,2.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
2022-11,NaN,1.0,2.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-12,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0
2023-01,1.0,7.0,12.0,12.0,13.0,16.0,13.0,13.0,13.0,13.0,14.0,14.0
2023-02,6.0,10.0,13.0,16.0,22.0,20.0,18.0,18.0,17.0,18.0,18.0,18.0
2023-03,3.0,8.0,7.0,10.0,17.0,16.0,16.0,15.0,15.0,15.0,15.0,15.0


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%
2023-01,61,1.64%,11.48%,19.67%,19.67%,21.31%,26.23%,21.31%,21.31%,21.31%,21.31%,22.95%
2023-02,72,8.33%,13.89%,18.06%,22.22%,30.56%,27.78%,25.00%,25.00%,23.61%,25.00%,25.00%
2023-03,69,4.35%,11.59%,10.14%,14.49%,24.64%,23.19%,23.19%,21.74%,21.74%,21.74%,21.74%
2023-04,137,8.03%,13.14%,16.06%,21.90%,26.28%,29.93%,29.20%,28.47%,27.74%,28.47%,28.47%


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,1.69%,11.86%,20.34%,20.34%,22.03%,25.42%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,7.14%,12.86%,17.14%,21.43%,28.57%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,4.41%,11.76%,10.29%,14.71%,25.00%,23.53%,23.53%,22.06%,22.06%,22.06%,22.06%
2023-04,109,7.34%,11.93%,16.51%,23.85%,28.44%,30.28%,29.36%,28.44%,27.52%,27.52%,27.52%
2023-05,15,13.33%,20.00%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%
2024-01,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,nan%,nan%,nan%,nan%,nan%
2024-02,27,nan%,7.41%,7.41%,7.41%,11.11%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,107,2.80%,3.74%,5.61%,12.15%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,303,2.97%,7.26%,10.23%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,10.71%,17.86%,14.29%,14.29%,17.86%,28.57%,28.57%,28.57%,28.57%,32.14%,32.14%


## 5_17.续租订单 vintage 4+ 金额口径

In [125]:
# （1）续租总体
# df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['paid_money_mob'] - df_mob_relet['paid_money_mob10']
# note 20241210
df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['实付金额'] - df_mob_relet['paid_money_mob']
# money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 

# money_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23'
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4
# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,23.64%,45.06%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,1.43%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,38.04%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,13.47%,12.06%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,19.62%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,5.25%,6.98%,7.95%,6.51%,5.64%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,9.05%,9.44%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,8.21%,9.97%,8.92%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,5.50%,7.31%,8.33%,6.82%,5.90%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,7.37%,7.77%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,8.47%,10.29%,9.20%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%
2023-04,217543.19,9.17%,9.90%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,6.64%,6.64%
2023-05,26153.18,7.42%,13.88%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%
2023-07,1545.80,100.00%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,13920.60,nan%,nan%,4.90%,4.90%,4.90%,4.90%,4.90%,nan%,nan%,nan%,nan%,nan%
2024-02,36840.68,nan%,3.25%,2.59%,2.59%,2.59%,2.59%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,169018.24,6.93%,4.41%,2.84%,2.77%,2.77%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,496101.91,4.69%,4.25%,4.28%,3.83%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,1.43%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,38.04%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,13.47%,12.06%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,19.62%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,22.87%,18.86%,21.24%,13.39%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,10.27%,10.27%


## 5_18.续租订单 vintage 7+ 金额口径

In [126]:
# （1）
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d7
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,23.64%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,5.82%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,13.47%,11.97%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,17.36%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,5.25%,6.98%,7.95%,6.51%,5.64%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,7.45%,8.98%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,6.37%,8.27%,8.92%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,5.50%,7.31%,8.33%,6.82%,5.90%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,5.71%,7.30%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,6.57%,8.54%,9.20%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%
2023-04,217543.19,8.10%,9.90%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,6.64%,6.64%
2023-05,26153.18,7.42%,13.88%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%
2023-07,1545.80,100.00%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,13920.60,nan%,nan%,4.90%,4.90%,4.90%,4.90%,4.90%,nan%,nan%,nan%,nan%,nan%
2024-02,36840.68,nan%,2.59%,2.59%,2.59%,2.59%,2.59%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,169018.24,5.95%,4.04%,2.84%,2.77%,2.77%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,496101.91,4.00%,4.04%,4.28%,3.83%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,5.82%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,13.47%,11.97%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,17.36%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,22.87%,18.86%,21.24%,13.39%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,10.27%,10.27%


## 5_19.续租订单 vintage 15+ 金额口径

In [127]:
# （1）
money_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15
#.reset_index()  #,margins=True

money_relet_d15 = money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]   #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d15 = money_relet_d15.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d15= pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d15= money_relet_d15.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d15
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d15= pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d15= money_relet_d15.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,NaN,1566.40,2985.40,1566.40,1566.40,1566.40,1071.40,1071.40,1071.40,NaN,NaN,NaN,NaN
2022-08,2110.01,NaN,3875.24,4290.44,3922.04,3875.24,3875.24,3875.24,3875.24,3365.24,3365.24,3365.24,3365.24
2022-09,NaN,1145.00,3042.68,3042.68,3042.68,3042.68,3042.68,3042.68,2598.68,2598.68,2598.68,2598.68,2598.68
2022-10,NaN,NaN,NaN,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86
2022-11,NaN,NaN,4366.90,4366.90,4843.30,2934.70,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40
2022-12,NaN,6698.50,6698.50,8764.30,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90
2023-01,1737.00,4915.40,6753.01,5624.61,4870.61,3681.61,3681.61,3681.61,3681.61,2751.61,2751.61,2751.61,2751.61
2023-02,4751.65,8132.54,8802.12,7724.12,7612.02,7612.02,4970.55,4572.95,4572.95,4572.95,4572.95,4572.95,4572.95
2023-03,3057.50,8515.64,11261.47,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,nan%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,7.02%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,10.79%,11.97%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,13.94%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,2.01%,5.69%,7.81%,6.51%,5.64%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,4.74%,8.11%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,2.25%,6.26%,8.27%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,2.11%,5.96%,8.19%,6.82%,5.90%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,2.91%,6.40%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,2.32%,6.46%,8.54%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%
2023-04,217543.19,5.36%,9.45%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,6.64%,6.64%
2023-05,26153.18,7.42%,13.72%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%
2023-07,1545.80,nan%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,13920.60,nan%,nan%,4.90%,4.90%,4.90%,4.90%,4.90%,nan%,nan%,nan%,nan%,nan%
2024-02,36840.68,nan%,2.59%,2.59%,2.59%,2.59%,2.59%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,169018.24,1.23%,3.62%,2.84%,2.77%,2.77%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,496101.91,1.93%,3.65%,4.28%,3.83%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,nan%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,7.02%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,10.79%,11.97%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,13.94%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,14.77%,14.77%,21.24%,11.66%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,10.27%,10.27%


## 5_20.续租订单 vintage 31+ 金额口径

In [128]:
# （1）
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d31

 #.reset_index()  #,margins=True

money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23''mob_11',
# vintage_df_d4
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
# money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']] 
money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon_6["relet_principal"]
# money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d31
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


MOB,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,
2022-07,1566.40,2985.40,1566.40,1566.40,1566.40,1071.40,1071.40,1071.40,NaN,NaN,NaN,NaN
2022-08,NaN,2647.23,3875.24,3875.24,3875.24,3875.24,3875.24,3875.24,3365.24,3365.24,3365.24,3365.24
2022-09,NaN,3042.68,3042.68,3042.68,3042.68,3042.68,3042.68,2598.68,2598.68,2598.68,2598.68,2598.68
2022-10,NaN,NaN,NaN,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86
2022-11,NaN,4366.90,4366.90,4366.90,2934.70,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40
2022-12,6698.50,6698.50,7782.70,6698.50,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90
2023-01,1737.00,5845.40,5624.61,4870.61,3681.61,3681.61,3681.61,3681.61,2751.61,2751.61,2751.61,2751.61
2023-02,7961.42,8458.38,7724.12,7214.42,7612.02,4970.55,4572.95,4572.95,4572.95,4572.95,4572.95,4572.95
2023-03,4275.80,11261.47,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,nan%,10.21%,14.94%,14.94%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,10.79%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,16.20%,13.94%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,2.01%,6.76%,6.51%,5.64%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,7.94%,8.43%,7.70%,7.19%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,3.14%,8.27%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%
2023-04,273499.31,8.14%,10.91%,9.04%,8.42%,7.74%,7.46%,7.38%,7.38%,7.38%,7.38%,7.38%


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,2.11%,7.09%,6.82%,5.90%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,6.22%,6.73%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,3.24%,8.54%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%
2023-04,217543.19,6.44%,8.26%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,6.64%,6.64%
2023-05,26153.18,13.72%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%,7.59%
2023-07,1545.80,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,13920.60,nan%,4.90%,4.90%,4.90%,4.90%,4.90%,nan%,nan%,nan%,nan%,nan%
2024-02,36840.68,nan%,2.59%,2.59%,2.59%,2.59%,nan%,nan%,nan%,nan%,nan%,nan%
2024-03,169018.24,3.36%,2.64%,2.77%,2.77%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-04,496101.91,3.10%,4.15%,3.83%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,nan%,10.21%,14.94%,14.94%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,10.79%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,16.20%,13.94%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,41.94%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,14.77%,21.24%,11.66%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,10.27%,10.27%


In [129]:
# df_mob_relet[df_mob_relet.下单月份 == '2023-01'].to_excel('tmp.xlsx', index = False)

# 6.每月还款状态

## 6_11.每月截面数据

In [130]:
# unrecovered_cost overdue_type purchase_amount 来源租后应收监控表 t_postlease_receivables_monitoring T+1更新，最新的数据
# 未回收成本 = 采购成本 - 已收总租金  -- 不成立
# note 20241227
# 未回收成本 = 采购金额 - 总已收（total_received） 
# 总已收（total_received） = 已收租金 + 溢价费 + ......  
# 未回收成本已经剔除了溢价费，但未剔除碎屏险，故减去碎屏险
 
dfzh3["下单月份"] = dfzh3['search_time']
dfzh3["未回收成本"] = dfzh3['unrecovered_cost']
dfzh3["逾期类型"] = dfzh3['overdue_type']
dfzh3["采购金额"] = dfzh3['purchase_amount']

In [131]:
# （1）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o[df_s_o.price > 0].shape
# df_s_o[(df_s_o.price > 0)&(df_s_o.special_price > 0)].shape
# df_s_o[(df_s_o.price < df_s_o.special_price)].shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape
# df_s_o[(df_s_o.price>0)&(df_s_o.special_price>0)].tail()

dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

# （2）溢价费 -- 未回收成本中已经减掉了 溢价费
# sql = ''' 
# SELECT  distinct product_id,commodity_premium
# from db_digua_business.t_product_sku
# '''
# df_p_s = query(sql)
# df_p_s.shape
# count(*)	count(distinct product_id)
# 	319176	6868
# df_p_s.product_id.value_counts()
# df_p_s[df_p_s.product_id == 4321]

# dfzh3['commodity_premium'] = dfzh3['commodity_premium'].fillna(0)

# （3）押金
# 2025.3.3
# `residue_money` double(10,2) DEFAULT NULL COMMENT '剩余未使用押金金额'
sql = ''' 
SELECT  distinct order_id,residue_money
from db_digua_business.t_order_pledge
where residue_money > 0
'''
df_p = query(sql)
df_p.shape
df_p.drop_duplicates(subset = ['order_id']).shape

dfzh3 = dfzh3.merge(df_p, on = 'order_id', how = 'left')
dfzh3['residue_money'] = dfzh3['residue_money'].fillna(0)
dfzh3.shape

(304275, 2)

(304275, 2)

(301999, 2)

(58624, 98)

(58624, 99)

(2994, 2)

(2994, 2)

(58624, 100)

In [132]:
# 更新未回收成本
dfzh3["未回收成本"] = dfzh3.未回收成本 - dfzh3.price - dfzh3.residue_money

In [133]:
# （1）总体
# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180
# 逾期类型：订单当前逾期类型
# note 20241127 续租情况也包含在内 A2023052816303242

df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]
# dfyq1.iloc[:,1:]#

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,36504.48,36504.48,1.254050e+05,29.11%,29.11%,29.11%,29.11%
2022-08,0.00,0.00,0.00,0.00,102487.97,102487.97,3.584510e+05,28.59%,28.59%,28.59%,28.59%
2022-09,0.00,0.00,0.00,0.00,52385.30,52385.30,2.779828e+05,18.84%,18.84%,18.84%,18.84%
2022-10,0.00,0.00,0.00,0.00,43583.45,43583.45,2.495431e+05,17.47%,17.47%,17.47%,17.47%
2022-11,0.00,0.00,0.00,0.00,48077.70,48077.70,3.442588e+05,13.97%,13.97%,13.97%,13.97%
2022-12,0.00,0.00,0.00,0.00,92665.70,92665.70,5.662766e+05,16.36%,16.36%,16.36%,16.36%
2023-01,0.00,0.00,0.00,0.00,241567.20,241567.20,1.533459e+06,15.75%,15.75%,15.75%,15.75%
2023-02,0.00,0.00,0.00,0.00,348485.92,348485.92,2.181553e+06,15.97%,15.97%,15.97%,15.97%
2023-03,0.00,0.00,0.00,0.00,364328.14,364328.14,2.083021e+06,17.49%,17.49%,17.49%,17.49%


In [134]:
# （2）总体非抖音非拒量非号卡
conditions_all_2 = ((dfzh3.归属渠道!='抖音渠道')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_zm = ((dfzh3.归属渠道=='芝麻租物')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_ss = ((dfzh3.归属渠道=='搜索渠道')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_drhh = ((dfzh3.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_sl = ((dfzh3.归属渠道=='S量')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_dy = (dfzh3.归属渠道=='抖音渠道')
conditions_esfdy = ((dfzh3["是否二手"]=="二手")&(dfzh3.归属渠道!='抖音渠道')&(dfzh3.是否拒量 != 1)&(dfzh3.是否号卡 != 1))
conditions_jl = (dfzh3.是否拒量 == 1)

df_zw = dfzh3[conditions_all_2]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,36504.48,36504.48,1.254050e+05,29.11%,29.11%,29.11%,29.11%
2022-08,0.00,0.00,0.00,0.00,102487.97,102487.97,3.584510e+05,28.59%,28.59%,28.59%,28.59%
2022-09,0.00,0.00,0.00,0.00,52385.30,52385.30,2.779828e+05,18.84%,18.84%,18.84%,18.84%
2022-10,0.00,0.00,0.00,0.00,43583.45,43583.45,2.495431e+05,17.47%,17.47%,17.47%,17.47%
2022-11,0.00,0.00,0.00,0.00,48077.70,48077.70,3.442588e+05,13.97%,13.97%,13.97%,13.97%
2022-12,0.00,0.00,0.00,0.00,92665.70,92665.70,5.662766e+05,16.36%,16.36%,16.36%,16.36%
2023-01,0.00,0.00,0.00,0.00,241567.20,241567.20,1.533459e+06,15.75%,15.75%,15.75%,15.75%
2023-02,0.00,0.00,0.00,0.00,348485.92,348485.92,2.181553e+06,15.97%,15.97%,15.97%,15.97%
2023-03,0.00,0.00,0.00,0.00,364328.14,364328.14,2.083021e+06,17.49%,17.49%,17.49%,17.49%


In [135]:
# 跟账龄对比
# df_mon = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.purchase_amount > 0)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'}) 
# vintage_money_d1 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_1==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

# vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
# vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [136]:
# （3）芝麻租物
# df_zw = dfzh3[dfzh3["归属渠道"]=="芝麻租物"]
df_zw = dfzh3[conditions_zm]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-04,0.00,0.00,0.00,0.00,251406.28,251406.28,1.355156e+06,18.55%,18.55%,18.55%,18.55%
2023-05,0.00,0.00,0.00,0.00,130916.69,130916.69,8.445460e+05,15.50%,15.50%,15.50%,15.50%
2023-06,0.00,0.00,0.00,0.00,324819.65,324819.65,1.607377e+06,20.21%,20.21%,20.21%,20.21%
2023-07,0.00,0.00,0.00,0.00,789486.89,789486.89,3.600770e+06,21.93%,21.93%,21.93%,21.93%
2023-08,0.00,0.00,0.00,0.00,651299.97,651299.97,3.504454e+06,18.58%,18.58%,18.58%,18.58%
2023-09,0.00,0.00,0.00,20947.56,672674.44,693622.00,3.988756e+06,17.39%,17.39%,17.39%,16.86%
2023-10,0.00,0.00,0.00,17393.97,1075206.02,1092599.99,5.723013e+06,19.09%,19.09%,19.09%,18.79%
2023-11,0.00,0.00,1493.71,4653.73,1056872.01,1063019.45,7.234215e+06,14.69%,14.69%,14.67%,14.61%
2023-12,0.00,1052.53,0.00,1478.78,509669.86,512201.17,3.352399e+06,15.28%,15.28%,15.25%,15.20%


In [137]:
# （4）搜索渠道
# df_ss = dfzh3[dfzh3["归属渠道"]=="搜索渠道"]  
df_ss = dfzh3[conditions_ss]  
df_ss['M0'] = np.where(df_ss.逾期类型=='M0',df_ss["未回收成本"],0)
df_ss['M1'] = np.where(df_ss.逾期类型=='M1',df_ss["未回收成本"],0)
df_ss['M2'] = np.where(df_ss.逾期类型=='M2',df_ss["未回收成本"],0)
df_ss['M3'] = np.where(df_ss.逾期类型=='M3',df_ss["未回收成本"],0)
df_ss['M4'] = np.where(df_ss.逾期类型=='M4',df_ss["未回收成本"],0)
df_ss['M5'] = np.where(df_ss.逾期类型=='M5',df_ss["未回收成本"],0)
df_ss['M6'] = np.where(df_ss.逾期类型=='M6',df_ss["未回收成本"],0)

dfyq = df_ss[df_ss["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_ss[df_ss["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_ss.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-04,0.00,0.00,0.00,0.00,221601.26,221601.26,1.305173e+06,16.98%,16.98%,16.98%,16.98%
2023-05,0.00,0.00,0.00,0.00,159571.88,159571.88,9.256660e+05,17.24%,17.24%,17.24%,17.24%
2023-06,0.00,0.00,0.00,0.00,186634.94,186634.94,8.301150e+05,22.48%,22.48%,22.48%,22.48%
2023-07,0.00,0.00,0.00,0.00,199237.39,199237.39,1.062805e+06,18.75%,18.75%,18.75%,18.75%
2023-08,0.00,0.00,0.00,0.00,291619.97,291619.97,1.418637e+06,20.56%,20.56%,20.56%,20.56%
2023-09,0.00,0.00,0.00,2057.93,164221.69,166279.62,9.105580e+05,18.26%,18.26%,18.26%,18.04%
2023-10,0.00,0.00,0.00,0.00,197457.08,197457.08,8.737175e+05,22.60%,22.60%,22.60%,22.60%
2023-11,0.00,0.00,0.00,0.00,246848.71,246848.71,1.579325e+06,15.63%,15.63%,15.63%,15.63%
2023-12,0.00,0.00,0.00,0.00,185189.76,185189.76,1.373730e+06,13.48%,13.48%,13.48%,13.48%


In [138]:
# （5）单人会话
# note 20241209 增加
# df_zw = dfzh3[(dfzh3.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')]
df_zw = dfzh3[conditions_drhh]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-06,0.00,0.00,0.00,0.00,10476.63,10476.63,79168.00,13.23%,13.23%,13.23%,13.23%
2023-07,0.00,0.00,0.00,0.00,30517.70,30517.70,149045.00,20.48%,20.48%,20.48%,20.48%
2023-08,0.00,0.00,0.00,0.00,27664.31,27664.31,126234.00,21.92%,21.92%,21.92%,21.92%
2023-09,0.00,0.00,0.00,0.00,54563.83,54563.83,261610.00,20.86%,20.86%,20.86%,20.86%
2023-10,0.00,0.00,0.00,0.00,103128.91,103128.91,474985.00,21.71%,21.71%,21.71%,21.71%
2023-11,0.00,0.00,0.00,0.00,283543.99,283543.99,1226260.00,23.12%,23.12%,23.12%,23.12%
2023-12,0.00,0.00,0.00,2718.46,350906.58,353625.04,2049806.25,17.25%,17.25%,17.25%,17.12%
2024-01,0.00,0.00,0.00,0.00,159872.40,159872.40,1104144.00,14.48%,14.48%,14.48%,14.48%
2024-06,0.00,1663.38,0.00,0.00,0.00,1663.38,39138.00,4.25%,4.25%,0.00%,0.00%


In [139]:
# （6）S量
# note 20241209 增加
# df_zw = dfzh3[(dfzh3.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')]
df_zw = dfzh3[conditions_sl]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-04,0.00,0.00,0.00,0.00,39099.43,39099.43,244612.0,15.98%,15.98%,15.98%,15.98%
2023-05,0.00,0.00,0.00,0.00,27511.76,27511.76,208404.0,13.20%,13.20%,13.20%,13.20%
2023-06,0.00,0.00,0.00,0.00,9970.05,9970.05,132717.0,7.51%,7.51%,7.51%,7.51%
2023-07,0.00,0.00,0.00,0.00,48879.78,48879.78,277310.0,17.63%,17.63%,17.63%,17.63%
2023-08,0.00,0.00,0.00,0.00,62095.21,62095.21,378466.0,16.41%,16.41%,16.41%,16.41%
2023-09,0.00,0.00,0.00,0.00,55995.42,55995.42,356086.0,15.73%,15.73%,15.73%,15.73%
2023-10,0.00,0.00,0.00,0.00,102193.43,102193.43,373112.0,27.39%,27.39%,27.39%,27.39%
2023-11,0.00,0.00,0.00,0.00,101188.66,101188.66,710870.0,14.23%,14.23%,14.23%,14.23%
2023-12,0.00,0.00,0.00,0.00,7927.18,7927.18,9010.0,87.98%,87.98%,87.98%,87.98%


In [140]:
# （7）抖音
df_dy = dfzh3[conditions_dy]  #df_dy=df[df["渠道名称"]=="芝麻租物"]#
df_dy['M0'] = np.where(df_dy.逾期类型=='M0',df_dy["未回收成本"],0)
df_dy['M1'] = np.where(df_dy.逾期类型=='M1',df_dy["未回收成本"],0)
df_dy['M2'] = np.where(df_dy.逾期类型=='M2',df_dy["未回收成本"],0)
df_dy['M3'] = np.where(df_dy.逾期类型=='M3',df_dy["未回收成本"],0)
df_dy['M4'] = np.where(df_dy.逾期类型=='M4',df_dy["未回收成本"],0)
df_dy['M5'] = np.where(df_dy.逾期类型=='M5',df_dy["未回收成本"],0)
df_dy['M6'] = np.where(df_dy.逾期类型=='M6',df_dy["未回收成本"],0)

dfyq = df_dy[df_dy["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_dy[df_dy["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_dy.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2024-04,0.00,0.00,0.00,0.00,4587.85,4587.85,15930.0,28.80%,28.80%,28.80%,28.80%
2024-06,0.00,836.72,123.28,1368.85,59391.94,61720.79,220205.0,28.03%,28.03%,27.59%,26.97%
2024-07,6535.61,2062.73,3408.57,7153.63,129931.47,149092.01,448562.0,33.24%,31.78%,30.56%,28.97%
2024-08,904.10,675.68,0.00,11727.08,63963.57,77270.43,271022.0,28.51%,28.18%,27.93%,23.60%
2024-09,775.16,1184.39,0.00,2063.46,7999.00,12022.01,30173.0,39.84%,37.27%,33.35%,26.51%
2024-11,0.00,0.00,0.00,0.00,5643.00,5643.00,9865.0,57.20%,57.20%,57.20%,57.20%
2024-12,2653.13,0.00,0.00,11516.04,18892.00,33061.17,105399.0,31.37%,28.85%,28.85%,17.92%
2025-01,2626.30,3283.04,1845.70,11928.00,0.00,19683.04,95598.0,20.59%,17.84%,12.48%,0.00%
2025-02,5904.10,3087.29,10183.98,7801.00,0.00,26976.37,119574.0,22.56%,17.62%,6.52%,0.00%


In [141]:
# （8）二手非抖音
# note 20241209 增加   抖音放安卓新机和苹果11-15二手
# df_zw = dfzh3[(dfzh3["是否二手"]=="二手")&(dfzh3.归属渠道!='抖音渠道')]
df_zw = dfzh3[conditions_esfdy]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-08,0.00,0.00,0.0,0.00,660.00,660.00,19130.00,3.45%,3.45%,3.45%,3.45%
2022-09,0.00,0.00,0.0,0.00,11586.46,11586.46,48327.68,23.97%,23.97%,23.97%,23.97%
2022-10,0.00,0.00,0.0,0.00,7654.60,7654.60,62825.00,12.18%,12.18%,12.18%,12.18%
2022-11,0.00,0.00,0.0,0.00,5558.69,5558.69,69285.22,8.02%,8.02%,8.02%,8.02%
2022-12,0.00,0.00,0.0,0.00,0.00,0.00,13685.00,0.00%,0.00%,0.00%,0.00%
2023-01,0.00,0.00,0.0,0.00,9858.20,9858.20,89642.00,11.00%,11.00%,11.00%,11.00%
2023-02,0.00,0.00,0.0,0.00,29630.36,29630.36,147754.00,20.05%,20.05%,20.05%,20.05%
2023-03,0.00,0.00,0.0,0.00,42229.61,42229.61,226883.00,18.61%,18.61%,18.61%,18.61%
2023-04,0.00,0.00,0.0,0.00,16255.74,16255.74,336845.50,4.83%,4.83%,4.83%,4.83%


In [142]:
# （9）拒量
# note 20241209 增加   抖音放安卓新机和苹果11-15二手
# df_zw = dfzh3[(dfzh3["是否二手"]=="二手")&(dfzh3.归属渠道!='抖音渠道')]
df_zw = dfzh3[conditions_jl]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2024-12,10203.13,0.00,3926.33,5854.66,7315.5,27299.62,236025.0,11.57%,7.24%,5.58%,3.10%
2025-01,1448.52,0.00,10543.18,12304.11,0.0,24295.81,269323.0,9.02%,8.48%,4.57%,0.00%
2025-02,68955.29,32071.96,62608.22,181380.95,0.0,345016.42,2132295.0,16.18%,12.95%,8.51%,0.00%
2025-03,73089.84,81872.08,108458.64,165000.87,0.0,428421.43,2499753.0,17.14%,14.21%,6.60%,0.00%
2025-04,147824.71,200711.34,227225.86,30423.80,0.0,606185.71,3521285.0,17.21%,13.02%,0.86%,0.00%
2025-05,198996.95,153207.92,14917.52,0.00,0.0,367122.39,2961684.0,12.40%,5.68%,0.00%,0.00%
2025-06,186654.65,4929.40,0.00,0.00,0.0,191584.05,2380146.0,8.05%,0.21%,0.00%,0.00%
2025-07,21271.50,0.00,0.00,0.00,0.0,21271.50,1674599.0,1.27%,0.00%,0.00%,0.00%
2025-08,0.00,0.00,0.00,0.00,0.0,0.00,149441.0,0.00%,0.00%,0.00%,0.00%


## 6_12.总体订单口径每月当前逾期率

In [143]:
# （1）
dfzh3["还款状态"] = dfzh3["rembursement_status"]

#总体每月当前逾期率 cpd1
df0 = pd.crosstab(dfzh3["下单月份"],dfzh3["还款状态"],margins=True)
df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","已逾期","续租中","还款中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["买断率"]=df0["已买断"]/df0["All"]
df0["买断率"]=df0["买断率"].apply(lambda x:format(x,".2%"))
df0["归还率"]=(df0["已完成"]+df0["检测中"])/df0["All"]
df0["归还率"]=df0["归还率"].apply(lambda x:format(x,".2%"))
df0["续租率"]=df0["续租中"]/df0["All"]
df0["续租率"]=df0["续租率"].apply(lambda x:format(x,".2%"))
df0#[["已买断","All","买断率"]]#[14:]

dfzh3["续租期数"] = dfzh3["relet_periods"] 
dfzh3["已还期数"] = dfzh3["paid_periods"] 

还款状态,已买断,已完成,已逾期,未到首期还款日,检测中,续租中,还款中,All,逾期率,买断率,归还率,续租率
下单月份,,,,,,,,,,,,
2022-07,12,2,7,0,0,1,0,22,31.82%,54.55%,9.09%,4.55%
2022-08,21,5,28,0,1,1,0,56,50.00%,37.50%,10.71%,1.79%
2022-09,18,6,19,0,0,1,0,44,43.18%,40.91%,13.64%,2.27%
2022-10,22,6,11,0,1,1,0,41,26.83%,53.66%,17.07%,2.44%
2022-11,23,7,13,0,0,0,0,43,30.23%,53.49%,16.28%,0.00%
2022-12,39,4,23,0,0,0,0,66,34.85%,59.09%,6.06%,0.00%
2023-01,109,20,88,0,3,0,1,221,39.82%,49.32%,10.41%,0.00%
2023-02,168,48,113,0,1,0,2,332,34.04%,50.60%,14.76%,0.00%
2023-03,152,45,116,0,3,0,0,316,36.71%,48.10%,15.19%,0.00%


In [144]:
# （2）
# 以下代码按顺序运行完到到期状态数据
# pd.crosstab(dfzh1["下单月份"],dfzh1["已还期数"],margins=True)
df_yq = dfzh3[dfzh3["还款状态"]=="已逾期"]

# df_yq=df[df["还款状态"]=="已逾期"]
# pd.crosstab(df_yq["下单月份"],df_yq["续租期数"],margins=True)

df_xz = dfzh3[dfzh3["续租期数"]>0]
# pd.crosstab(df_xz["下单月份"],df_xz["还款状态"],margins=True)

df_timeup = dfzh3[dfzh3["已还期数"]==12]


df_timeon = dfzh3[dfzh3["已还期数"]<12]
df_timeon_1 = pd.crosstab(df_timeon["下单月份"],df_timeon["还款状态"],margins=True).rename(columns={"已买断":"提前买断","已完成":"提前归还","已逾期":"租期逾期"})
df_timeup_xz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]>0)]
df_timeup_xz_1 = pd.crosstab(df_timeup_xz["下单月份"],df_timeup_xz["还款状态"],margins=True).rename(columns={"已买断":"续租后买断","已完成":"续租后归还","已逾期":"续租后逾期","检测中":"续租归还检测中"})
df_timeup_nxz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]<1)]
df_timeup_nxz_1 = pd.crosstab(df_timeup_nxz["下单月份"],df_timeup_nxz["还款状态"],margins=True).rename(columns={"已买断":"到期买断","已完成":"到期归还","已逾期":"到期逾期","检测中":"到期归还检测中","续租中":"待续租","还款中":"已付总租金待归还"})

In [145]:
# （3）
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单数，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单数
# note 20241216 修改
# df_timeon_1.drop(columns=['检测中','续租中'],inplace = True)
# df_timeon_1.drop(columns=['续租中'],inplace = True)
# note 2025.4.17 修改
df_timeup_xz_1.drop(columns=['还款中'],inplace = True)

df_due = pd.merge(pd.merge(df_timeon_1,df_timeup_nxz_1,left_on="下单月份",right_on="下单月份",how="left"),df_timeup_xz_1,left_on="下单月份",right_on="下单月份",how="left")
df_due=df_due.fillna(0)
df_due["合计"]=df_due["All_x"]+df_due["All_y"]+df_due["All"]
#df_due["合计"]=df_due["合计"].astype("int")
df_due.drop(columns=["All_x","All_y","All"],axis=1,inplace=True)
df_due=df_due.astype("int")
df_due#[["提前买断","到期买断","续租后买断"]]#[14:]#[14:]

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,1,1,7,0,0,0,6,1,0,0,0,0,5,0,0,0,1,22
2022-08,2,1,21,0,0,0,8,3,5,0,0,0,11,1,2,1,1,56
2022-09,4,2,10,0,0,0,4,2,5,0,0,0,10,2,4,0,1,44
2022-10,1,1,8,0,0,0,8,2,1,1,0,0,13,3,2,0,1,41
2022-11,5,0,8,0,0,0,6,6,1,0,0,0,12,1,4,0,0,43
2022-12,8,2,16,0,0,0,14,2,5,0,0,0,17,0,2,0,0,66
2023-01,17,4,63,0,0,1,43,16,13,3,0,0,49,0,12,0,0,221
2023-02,27,5,69,0,0,2,89,43,30,1,0,0,52,0,14,0,0,332
2023-03,29,13,79,0,0,0,71,32,24,3,0,0,52,0,13,0,0,316


In [146]:
# （4）
#以下为分别统计每个下单月的订单在12期租期内租期逾期、提前买断、提前归还的的订单占比，
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单占比，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单占比
#该代码需先运行上述代码方不会报错
# applymap(lambda x:format(x,".2%"))
# df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]
df_due=df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]

df_due

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,4.55%,4.55%,31.82%,0.00%,0.00%,0.00%,27.27%,4.55%,0.00%,0.00%,0.00%,0.00%,22.73%,0.00%,0.00%,0.00%,4.55%,100.00%
2022-08,3.57%,1.79%,37.50%,0.00%,0.00%,0.00%,14.29%,5.36%,8.93%,0.00%,0.00%,0.00%,19.64%,1.79%,3.57%,1.79%,1.79%,100.00%
2022-09,9.09%,4.55%,22.73%,0.00%,0.00%,0.00%,9.09%,4.55%,11.36%,0.00%,0.00%,0.00%,22.73%,4.55%,9.09%,0.00%,2.27%,100.00%
2022-10,2.44%,2.44%,19.51%,0.00%,0.00%,0.00%,19.51%,4.88%,2.44%,2.44%,0.00%,0.00%,31.71%,7.32%,4.88%,0.00%,2.44%,100.00%
2022-11,11.63%,0.00%,18.60%,0.00%,0.00%,0.00%,13.95%,13.95%,2.33%,0.00%,0.00%,0.00%,27.91%,2.33%,9.30%,0.00%,0.00%,100.00%
2022-12,12.12%,3.03%,24.24%,0.00%,0.00%,0.00%,21.21%,3.03%,7.58%,0.00%,0.00%,0.00%,25.76%,0.00%,3.03%,0.00%,0.00%,100.00%
2023-01,7.69%,1.81%,28.51%,0.00%,0.00%,0.45%,19.46%,7.24%,5.88%,1.36%,0.00%,0.00%,22.17%,0.00%,5.43%,0.00%,0.00%,100.00%
2023-02,8.13%,1.51%,20.78%,0.00%,0.00%,0.60%,26.81%,12.95%,9.04%,0.30%,0.00%,0.00%,15.66%,0.00%,4.22%,0.00%,0.00%,100.00%
2023-03,9.18%,4.11%,25.00%,0.00%,0.00%,0.00%,22.47%,10.13%,7.59%,0.95%,0.00%,0.00%,16.46%,0.00%,4.11%,0.00%,0.00%,100.00%


In [147]:
# （5）以下为每月预收租金期数占比
dfzh3["预收期数"] = dfzh3["advance_periods"] 
df_ys=pd.crosstab(dfzh3["下单月份"],dfzh3["预收期数"],margins=True)
df_ys.divide(df_ys["All"],axis=0).applymap(lambda x:format(x,".2%"))

预收期数,0,1,2,3,4,5,6,7,8,9,10,12,All
下单月份,,,,,,,,,,,,,
2022-07,0.00%,13.64%,36.36%,45.45%,4.55%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-08,0.00%,1.79%,28.57%,48.21%,12.50%,5.36%,3.57%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-09,0.00%,0.00%,34.09%,45.45%,18.18%,2.27%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-10,2.44%,24.39%,41.46%,14.63%,12.20%,4.88%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-11,0.00%,11.63%,51.16%,16.28%,18.60%,2.33%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-12,0.00%,18.18%,42.42%,19.70%,18.18%,1.52%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2023-01,0.45%,16.29%,54.30%,19.00%,5.88%,2.26%,0.45%,0.45%,0.00%,0.00%,0.00%,0.90%,100.00%
2023-02,0.00%,5.72%,61.14%,17.47%,8.13%,4.82%,1.81%,0.00%,0.30%,0.30%,0.00%,0.30%,100.00%
2023-03,0.63%,15.51%,59.81%,10.76%,4.11%,3.16%,5.70%,0.00%,0.32%,0.00%,0.00%,0.00%,100.00%


In [148]:
# （6）平台套餐统计
dfzh3['平台套餐'] = dfzh3['platform_package']
dfzh3['租赁方案'] = np.where(dfzh3.平台套餐=='租完即送','租完即送','租完归还/可买断')
dfzh3['租赁方案'].value_counts()

zl_class_df = pd.crosstab(dfzh3['下单月份'],dfzh3['租赁方案'],margins=True)
zl_class_df.divide(zl_class_df["All"],axis=0).applymap(lambda x:format(x,".2%"))

租赁方案
租完归还/可买断    53463
租完即送         5161
Name: count, dtype: int64

租赁方案,租完即送,租完归还/可买断,All
下单月份,,,
2022-07,0.00%,100.00%,100.00%
2022-08,8.93%,91.07%,100.00%
2022-09,2.27%,97.73%,100.00%
2022-10,0.00%,100.00%,100.00%
2022-11,6.98%,93.02%,100.00%
2022-12,12.12%,87.88%,100.00%
2023-01,19.46%,80.54%,100.00%
2023-02,9.04%,90.96%,100.00%
2023-03,14.87%,85.13%,100.00%


# 7.每月各渠道逾期率

In [149]:
# # （1-1）首逾发生在 前N期 的逻辑
# df_fq_concat['当前期数'] = df_fq_concat['当前期数'].astype(int)
# # df_fq_concat['当前期数'] 

# df_yq_fq = df_fq_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
# df_yq_fq = df_yq_fq.reset_index()

# df_yq_fq["FPD"] = df_yq_fq[2]
# # note 20241222 修正SPD TPD QPD
# df_yq_fq.loc[:,"SPD"] = np.where( df_yq_fq[2] == 0, df_yq_fq[3], 0)
# df_yq_fq.loc[:,"TPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0), df_yq_fq[4], 0)
# df_yq_fq.loc[:,"QPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0) & (df_yq_fq[4] == 0), df_yq_fq[5], 0)
# df_yq_fq[['order_id',2,3,4,'FPD','SPD','TPD','QPD']][:2]




# （1-2）前N期的最大逾期天数 的逻辑
# 20241222 之前的口劲，下面代码是前N期的最大逾期天数
df_fq_concat['当前期数'] = df_fq_concat['当前期数'].astype(int)
df_yq_fq = df_fq_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_yq_fq = df_yq_fq.reset_index()

df_yq_fq["FPD"] = df_yq_fq[2]
df_yq_fq.loc[:,"SPD"] = np.where(df_yq_fq[3]>df_yq_fq[2],df_yq_fq[3],df_yq_fq[2])
df_yq_fq.loc[:,"TPD"] = np.where(df_yq_fq[4]>df_yq_fq["SPD"],df_yq_fq[4],df_yq_fq["SPD"])
df_yq_fq.loc[:,"QPD"] = np.where(df_yq_fq[5]>df_yq_fq["TPD"],df_yq_fq[5],df_yq_fq["TPD"])

df_yq_fq[['order_id',2,3,4,'FPD','SPD','TPD','QPD']]


# 数据检查
# df_yq_fq[(df_yq_fq.FPD>0)&(df_yq_fq.SPD>0)][['order_id',2,3,4,'FPD','SPD','TPD','QPD']][:3]
# df_yq_fq.loc[:,"SPD"] = np.where( df_yq_fq[2] == 0, df_yq_fq[3], 0)
# df_yq_fq.loc[:,"TPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0), df_yq_fq[4], 0)
# df_yq_fq.loc[:,"QPD"] = np.where((df_yq_fq[2] == 0) & (df_yq_fq[3] == 0) & (df_yq_fq[4] == 0), df_yq_fq[5], 0)
# df_yq_fq[df_yq_fq.order_id.isin([508404,1678492,1692916,12449])]
# df_yq_fq.shape
# tmp1 = df_fq_concat[['order_id','search_time']].drop_duplicates(subset = ['order_id'])
# tmp = df_yq_fq.merge(tmp1, on = 'order_id', how = 'left')
# tmp.shape
# df_yq_fq.shape
# tmp[tmp.search_time == '2024-08'].shape
# tmp[(tmp.search_time == '2024-08')&(tmp.SPD >= 1)].shape
# tmp[(tmp.search_time == '2024-08')&(tmp.SPD >= 4)].shape
# tmp[(tmp.search_time == '2024-08')&(tmp.SPD >= 7)].shape
# tmp[(tmp.search_time == '2024-08')&(tmp.SPD >= 15)].shape
# tmp[(tmp.search_time == '2024-08')&(tmp.SPD >= 30)].shape

# 180/2238
# 114/2238
# 88/2238
# 73/2238
# 57/2238
# 8.04%	6.25%	5.14%	4.65%	3.98%
# 381/2238
# 265/2238
# 217/2238
# 181/2238
# 155/2238
# 17.02%	11.84%	9.70%	8.09%	6.93%

当前期数,order_id,2,3,4,FPD,SPD,TPD,QPD
0,8924,0.0,0.0,4.0,0.0,0.0,4.0,4.0
1,9305,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9514,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9600,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9683,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
57903,3492094,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57904,3492342,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57905,3492366,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57906,3492508,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
# （2）
df_agr_1 =df_fq_concat[["order_id","当前期数","agr_1d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_1d_cum").reset_index()
df_agr_1 = df_agr_1.copy().add_suffix('_agr1')

df_agr_4 =df_fq_concat[["order_id","当前期数","agr_4d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_4d_cum").reset_index()
df_agr_4 = df_agr_4.copy().add_suffix('_agr4')
df_agr_7 =df_fq_concat[["order_id","当前期数","agr_7d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_7d_cum").reset_index()
df_agr_7 = df_agr_7.copy().add_suffix('_agr7')
df_agr_15 =df_fq_concat[["order_id","当前期数","agr_15d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_15d_cum").reset_index()
df_agr_15 = df_agr_15.copy().add_suffix('_agr15')
df_agr_30 =df_fq_concat[["order_id","当前期数","agr_30d_cum"]].pivot(index="order_id", columns="当前期数", values="agr_30d_cum").reset_index()
df_agr_30 = df_agr_30.copy().add_suffix('_agr30')

df_yq_total = pd.concat([df_yq_fq,df_agr_1,df_agr_4,df_agr_7,df_agr_15,df_agr_30],axis=1)
df_yq_total[:2]

# 合并数据 
df = pd.merge(dfzh3,df_yq_total,on="order_id",how="left")
df.shape

当前期数,order_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,FPD,SPD,TPD,QPD,order_id_agr1,1_agr1,2_agr1,3_agr1,4_agr1,5_agr1,6_agr1,7_agr1,8_agr1,9_agr1,10_agr1,11_agr1,12_agr1,13_agr1,14_agr1,15_agr1,16_agr1,17_agr1,18_agr1,19_agr1,20_agr1,21_agr1,22_agr1,23_agr1,24_agr1,order_id_agr4,1_agr4,2_agr4,3_agr4,4_agr4,5_agr4,6_agr4,7_agr4,8_agr4,9_agr4,10_agr4,11_agr4,12_agr4,13_agr4,14_agr4,15_agr4,16_agr4,17_agr4,18_agr4,19_agr4,20_agr4,21_agr4,22_agr4,23_agr4,24_agr4,order_id_agr7,1_agr7,2_agr7,3_agr7,4_agr7,5_agr7,6_agr7,7_agr7,8_agr7,9_agr7,10_agr7,11_agr7,12_agr7,13_agr7,14_agr7,15_agr7,16_agr7,17_agr7,18_agr7,19_agr7,20_agr7,21_agr7,22_agr7,23_agr7,24_agr7,order_id_agr15,1_agr15,2_agr15,3_agr15,4_agr15,5_agr15,6_agr15,7_agr15,8_agr15,9_agr15,10_agr15,11_agr15,12_agr15,13_agr15,14_agr15,15_agr15,16_agr15,17_agr15,18_agr15,19_agr15,20_agr15,21_agr15,22_agr15,23_agr15,24_agr15,order_id_agr30,1_agr30,2_agr30,3_agr30,4_agr30,5_agr30,6_agr30,7_agr30,8_agr30,9_agr30,10_agr30,11_agr30,12_agr30,13_agr30,14_agr30,15_agr30,16_agr30,17_agr30,18_agr30,19_agr30,20_agr30,21_agr30,22_agr30,23_agr30,24_agr30
0,8924,0.0,0.0,0.0,4.0,3.0,3.0,2.0,1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4.0,4.0,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8924,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,9305,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9305,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9305,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9305,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9305,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(58624, 259)

In [151]:
# （3）
df["客户标签fpd4"]=np.where(df["FPD"]>3,"逾期","正常")
df["客户标签fpd1"]=np.where(df["FPD"]>0,"逾期","正常")
df["客户标签fpd7"]=np.where(df["FPD"]>6,"逾期","正常")
df["客户标签fpd15"]=np.where(df["FPD"]>14,"逾期","正常")
df["客户标签spd4"]=np.where(df["SPD"]>3,"逾期","正常")
df["客户标签spd1"]=np.where(df["SPD"]>1,"逾期","正常")
df["客户标签spd15"]=np.where(df["SPD"]>14,"逾期","正常")
df["客户标签tpd15"]=np.where(df["TPD"]>14,"逾期","正常")

# now_overdue_days
# 来自租后监控表，号卡的逾期天数有偏差

df["当前逾期天数"] = df["now_overdue_days"]
df["最长逾期天数"] = df["max_overdue_days"]

def getremark(a,b,c):
    if a>=c:
        return "逾期"
    elif a<c and b =="未到首期还款日":
        return "未到首期还款日"
    else:
        return "正常"
df["当前逾期1+"] = df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],1),axis=1)
df["当前逾期1+"].value_counts()

df["当前逾期4+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],4),axis=1)
df["当前逾期7+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],7),axis=1)
df["当前逾期15+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],15),axis=1)
df["当前逾期31+"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],31),axis=1)
df["M12"]=df.apply(lambda x:getremark(x["当前逾期天数"],x["还款状态"],366),axis=1)


df["历史逾期1+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],1),axis=1)
df["历史逾期7+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],7),axis=1)
df["历史逾期15+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],15),axis=1)
df["历史逾期31+"]=df.apply(lambda x:getremark(x["最长逾期天数"],x["还款状态"],31),axis=1)

当前逾期1+
正常         38276
逾期         13503
未到首期还款日     6845
Name: count, dtype: int64

In [152]:
# （4）上个月下单的当前逾期--渠道名称

current_date = pd.Timestamp.now()
# 前一个月
before_date_1 = current_date - pd.DateOffset(months=1)


# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"] == before_date_1.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift','未到首期还款日']]

当前逾期1+,正常,逾期,All,逾期率,lift,未到首期还款日
channel_name,,,,,,
京东618活动,2,1,3,33.33%,7.02,0
京享租右卡,4,1,38,20.00%,4.21,33
侠客行全域租物,0,0,1,nan%,nan,1
侠客行全域自定义,1,0,1,0.00%,0.0,0
八派信息,1,0,5,0.00%,0.0,4
其他渠道场景渠道。,6,0,15,0.00%,0.0,9
单人聊天会话中的小程序消息卡片（分享）,15,1,68,6.25%,1.32,52
小程序商家消息（服务提醒）,1,0,4,0.00%,0.0,3
广州麒升租物搜索2,0,0,1,nan%,nan,1


In [153]:
# （5）上上个月下单的当前逾期--渠道名称

current_date = pd.Timestamp.now()
# 上上一个月
before_date_2 = current_date - pd.DateOffset(months=2)


# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"] == before_date_2.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift','未到首期还款日']]

当前逾期1+,正常,逾期,All,逾期率,lift,未到首期还款日
channel_name,,,,,,
京东618活动,17,1,22,5.56%,0.964,4
京享租右卡,29,1,30,3.33%,0.577,0
付费流量(通过商家数字推广平台，灯火等投放的广告),1,0,1,0.00%,0.0,0
侠客行全域租物,2,0,2,0.00%,0.0,0
八派信息,3,0,3,0.00%,0.0,0
其他渠道场景渠道。,9,1,10,10.00%,1.73,0
单人聊天会话中的小程序消息卡片（分享）,43,2,45,4.44%,0.769,0
宜品花,2,0,2,0.00%,0.0,0
小程序商家消息（服务提醒）,11,0,11,0.00%,0.0,0


In [154]:
# （6）上上上个月下单的当前逾期--渠道名称

current_date = pd.Timestamp.now()
# 上上上一个月
before_date_3 = current_date - pd.DateOffset(months=3)


# 当前逾期1+  按渠道名称
df_qdmc_4 = df[df["下单月份"] == before_date_3.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["channel_name"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df0[['正常','逾期','All','逾期率','lift','未到首期还款日']]

当前逾期1+,正常,逾期,All,逾期率,lift,未到首期还款日
channel_name,,,,,,
九州信息,2,0,2,0.00%,0.0,0
侠客行全域租物,3,0,3,0.00%,0.0,0
侠客行全域自定义,1,1,2,50.00%,4.41,0
八派信息,12,4,17,25.00%,2.2,1
其他渠道场景渠道。,12,5,17,29.41%,2.59,0
单人聊天会话中的小程序消息卡片（分享）,15,3,19,16.67%,1.47,1
小程序商家消息（服务提醒）,2,1,3,33.33%,2.94,0
我的小程序入口,24,1,25,4.00%,0.352,0
搜索-租手机,117,25,142,17.61%,1.55,0


In [155]:
# （7）上个月下单的当前逾期--渠道归属
# 当前逾期1+ 按归属渠道
current_date = pd.Timestamp.now()
# 前一个月
before_date_1 = current_date - pd.DateOffset(months=1)

df_qdmc_4 = df[df["下单月份"] == before_date_1.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2"))
# df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df_1 = df0[['正常','逾期','未到首期还款日','All','逾期率','lift']].reset_index()
df_1

当前逾期1+,归属渠道,正常,逾期,未到首期还款日,All,逾期率,lift
0,S量,1,0,5,6,0.0,0.0
1,京东渠道,217,12,1239,1468,0.052,1.08
2,侠客行全域自定义,1,0,0,1,0.0,0.0
3,其他渠道场景渠道。,3,0,8,11,0.0,0.0
4,单人聊天会话中的小程序消息卡片（分享）,15,1,52,68,0.062,1.29
5,小程序商家消息（服务提醒）,1,0,3,4,0.0,0.0
6,我的小程序入口,6,0,10,16,0.0,0.0
7,扫描二维码,0,0,2,2,nan,nan
8,搜索渠道,278,16,1254,1548,0.054,1.12
9,支付宝 push 消息（同1014）,2,0,2,4,0.0,0.0


In [156]:
# （8）上上个月下单的当前逾期--渠道归属
# 当前逾期1+ 按归属渠道
current_date = pd.Timestamp.now()
# 上上一个月
before_date_2 = current_date - pd.DateOffset(months=2)

df_qdmc_4 = df[df["下单月份"] == before_date_2.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df_2 = df0[['正常','逾期','未到首期还款日','All','逾期率','lift']].reset_index()
df_2

当前逾期1+,归属渠道,正常,逾期,未到首期还款日,All,逾期率,lift
0,S量,5,0,0,5,0.00%,0.0
1,京东渠道,724,38,6,768,4.99%,0.865
2,付费流量(通过商家数字推广平台，灯火等投放的广告),1,0,0,1,0.00%,0.0
3,其他渠道场景渠道。,7,1,0,8,12.50%,2.17
4,单人聊天会话中的小程序消息卡片（分享）,43,2,0,45,4.44%,0.769
5,宜品花,2,0,0,2,0.00%,0.0
6,小程序商家消息（服务提醒）,8,0,0,8,0.00%,0.0
7,我的小程序入口,23,0,0,23,0.00%,0.0
8,抖音渠道,12,4,3,19,25.00%,4.33
9,搜索渠道,1577,96,29,1702,5.74%,0.995


In [157]:
# （9）上上上个月下单的当前逾期--渠道归属
# 当前逾期1+ 按归属渠道
current_date = pd.Timestamp.now()
# 上上上一个月
before_date_3 = current_date - pd.DateOffset(months=3)

df_qdmc_4 = df[df["下单月份"] == before_date_3.strftime('%Y-%m')]
df0=pd.crosstab(df_qdmc_4["归属渠道"],df_qdmc_4["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["lift"]=df0["逾期率"].str.strip('%').astype(float)/float(df0.at["All","逾期率"].strip('%'))
df0["lift"]=df0["lift"].apply(lambda x:format(x,".3"))
df_3 = df0[['正常','逾期','未到首期还款日','All','逾期率','lift']].reset_index()
df_3
df_4 = df_1.merge(df_2, on = '归属渠道', how = 'outer')
df_4 = df_4.merge(df_3, on = '归属渠道', how = 'outer')
df_4 = df_4[df_4.归属渠道 != 'All']
df_4 = df_4.sort_values(by = '逾期率_x', ascending = False)
df_4['逾期率_x'] = df_4['逾期率_x'].astype('float')
df_4
# 正常_x 为上个月 正常_y为上上个月

当前逾期1+,归属渠道,正常,逾期,未到首期还款日,All,逾期率,lift
0,S量,18,4,1,23,18.18%,1.6
1,京东渠道,20,3,0,23,13.04%,1.15
2,侠客行全域自定义,1,1,0,2,50.00%,4.41
3,其他渠道场景渠道。,10,3,0,13,23.08%,2.03
4,单人聊天会话中的小程序消息卡片（分享）,15,3,1,19,16.67%,1.47
5,小程序商家消息（服务提醒）,2,1,0,3,33.33%,2.94
6,我的小程序入口,22,1,0,23,4.35%,0.383
7,抖音渠道,20,13,2,35,39.39%,3.47
8,搜索渠道,1622,249,16,1887,13.31%,1.17
9,支付宝 push 消息（同1014）,7,0,0,7,0.00%,0.0


当前逾期1+,归属渠道,正常_x,逾期_x,未到首期还款日_x,All_x,逾期率_x,lift_x,正常_y,逾期_y,未到首期还款日_y,All_y,逾期率_y,lift_y,正常,逾期,未到首期还款日,All,逾期率,lift
14,支付宝学生特惠小程序,0.0,0.0,2.0,2.0,NaN,nan,1.0,0.0,0.0,1.0,0.00%,0.0,NaN,NaN,NaN,NaN,NaN,NaN
17,支付宝社群,0.0,0.0,3.0,3.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.00%,0.0
23,生活号-商品组件,0.0,0.0,2.0,2.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,生活号-J-苹果旗舰,0.0,0.0,4.0,4.0,NaN,nan,1.0,0.0,0.0,1.0,0.00%,0.0,3.0,0.0,0.0,3.0,0.00%,0.0
20,未知渠道,0.0,0.0,6.0,6.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,收藏卡片01,0.0,0.0,1.0,1.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,扫描二维码,0.0,0.0,2.0,2.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,支群-社群专享,0.0,0.0,1.0,1.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,单人聊天会话中的小程序消息卡片（分享）,15.0,1.0,52.0,68.0,0.062,1.29,43.0,2.0,0.0,45.0,4.44%,0.769,15.0,3.0,1.0,19.0,16.67%,1.47
12,搜索渠道,278.0,16.0,1254.0,1548.0,0.054,1.12,1577.0,96.0,29.0,1702.0,5.74%,0.995,1622.0,249.0,16.0,1887.0,13.31%,1.17


In [158]:
# df[(df.归属渠道 == '广州易合顺')&(df['当前逾期1+'] == '逾期')][['order_number','search_time']]

# 8.主要渠道当前逾期对比

## 8_11.主要渠道当前逾期

In [159]:
# （1）当前逾期1+
df0 = pd.crosstab(df["下单月份"],df["当前逾期1+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})
# df0
df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期1+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期1+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期1+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期1+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# 二手非抖音
df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["当前逾期1+"],margins=True)
df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


df0_5[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

当前逾期1+,未到首期_总体,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_会话,逾期率_会话,lift_会话,月合计_抖音,逾期率_抖音,lift_抖音,月合计_二手非抖音,逾期率_二手非抖音,lift_二手非抖音
下单月份,,,,,,,,,,,,,,,,,,,,
2022-07,0,14,8,22,36.36%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,2,0.00%,0.0
2022-08,0,26,30,56,53.57%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,4,25.00%,0.467
2022-09,0,24,20,44,45.45%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,55.56%,1.22
2022-10,0,28,13,41,31.71%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,22.22%,0.701
2022-11,0,30,13,43,30.23%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,11.11%,0.368
2022-12,0,43,23,66,34.85%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,3,0.00%,0.0
2023-01,0,130,91,221,41.18%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,17,35.29%,0.857
2023-02,0,218,114,332,34.34%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,30,36.67%,1.07
2023-03,0,197,119,316,37.66%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,44,40.91%,1.09


In [160]:
# # （2）当前逾期4+
# df0=pd.crosstab(df["下单月份"],df["当前逾期4+"],margins=True)
# df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
# df0=df0.rename(columns={"正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

# df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
# df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期4+"],margins=True)
# df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
# df1=df1.rename(columns={"正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
# df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
# df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
# df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

# df_zm=df[df["归属渠道"]=="芝麻租物"]
# df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期4+"],margins=True)
# df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
# df2=df2.rename(columns={"正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
# df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
# df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
# df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

# df_ss=df[df["归属渠道"]=="搜索渠道"]
# df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期4+"],margins=True)
# df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
# df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
# # df3.columns
# df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
# df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
# df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

# df_dy=df[df["归属渠道"]=="抖音渠道"]
# df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期4+"],margins=True)
# df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
# df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
# df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
# df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
# df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# # df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# # df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",


# # 二手非抖音
# df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
# df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["当前逾期4+"],margins=True)
# df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
# df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
# df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
# df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
# df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


# df0_5[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

In [161]:
# # （3）当前逾期7+
# df0=pd.crosstab(df["下单月份"],df["当前逾期7+"],margins=True)
# df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
# df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

# df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
# df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期7+"],margins=True)
# df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
# df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
# df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
# df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
# df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

# df_zm=df[df["归属渠道"]=="芝麻租物"]
# df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期7+"],margins=True)
# df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
# df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
# df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
# df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
# df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

# df_ss=df[df["归属渠道"]=="搜索渠道"]
# df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期7+"],margins=True)
# df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
# df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
# df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
# df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
# df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

# df_dy=df[df["归属渠道"]=="抖音渠道"]
# df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期7+"],margins=True)
# df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
# df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
# df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
# df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
# df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# # df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# # df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",

# # 二手非抖音
# df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
# df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["当前逾期7+"],margins=True)
# df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
# df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
# df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
# df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
# df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
# df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


# df0_5[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

In [162]:
# （4）当前逾期15+
df0=pd.crosstab(df["下单月份"],df["当前逾期15+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期15+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期15+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期15+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期15+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",

# 二手非抖音
df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["当前逾期15+"],margins=True)
df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


df0_5[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

当前逾期15+,未到首期_总体,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_会话,逾期率_会话,lift_会话,月合计_抖音,逾期率_抖音,lift_抖音,月合计_二手非抖音,逾期率_二手非抖音,lift_二手非抖音
下单月份,,,,,,,,,,,,,,,,,,,,
2022-07,0,14,8,22,36.36%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,2,0.00%,0.0
2022-08,0,26,30,56,53.57%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,4,25.00%,0.467
2022-09,0,24,20,44,45.45%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,55.56%,1.22
2022-10,0,28,13,41,31.71%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,22.22%,0.701
2022-11,0,30,13,43,30.23%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,11.11%,0.368
2022-12,0,43,23,66,34.85%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,3,0.00%,0.0
2023-01,0,130,91,221,41.18%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,17,35.29%,0.857
2023-02,0,218,114,332,34.34%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,30,36.67%,1.07
2023-03,0,197,119,316,37.66%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,44,40.91%,1.09


In [163]:
# （5）当前逾期31+
df0=pd.crosstab(df["下单月份"],df["当前逾期31+"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["当前逾期31+"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["当前逾期31+"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["当前逾期31+"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["当前逾期31+"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",

# 二手非抖音
df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["当前逾期31+"],margins=True)
df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


df0_5[["未到首期_总体","正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

当前逾期31+,未到首期_总体,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_会话,逾期率_会话,lift_会话,月合计_抖音,逾期率_抖音,lift_抖音,月合计_二手非抖音,逾期率_二手非抖音,lift_二手非抖音
下单月份,,,,,,,,,,,,,,,,,,,,
2022-07,0,14,8,22,36.36%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,2,0.00%,0.0
2022-08,0,26,30,56,53.57%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,4,25.00%,0.467
2022-09,0,24,20,44,45.45%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,55.56%,1.22
2022-10,0,28,13,41,31.71%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,22.22%,0.701
2022-11,0,30,13,43,30.23%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,11.11%,0.368
2022-12,0,43,23,66,34.85%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,3,0.00%,0.0
2023-01,0,130,91,221,41.18%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,17,35.29%,0.857
2023-02,0,218,114,332,34.34%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,30,36.67%,1.07
2023-03,0,198,118,316,37.34%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,44,40.91%,1.1


## 8_12.客户标签fpd15

In [164]:
df0=pd.crosstab(df["下单月份"],df["客户标签fpd15"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["客户标签fpd15"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["客户标签fpd15"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["客户标签fpd15"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["客户标签fpd15"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",

# 二手非抖音
df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["客户标签fpd15"],margins=True)
df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


df0_5[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

客户标签fpd15,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_会话,逾期率_会话,lift_会话,月合计_抖音,逾期率_抖音,lift_抖音,月合计_二手非抖音,逾期率_二手非抖音,lift_二手非抖音
下单月份,,,,,,,,,,,,,,,,,,,
2022-07,20,2,22,9.09%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,2,0.00%,0.0
2022-08,49,7,56,12.50%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,4,0.00%,0.0
2022-09,41,3,44,6.82%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,0.00%,0.0
2022-10,38,3,41,7.32%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,0.00%,0.0
2022-11,39,4,43,9.30%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,0.00%,0.0
2022-12,60,6,66,9.09%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,3,0.00%,0.0
2023-01,206,15,221,6.79%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,17,5.88%,0.866
2023-02,304,28,332,8.43%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,30,13.33%,1.58
2023-03,291,25,316,7.91%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,44,6.82%,0.862


## 8_13.客户标签spd15

In [165]:
df0=pd.crosstab(df["下单月份"],df["客户标签spd15"],margins=True)
df0["逾期率"]=df0["逾期"]/df0[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0=df0.rename(columns={"未到首期还款日":"未到首期_总体","正常":"正常_总体","逾期":"逾期_总体","All":"月合计_总体","逾期率":"逾期率_总体"})#"未到首期还款日":"未到首期_总体",

df_dr=df[df["归属渠道"]=="单人聊天会话中的小程序消息卡片（分享）"]
df1=pd.crosstab(df_dr["下单月份"],df_dr["客户标签spd15"],margins=True)
df1["逾期率"]=df1["逾期"]/df1[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df1["逾期率"]=df1["逾期率"].apply(lambda x:format(x,".2%"))
df1=df1.rename(columns={"未到首期还款日":"未到首期_会话","未到首期还款日":"未到首期_会话","正常":"正常_会话","逾期":"逾期_会话","All":"月合计_会话","逾期率":"逾期率_会话"})#"未到首期还款日":"未到首期_会话",
df0_1=pd.merge(df0,df1,left_on="下单月份",right_on="下单月份",how="left")
df0_1["lift_会话"]=df0_1["逾期率_会话"].str.strip('%').astype(float)/df0_1["逾期率_总体"].str.strip('%').astype(float)
df0_1["lift_会话"]=df0_1["lift_会话"].apply(lambda x:format(x,".3"))

df_zm=df[df["归属渠道"]=="芝麻租物"]
df2=pd.crosstab(df_zm["下单月份"],df_zm["客户标签spd15"],margins=True)
df2["逾期率"]=df2["逾期"]/df2[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df2["逾期率"]=df2["逾期率"].apply(lambda x:format(x,".2%"))
df2=df2.rename(columns={"未到首期还款日":"未到首期_租物","正常":"正常_租物","逾期":"逾期_租物","All":"月合计_租物","逾期率":"逾期率_租物"})#"未到首期还款日":"未到首期_租物",
df0_2=pd.merge(df0_1,df2,left_on="下单月份",right_on="下单月份",how="left")
df0_2["lift_租物"]=df0_2["逾期率_租物"].str.strip('%').astype(float)/df0_2["逾期率_总体"].str.strip('%').astype(float)
df0_2["lift_租物"]=df0_2["lift_租物"].apply(lambda x:format(x,".3"))

df_ss=df[df["归属渠道"]=="搜索渠道"]
df3=pd.crosstab(df_ss["下单月份"],df_ss["客户标签spd15"],margins=True)
df3["逾期率"]=df3["逾期"]/df3[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df3["逾期率"]=df3["逾期率"].apply(lambda x:format(x,".2%"))
df3=df3.rename(columns={"未到首期还款日":"未到首期_搜索","正常":"正常_搜索","逾期":"逾期_搜索","All":"月合计_搜索","逾期率":"逾期率_搜索"})#"未到首期还款日":"未到首期_搜索",
df0_3=pd.merge(df0_2,df3,left_on="下单月份",right_on="下单月份",how="left")
df0_3["lift_搜索"]=df0_3["逾期率_搜索"].str.strip('%').astype(float)/df0_3["逾期率_总体"].str.strip('%').astype(float)
df0_3["lift_搜索"]=df0_3["lift_搜索"].apply(lambda x:format(x,".3"))

df_dy=df[df["归属渠道"]=="抖音渠道"]
df4 = pd.crosstab(df_dy["下单月份"],df_dy["客户标签spd15"],margins=True)
df4["逾期率"]=df4["逾期"]/df4[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df4["逾期率"]=df4["逾期率"].apply(lambda x:format(x,".2%"))
df4=df4.rename(columns={"未到首期还款日":"未到首期_抖音","正常":"正常_抖音","逾期":"逾期_抖音","All":"月合计_抖音","逾期率":"逾期率_抖音"})
df0_4=pd.merge(df0_3,df4,left_on="下单月份",right_on="下单月份",how="left")
df0_4["lift_抖音"]=df0_4["逾期率_抖音"].str.strip('%').astype(float)/df0_4["逾期率_总体"].str.strip('%').astype(float)
df0_4["lift_抖音"]=df0_4["lift_抖音"].apply(lambda x:format(x,".3"))

# df0_4[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_抖音","逾期率_抖音","lift_抖音"]]
# df0_3[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_会话","逾期率_会话","lift_会话","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索"]]#"未到首期_总体",

# 二手非抖音
df_erfdy=df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")]
df5 = pd.crosstab(df_erfdy["下单月份"],df_erfdy["客户标签spd15"],margins=True)
df5["逾期率"]=df5["逾期"]/df5[["逾期","正常"]].apply(lambda x:x.sum(),axis=1)
df5["逾期率"]=df5["逾期率"].apply(lambda x:format(x,".2%"))
df5=df5.rename(columns={"未到首期还款日":"未到首期_二手非抖音","正常":"正常_二手非抖音","逾期":"逾期_二手非抖音","All":"月合计_二手非抖音","逾期率":"逾期率_二手非抖音"})
df0_5=pd.merge(df0_4,df5,left_on="下单月份",right_on="下单月份",how="left")
df0_5["lift_二手非抖音"]=df0_5["逾期率_二手非抖音"].str.strip('%').astype(float)/df0_5["逾期率_总体"].str.strip('%').astype(float)
df0_5["lift_二手非抖音"]=df0_5["lift_二手非抖音"].apply(lambda x:format(x,".3"))


df0_5[["正常_总体","逾期_总体","月合计_总体","逾期率_总体","月合计_租物","逾期率_租物","lift_租物","月合计_搜索","逾期率_搜索","lift_搜索","月合计_会话","逾期率_会话","lift_会话","月合计_抖音","逾期率_抖音","lift_抖音","月合计_二手非抖音","逾期率_二手非抖音","lift_二手非抖音"]]

客户标签spd15,正常_总体,逾期_总体,月合计_总体,逾期率_总体,月合计_租物,逾期率_租物,lift_租物,月合计_搜索,逾期率_搜索,lift_搜索,月合计_会话,逾期率_会话,lift_会话,月合计_抖音,逾期率_抖音,lift_抖音,月合计_二手非抖音,逾期率_二手非抖音,lift_二手非抖音
下单月份,,,,,,,,,,,,,,,,,,,
2022-07,16,6,22,27.27%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,2,0.00%,0.0
2022-08,44,12,56,21.43%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,4,0.00%,0.0
2022-09,39,5,44,11.36%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,11.11%,0.978
2022-10,38,3,41,7.32%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,0.00%,0.0
2022-11,38,5,43,11.63%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,9,11.11%,0.955
2022-12,59,7,66,10.61%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,3,0.00%,0.0
2023-01,190,31,221,14.03%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,17,5.88%,0.419
2023-02,286,46,332,13.86%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,30,20.00%,1.44
2023-03,274,42,316,13.29%,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,44,13.64%,1.03


# 9.每月前四期逾期率对比

## 9_11.数据准备

In [166]:
df["fpd4"] = np.where((df["FPD"]>3) & (df['2_agr4'] ==1),1,0)
df["fpd7"] = np.where((df["FPD"]>6) & (df['2_agr7'] ==1),1,0)
df["fpd15"] = np.where((df["FPD"]>14) & (df['2_agr15'] ==1),1,0)
df["fpd30"] = np.where((df["FPD"]>29) & (df['2_agr30'] ==1),1,0)

df["spd4"]=np.where((df["SPD"]>3) & (df['3_agr4'] ==1),1,0)
df["spd7"]=np.where((df["SPD"]>6) & (df['3_agr7'] ==1),1,0)
df["spd15"]=np.where((df["SPD"]>14) & (df['3_agr15'] ==1),1,0)
df["spd30"]=np.where((df["SPD"]>29) & (df['3_agr30'] ==1),1,0)

df["tpd4"]=np.where((df["TPD"]>3) & (df['4_agr4'] ==1),1,0)
df["tpd7"]=np.where((df["TPD"]>6) & (df['4_agr7'] ==1),1,0)
df["tpd15"]=np.where((df["TPD"]>14) & (df['4_agr15'] ==1),1,0)
df["tpd30"]=np.where((df["TPD"]>29) & (df['4_agr30'] ==1),1,0)

df["qpd4"]=np.where((df["QPD"]>3) & (df['5_agr4'] ==1),1,0)
df["qpd7"]=np.where((df["QPD"]>6) & (df['5_agr7'] ==1),1,0)
df["qpd15"]=np.where((df["QPD"]>14) & (df['5_agr15'] ==1),1,0)
df["qpd30"]=np.where((df["QPD"]>29) & (df['5_agr30'] ==1),1,0)

df["fpd1"] = np.where((df["FPD"]>=1) & (df['2_agr1'] ==1),1,0)
df["spd1"]=np.where((df["SPD"]>=1) & (df['3_agr1'] ==1),1,0)
df["tpd1"]=np.where((df["TPD"]>=1) & (df['4_agr1'] ==1),1,0)
df["qpd1"]=np.where((df["QPD"]>=1) & (df['5_agr1'] ==1),1,0)

## 9_12.月度fstq-总体

In [167]:
df_tag4 = df.groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

# 不包含拒量的话，拒量也不在分母
# tmp = df_tag4.reset_index()
# tmp[tmp.下单月份 == '2025-02'][['下单月份','order_id','fpd1','2_agr1','fpd1_rate','fpd7','2_agr7','fpd7_rate']]

In [168]:
import datetime 
Today= datetime.date.today()
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='xlsxwriter') as writer:    #, mode='a'
    # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq.to_excel(writer, sheet_name='mon_fstq', index=True)  # index=False 表示不导出行索引  

df_tag4 = df.groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})

# df_tag4
# mon_fstq

OSError: Cannot save file into a non-existent directory: 'E:\myfile\业务相关数据'

## 9_13.周度fstq-总体

In [ ]:
# df['week_number'] = df['下单日期'].dt.isocalendar().week
df['week_start_date'] = df['下单日期'] - pd.to_timedelta(df['下单日期'].dt.weekday, unit='D')  

# df['week_start_date']

df_tag4 = df.groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# result[result.index>='2024-01-01']

In [ ]:
# with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='xlsxwriter') as writer:    #, mode='a' 
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq.to_excel(writer, sheet_name='week_fstq', index=True)  # index=False 表示不导出行索引  

## 9_14.月度fstq-总体非抖音非拒量非号卡

In [ ]:
conditions_all_2 = ((df.归属渠道!='抖音渠道')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_zm = ((df.归属渠道=='芝麻租物')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_ss = ((df.归属渠道=='搜索渠道')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_drhh = ((df.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_sl = ((df.归属渠道=='S量')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_dy = (df.归属渠道=='抖音渠道')
conditions_esfdy = ((df["是否二手"]=="二手")&(df.归属渠道!='抖音渠道')&(df.是否拒量 != 1)&(df.是否号卡 != 1))
conditions_jl = (df.是否拒量 == 1)

In [ ]:
df_tag4 = df[conditions_all_2].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

# 不包含拒量的话，拒量也不在分母
# tmp = df_tag4.reset_index()
# tmp[tmp.下单月份 == '2025-02'][['下单月份','order_id','fpd1','2_agr1','fpd1_rate','fpd7','2_agr7','fpd7_rate']]

In [ ]:
# import datetime 
# Today= datetime.date.today()
# with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='xlsxwriter') as writer:    #, mode='a'
#     # 将df写入新的sheet，名为'new_sheet'  
#     mon_fstq.to_excel(writer, sheet_name='mon_fstq_o', index=True)  # index=False 表示不导出行索引  


with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq.to_excel(writer, sheet_name='mon_fstq_o', index=True)  # index=False 表示不导出行索引 

## 9_15.周度fstq-总体非抖音非拒量非号卡

In [ ]:
# df['week_number'] = df['下单日期'].dt.isocalendar().week
# df['week_start_date'] = df['下单日期'] - pd.to_timedelta(df['下单日期'].dt.weekday, unit='D')  

# df['week_start_date']

df_tag4 = df[conditions_all_2].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# result[result.index>='2024-01-01']

In [ ]:
# with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
# #     # 将df写入新的sheet，名为'new_sheet'  
#     week_fstq.to_excel(writer, sheet_name='week_fstq_o', index=True)  # index=False 表示不导出行索引  

with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq.to_excel(writer, sheet_name='week_fstq_o', index=True)  # index=False 表示不导出行索引 

## 9_16.月度fstq-芝麻租物

In [ ]:
df_tag4 = df[conditions_zm].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_zm = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_zm.to_excel(writer, sheet_name='mon_fstq_zm', index=True)  # index=False 表示不导出行索引  

## 9_17.周度fstq-芝麻租物

In [ ]:
df_tag4 = df[conditions_zm].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_zm = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_zm[week_fstq_zm.index>='2024-01-01']

In [ ]:
df_tag4_zm = df[df.归属渠道=='芝麻租物'].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4_zm = df_tag4_zm.reset_index()

In [ ]:
df_tag4_zm['fpd1_rate'] = np.where(df_tag4_zm['2_agr1']>0,df_tag4_zm['fpd1']/df_tag4_zm['2_agr1'],np.nan)
df_tag4_zm['fpd1_rate'] = df_tag4_zm['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4_zm['fpd4_rate'] = np.where(df_tag4_zm['2_agr4']>0,df_tag4_zm['fpd4']/df_tag4_zm['2_agr4'],np.nan)
df_tag4_zm['fpd4_rate'] = df_tag4_zm['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4_zm['fpd7_rate'] = np.where(df_tag4_zm['2_agr7']>0,df_tag4_zm['fpd7']/df_tag4_zm['2_agr7'],np.nan)
df_tag4_zm['fpd7_rate'] = df_tag4_zm['fpd7_rate'].apply(lambda x:format(x,".2%"))

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_zm.to_excel(writer, sheet_name='week_fstq_zm', index=True)  # index=False 表示不导出行索引  

## 9_18.月度fstq-搜索渠道

In [ ]:
df_tag4 = df[conditions_ss].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_ss = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]


In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_ss.to_excel(writer, sheet_name='mon_fstq_ss', index=True)  # index=False 表示不导出行索引  

## 9_19.周度fstq-搜索渠道

In [ ]:
df_tag4 = df[conditions_ss].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_ss = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_ss[week_fstq_ss.index>='2024-01-01']

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_ss.to_excel(writer, sheet_name='week_fstq_ss', index=True)  # index=False 表示不导出行索引  

## 9_20.月度fstq-单人会话

In [ ]:
df_tag4 = df[conditions_drhh].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_drhh = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_drhh.to_excel(writer, sheet_name='mon_fstq_drhh', index=True)  # index=False 表示不导出行索引  

In [ ]:
# mon_fstq_drhh.tail()
# df_tag4[['order_id','2_agr1','fpd1','2_agr4','fpd4']].tail()
# df[(df.归属渠道=="单人聊天会话中的小程序消息卡片（分享）")&(df.下单日期>='2024-11-1')&(df.下单日期<'2024-11-9')][['order_number','下单日期','order_id']]
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number,tprm.order_id,tprm.max_overdue_days,tprm.channel_id,tprm.order_create_time,om.status
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# where tprm.order_number in 
# ('A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# )
# ;
# '''
# query(sql)

# tmp = df_tag4.reset_index()
# tmp[tmp.下单月份 == '2024-11'][['fpd7','2_agr7','fpd15','2_agr15']]
# 2/31
# 1/9

# df[(df.归属渠道=="单人聊天会话中的小程序消息卡片（分享）")&(df.下单日期 >= '2024-11-1')&(df.下单日期 <= '2024-11-5')][['order_number','下单日期','order_id']]
# df_xz_concat[(df_xz_concat.归属渠道=="单人聊天会话中的小程序消息卡片（分享）")&(df_xz_concat.下单日期 >= '2024-11-1')&(df_xz_concat.下单日期 <= '2024-11-5')][['order_number','下单日期','order_id','逾期天数']].drop_duplicates(subset = ['order_number','逾期天数'])

## 9_21.周度fstq-单人会话

In [ ]:
# 分周 
df_tag4 = df[conditions_drhh].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_drhh = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_drhh.to_excel(writer, sheet_name='week_fstq_drhh', index=True)  # index=False 表示不导出行索引  

## 9_22.月度fstq-S量

In [ ]:
df_tag4 = df[conditions_sl].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_sl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_sl.to_excel(writer, sheet_name='mon_fstq_sl', index=True)  # index=False 表示不导出行索引  

## 9_23.周度fstq-S量

In [ ]:
# 分周 
df_tag4 = df[conditions_sl].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_sl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_sl.to_excel(writer, sheet_name='week_fstq_sl', index=True)  # index=False 表示不导出行索引  

## 9_24.月度fstq-抖音

In [ ]:
df_tag4 = df[conditions_dy].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_dy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# mon_fstq_dy

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_dy.to_excel(writer, sheet_name='mon_fstq_dy', index=True)  # index=False 表示不导出行索引  

## 9_25.周度fstq-抖音

In [ ]:
# 分周 
df_tag4 = df[conditions_dy].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_dy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# week_fstq_dy

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_dy.to_excel(writer, sheet_name='week_fstq_dy', index=True)  # index=False 表示不导出行索引  

## 9_26.月度fstq-二手非抖音

In [ ]:
df_tag4 = df[conditions_esfdy].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_esfdy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_esfdy.to_excel(writer, sheet_name='mon_fstq_esfdy', index=True)  # index=False 表示不导出行索引  

## 9_27.周度fstq-二手非抖音

In [ ]:
# 分周 
df_tag4 = df[conditions_esfdy].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_esfdy = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_esfdy.to_excel(writer, sheet_name='week_fstq_esfdy', index=True)  # index=False 表示不导出行索引  

In [ ]:
# df[(df["是否二手"]=="二手")&(df["归属渠道"]!="抖音渠道")&(df.search_time >= '2024-11')][['order_number','下单日期','2_agr1','2_agr4','2_agr7','2_agr15','fpd1','fpd4','fpd7','fpd15']]
# df[(df.归属渠道=="单人聊天会话中的小程序消息卡片（分享）")&(df.search_time == '2024-12')&(df['下单日期'] <= '2024-12-08')][['order_number','下单日期','2_agr1','2_agr4','2_agr7','2_agr15','2_agr30','fpd1','fpd4','fpd7','fpd15','fpd30']].sort_values(by = '下单日期')

## 9_28.月度fstq-拒量

In [ ]:
df_tag4 = df[conditions_jl].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_jl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_jl.to_excel(writer, sheet_name='mon_fstq_jl', index=True)  # index=False 表示不导出行索引  

## 9_29.周度fstq-拒量

In [ ]:
# 分周 
df_tag4 = df[conditions_jl].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_jl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

In [ ]:
with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_jl.to_excel(writer, sheet_name='week_fstq_jl', index=True)  # index=False 表示不导出行索引  

In [ ]:
# # 2025.4.22
# df.shape
# df_tmp = df.merge(df_reject_list[['order_number','是否拒量']], on = 'order_number', how = 'left')
# df_tmp.shape

In [ ]:
# # （1）总体-不含拒量
# df_tag4 = df_tmp[df_tmp.是否拒量 != 1].groupby(by=['下单月份']).agg({'order_id':'count'
# ,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
# ,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
# ,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
# ,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
# })
# df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
# df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
# df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
# df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
# df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
# df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
# df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
# df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
# df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
# df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
# df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
# df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
# df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
# df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
# df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
# df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
# df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
# df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
# df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
# df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
# df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

# mon_fstq_no_jl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
#          ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# mon_fstq_no_jl['type'] = 'all_no_jl'
# mon_fstq_no_jl = mon_fstq_no_jl.reset_index()
# mon_fstq_no_jl = mon_fstq_no_jl[mon_fstq_no_jl.下单月份 >= '2024-12']
# # 不包含拒量的话，拒量也不在分母
# # tmp = df_tag4.reset_index()
# # tmp[tmp.下单月份 == '2025-02'][['下单月份','order_id','fpd1','2_agr1','fpd1_rate','fpd7','2_agr7','fpd7_rate']]

In [ ]:
# # （2）租物-不含拒量
# df_tag4 = df_tmp[(df_tmp.归属渠道=='芝麻租物')&(df_tmp.是否拒量 != 1)].groupby(by=['下单月份']).agg({'order_id':'count'
# ,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
# ,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
# ,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
# ,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
# })
# df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
# df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
# df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
# df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
# df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
# df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
# df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
# df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
# df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
# df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
# df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
# df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
# df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
# df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
# df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
# df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
# df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
# df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
# df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
# df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
# df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

# mon_fstq_no_jl_zw = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
#          ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# mon_fstq_no_jl_zw['type'] = 'all_no_jl_zw'
# mon_fstq_no_jl_zw = mon_fstq_no_jl_zw.reset_index()
# mon_fstq_no_jl_zw = mon_fstq_no_jl_zw[mon_fstq_no_jl_zw.下单月份 >= '2024-12']

In [ ]:
# # （2）搜索-不含拒量
# df_tag4 = df_tmp[(df_tmp.归属渠道=='搜索渠道')&(df_tmp.是否拒量 != 1)].groupby(by=['下单月份']).agg({'order_id':'count'
# ,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
# ,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
# ,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
# ,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
# })
# df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
# df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
# df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
# df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
# df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
# df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
# df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
# df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
# df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
# df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
# df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
# df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
# df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
# df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
# df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
# df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
# df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
# df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
# df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
# df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
# df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

# mon_fstq_no_jl_ss = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
#          ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]
# mon_fstq_no_jl_ss['type'] = 'all_no_jl_ss'
# mon_fstq_no_jl_ss = mon_fstq_no_jl_ss.reset_index()
# mon_fstq_no_jl_ss = mon_fstq_no_jl_ss[mon_fstq_no_jl_ss.下单月份 >= '2024-12']

In [ ]:
# # （4）拒量
# df_tag4 = df_tmp[df_tmp.是否拒量 == 1].groupby(by=['下单月份']).agg({'order_id':'count'
# ,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
# ,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
# ,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
# ,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
# ,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
# ,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
# ,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
# ,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
# })
# df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
# df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
# df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
# df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
# df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
# df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
# df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
# df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
# df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
# df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
# df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
# df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
# df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
# df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
# df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
# df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
# df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
# df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
# df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
# df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
# df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
# df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

# mon_fstq_jl = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
#          ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

# mon_fstq_jl['type'] = 'jl'
# mon_fstq_jl = mon_fstq_jl.reset_index()

In [ ]:
# mon_fstq_tmp = pd.concat([mon_fstq_no_jl,mon_fstq_no_jl_zw,mon_fstq_no_jl_ss,mon_fstq_jl])

In [ ]:
# with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
# #     # 将df写入新的sheet，名为'new_sheet'  
#     mon_fstq_tmp.to_excel(writer, sheet_name='mon_fstq_jl', index=True)  # index=False 表示不导出行索引  

# 10.免人审fstq

In [ ]:
# （1）数据准备
sql = '''-- 免人审 
SELECT date(r.time) as risk_date 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.grade_msg') end,'"','') as grade_msg 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.result') end,'"','') as risk_result 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.rejected') end,'"','') as reject_tag 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.trace_id') end,'"','') as trace_id 
,r.mobile,r.id_card,r.data  
from db_credit.risk r  
where  r.data like '%免人审%'
# and  r.time > '2024-07-01 00:00:00'

;
'''
risk_data = query(sql)
risk_data.shape

sql = '''-- 免人审 
select oa.* 
from db_credit.order_association oa 

;
'''
order_data = query(sql)
order_data.shape

risk_data.sort_values(by=["trace_id","risk_date"],inplace=True)
risk_data.drop_duplicates(subset=["trace_id"],keep="last",inplace=True)
risk_data.shape

ms_df = risk_data.merge(order_data,left_on='trace_id',right_on='risk_trace_id',how='left')
ms_df.shape

ms_df.drop_duplicates(subset=["order_id"],keep="last",inplace=True)
ms_df.shape

ms_df1 = df.merge(ms_df[['order_id','order_number','grade_msg']],on='order_id',how='left')
ms_df1.grade_msg.notnull().sum()

(91917, 8)

(2343313, 5)

(91917, 8)

(92010, 13)

(90881, 13)

33674

In [ ]:
# （2）月度
df_tag4 = ms_df1[ms_df1.grade_msg.notnull()].groupby(by=['下单月份']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

mon_fstq_ms = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    mon_fstq_ms.to_excel(writer, sheet_name='mon_fstq_ms', index=True)  # index=False 表示不导出行索引  

In [ ]:
# ms_df1[(ms_df1["下单日期"] <= '2024-10-02')&(ms_df1.search_time == '2024-10')][['order_number_x','下单日期','tpd1','tpd4','4_agr1','4_agr4','grade_msg']]
# ms_df1[(ms_df1["下单日期"] <= '2024-10-02')&(ms_df1.search_time == '2024-10')].shape
# ms_df1[(ms_df1["下单日期"] <= '2024-10-02')&(ms_df1.search_time == '2024-10')&(ms_df1.tpd1>=1)].shape
# 7/51
# ms_df1[(ms_df1["下单日期"] <= '2024-10-02')&(ms_df1.search_time == '2024-10')]['4_agr1'].value_counts()
# ms_df1[(ms_df1["下单日期"] <= '2024-10-02')&(ms_df1.search_time == '2024-10')]['tpd1'].value_counts()

# df_tag4[['tpd1','tpd4','4_agr1','4_agr4']]

In [ ]:
# （3）周度
df_tag4 =ms_df1[ms_df1.grade_msg.notnull()].groupby(by=['week_start_date']).agg({'order_id':'count'
,'fpd1':'sum','fpd4':'sum','fpd7':'sum','fpd15':'sum','fpd30':'sum'
,'spd1':'sum','spd4':'sum','spd7':'sum','spd15':'sum','spd30':'sum'
,'tpd1':'sum','tpd4':'sum','tpd7':'sum','tpd15':'sum','tpd30':'sum'
,'qpd1':'sum','qpd4':'sum','qpd7':'sum','qpd15':'sum','qpd30':'sum'
,'2_agr1':'sum','2_agr4':'sum','2_agr7':'sum','2_agr15':'sum','2_agr30':'sum'
,'3_agr1':'sum','3_agr4':'sum','3_agr7':'sum','3_agr15':'sum','3_agr30':'sum'
,'4_agr1':'sum','4_agr4':'sum','4_agr7':'sum','4_agr15':'sum','4_agr30':'sum'
,'5_agr1':'sum','5_agr4':'sum','5_agr7':'sum','5_agr15':'sum','5_agr30':'sum'
})
df_tag4['fpd1_rate'] = np.where(df_tag4['2_agr1']>0,df_tag4['fpd1']/df_tag4['2_agr1'],np.nan)
df_tag4['fpd1_rate'] = df_tag4['fpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd4_rate'] = np.where(df_tag4['2_agr4']>0,df_tag4['fpd4']/df_tag4['2_agr4'],np.nan)
df_tag4['fpd4_rate'] = df_tag4['fpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd7_rate'] = np.where(df_tag4['2_agr7']>0,df_tag4['fpd7']/df_tag4['2_agr7'],np.nan)
df_tag4['fpd7_rate'] = df_tag4['fpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd15_rate'] = np.where(df_tag4['2_agr15']>0,df_tag4['fpd15']/df_tag4['2_agr15'],np.nan)
df_tag4['fpd15_rate'] = df_tag4['fpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['fpd30_rate'] = np.where(df_tag4['2_agr30']>0,df_tag4['fpd30']/df_tag4['2_agr30'],np.nan)
df_tag4['fpd30_rate'] = df_tag4['fpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd1_rate'] = np.where(df_tag4['3_agr1']>0,df_tag4['spd1']/df_tag4['3_agr1'],np.nan)
df_tag4['spd1_rate'] = df_tag4['spd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd4_rate'] = np.where(df_tag4['3_agr4']>0,df_tag4['spd4']/df_tag4['3_agr4'],np.nan)
df_tag4['spd4_rate'] = df_tag4['spd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd7_rate'] = np.where(df_tag4['3_agr7']>0,df_tag4['spd7']/df_tag4['3_agr7'],np.nan)
df_tag4['spd7_rate'] = df_tag4['spd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd15_rate'] = np.where(df_tag4['3_agr15']>0,df_tag4['spd15']/df_tag4['3_agr15'],np.nan)
df_tag4['spd15_rate'] = df_tag4['spd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['spd30_rate'] = np.where(df_tag4['3_agr30']>0,df_tag4['spd30']/df_tag4['3_agr30'],np.nan)
df_tag4['spd30_rate'] = df_tag4['spd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd1_rate'] = np.where(df_tag4['4_agr1']>0,df_tag4['tpd1']/df_tag4['4_agr1'],np.nan)
df_tag4['tpd1_rate'] = df_tag4['tpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd4_rate'] = np.where(df_tag4['4_agr4']>0,df_tag4['tpd4']/df_tag4['4_agr4'],np.nan)
df_tag4['tpd4_rate'] = df_tag4['tpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd7_rate'] = np.where(df_tag4['4_agr7']>0,df_tag4['tpd7']/df_tag4['4_agr7'],np.nan)
df_tag4['tpd7_rate'] = df_tag4['tpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd15_rate'] = np.where(df_tag4['4_agr15']>0,df_tag4['tpd15']/df_tag4['4_agr15'],np.nan)
df_tag4['tpd15_rate'] = df_tag4['tpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['tpd30_rate'] = np.where(df_tag4['4_agr30']>0,df_tag4['tpd30']/df_tag4['4_agr30'],np.nan)
df_tag4['tpd30_rate'] = df_tag4['tpd30_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd1_rate'] = np.where(df_tag4['5_agr1']>0,df_tag4['qpd1']/df_tag4['5_agr1'],np.nan)
df_tag4['qpd1_rate'] = df_tag4['qpd1_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd4_rate'] = np.where(df_tag4['5_agr4']>0,df_tag4['qpd4']/df_tag4['5_agr4'],np.nan)
df_tag4['qpd4_rate'] = df_tag4['qpd4_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd7_rate'] = np.where(df_tag4['5_agr7']>0,df_tag4['qpd7']/df_tag4['5_agr7'],np.nan)
df_tag4['qpd7_rate'] = df_tag4['qpd7_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd15_rate'] = np.where(df_tag4['5_agr15']>0,df_tag4['qpd15']/df_tag4['5_agr15'],np.nan)
df_tag4['qpd15_rate'] = df_tag4['qpd15_rate'].apply(lambda x:format(x,".2%"))
df_tag4['qpd30_rate'] = np.where(df_tag4['5_agr30']>0,df_tag4['qpd30']/df_tag4['5_agr30'],np.nan)
df_tag4['qpd30_rate'] = df_tag4['qpd30_rate'].apply(lambda x:format(x,".2%"))

week_fstq_ms = df_tag4[['order_id','fpd1_rate','fpd4_rate','fpd7_rate','fpd15_rate','fpd30_rate','spd1_rate','spd4_rate','spd7_rate','spd15_rate','spd30_rate'
         ,'tpd1_rate','tpd4_rate','tpd7_rate','tpd15_rate','tpd30_rate','qpd1_rate','qpd4_rate','qpd7_rate','qpd15_rate','qpd30_rate']]

with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_w_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    week_fstq_ms.to_excel(writer, sheet_name='week_fstq_ms', index=True)  # index=False 表示不导出行索引  

# 11.cpd hpd

## 11_1.加工

In [ ]:

# 租后监控表
#   `now_overdue_days` int DEFAULT NULL COMMENT '当前逾期天数'
#   `max_overdue_days` int DEFAULT NULL COMMENT '最长逾期天数'

# dfzh3 含未回收成本 >0的, now_overdue_days 和 max_overdue_days 包含租归逾期的
# dfzh3_tmp = dfzh3[dfzh3["未回收成本"]>0] 
# dfzh3_tmp = dfzh3_tmp.merge(df_reject_list[['order_number','是否拒量']], on = 'order_number', how = 'left')

# 渠道用最新的
# path = 'C:/Users/Administrator/Desktop/mine/5_周报/data/'
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# tmp1 = pd.read_excel(path + 'df_cpd_hpd_20250427.xlsx')
# tmp1['归属渠道_new'] = np.where(tmp1.归属渠道.isin(conditions_1), 'S量', tmp1.归属渠道)
# tmp1['归属渠道_new'] = np.where(tmp1.归属渠道_new.str.contains('搜索'), '搜索渠道', tmp1.归属渠道_new)


dfzh3_tmp = dfzh3[['order_number', 'search_time', '是否二手', 'now_overdue_days', 'max_overdue_days', '归属渠道', '是否拒量', '是否号卡']]
# dfzh3_tmp = dfzh3_tmp.merge(tmp1, on = 'order_number', how = 'left')

# dfzh3_tmp['归属渠道_new'] = np.where(dfzh3_tmp.归属渠道.isin(conditions_1), 'S量', dfzh3_tmp.归属渠道)

# dfzh3_tmp['归属渠道_new'] = dfzh3_tmp['归属渠道_new'].fillna('未知渠道')
dfzh3_tmp['是否拒量'] = dfzh3_tmp['是否拒量'].fillna(0)
dfzh3_tmp['是否号卡'] = dfzh3_tmp['是否号卡'].fillna(0)
dfzh3_tmp.shape

# dfzh3[dfzh3["未回收成本"]>0].shape
# dfzh3.shape
# dfzh3_tmp[dfzh3_tmp.是否拒量.isnull()].search_time.value_counts()

# 号卡的当前逾期口径租后监控表没有更新过来，号卡的逾期默认为 0
# dfzh3_tmp['now_overdue_days']  = np.where(dfzh3_tmp.归属渠道_new.isin(['广州易合顺','深圳刘全兵号卡1']), 0, dfzh3_tmp.now_overdue_days)
# dfzh3_tmp['max_overdue_days']  = np.where(dfzh3_tmp.归属渠道_new.isin(['广州易合顺','深圳刘全兵号卡1']), 0, dfzh3_tmp.max_overdue_days)

dfzh3_tmp['now_overdue_days']  = np.where(dfzh3_tmp.是否号卡 == 1, 0, dfzh3_tmp.now_overdue_days)
dfzh3_tmp['max_overdue_days']  = np.where(dfzh3_tmp.是否号卡 == 1, 0, dfzh3_tmp.max_overdue_days)

dfzh3_tmp['cpd_1'] = np.where(dfzh3_tmp['now_overdue_days']>= 1,1,0)
dfzh3_tmp['cpd_4'] = np.where(dfzh3_tmp['now_overdue_days']>= 4,1,0)
dfzh3_tmp['cpd_7'] = np.where(dfzh3_tmp['now_overdue_days']>= 7,1,0)
dfzh3_tmp['cpd_15'] = np.where(dfzh3_tmp['now_overdue_days']>= 15,1,0)
dfzh3_tmp['cpd_31'] = np.where(dfzh3_tmp['now_overdue_days']>= 31,1,0)

dfzh3_tmp['hpd_1'] = np.where(dfzh3_tmp['max_overdue_days']>= 1,1,0)
dfzh3_tmp['hpd_4'] = np.where(dfzh3_tmp['max_overdue_days']>= 4,1,0)
dfzh3_tmp['hpd_7'] = np.where(dfzh3_tmp['max_overdue_days']>= 7,1,0)
dfzh3_tmp['hpd_15'] = np.where(dfzh3_tmp['max_overdue_days']>= 15,1,0)
dfzh3_tmp['hpd_31'] = np.where(dfzh3_tmp['max_overdue_days']>= 31,1,0)

dfzh3_tmp[:1]



(46933, 8)

,order_number,search_time,是否二手,now_overdue_days,max_overdue_days,归属渠道,是否拒量,是否号卡,cpd_1,cpd_4,cpd_7,cpd_15,cpd_31,hpd_1,hpd_4,hpd_7,hpd_15,hpd_31
0,A202207050851373,2022-07,二手,0,4,未知渠道,0,0,0,0,0,0,0,1,1,0,0,0


## 11_2.cpd

In [ ]:
# 总体 总体非抖音非拒量 租物渠道 搜索渠道 抖音渠道 二手非抖音 单人聊天会话中的小程序消息卡片（分享） 拒量 S量
conditions_all_2 = ((dfzh3_tmp.归属渠道!='抖音渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_zm = ((dfzh3_tmp.归属渠道=='芝麻租物')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_ss = ((dfzh3_tmp.归属渠道=='搜索渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_drhh = ((dfzh3_tmp.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_sl = ((dfzh3_tmp.归属渠道=='S量')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_dy = (dfzh3_tmp.归属渠道=='抖音渠道')
conditions_esfdy = ((dfzh3_tmp["是否二手"]=="二手")&(dfzh3_tmp.归属渠道!='抖音渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_jl = (dfzh3_tmp.是否拒量 == 1)

cpd_all = dfzh3_tmp.groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_o = dfzh3_tmp[conditions_all_2].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_租物渠道 = dfzh3_tmp[conditions_zm].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_搜索渠道 = dfzh3_tmp[conditions_ss].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_单人会话 = dfzh3_tmp[conditions_drhh].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_S量 = dfzh3_tmp[conditions_sl].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_抖音渠道 = dfzh3_tmp[conditions_dy].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_二手非抖音 = dfzh3_tmp[conditions_esfdy].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()
cpd_all_拒量 = dfzh3_tmp[conditions_jl].groupby(['search_time']).agg({'order_number': 'count', 'cpd_1': 'sum', 'cpd_4': 'sum', 'cpd_7': 'sum', 'cpd_15': 'sum', 'cpd_31': 'sum'}).reset_index()

cpd_all_o = cpd_all_o.rename(columns = {'order_number': 'order_number_o', 'cpd_1': 'cpd_1_o','cpd_4':'cpd_4_o','cpd_7':'cpd_7_o','cpd_15':'cpd_15_o','cpd_31':'cpd_31_o'})
cpd_all_租物渠道 = cpd_all_租物渠道.rename(columns = {'order_number': 'order_number_zw', 'cpd_1': 'cpd_1_zw','cpd_4':'cpd_4_zw','cpd_7':'cpd_7_zw','cpd_15':'cpd_15_zw','cpd_31':'cpd_31_zw'})
cpd_all_搜索渠道 = cpd_all_搜索渠道.rename(columns = {'order_number': 'order_number_ss', 'cpd_1': 'cpd_1_ss','cpd_4':'cpd_4_ss','cpd_7':'cpd_7_ss','cpd_15':'cpd_15_ss','cpd_31':'cpd_31_ss'})
cpd_all_抖音渠道 = cpd_all_抖音渠道.rename(columns = {'order_number': 'order_number_dy', 'cpd_1': 'cpd_1_dy','cpd_4':'cpd_4_dy','cpd_7':'cpd_7_dy','cpd_15':'cpd_15_dy','cpd_31':'cpd_31_dy'})
cpd_all_二手非抖音 = cpd_all_二手非抖音.rename(columns = {'order_number': 'order_number_esfdy', 'cpd_1': 'cpd_1_esfdy','cpd_4':'cpd_4_esfdy','cpd_7':'cpd_7_esfdy','cpd_15':'cpd_15_esfdy','cpd_31':'cpd_31_esfdy'})
cpd_all_单人会话 = cpd_all_单人会话.rename(columns = {'order_number': 'order_number_drhh', 'cpd_1': 'cpd_1_drhh','cpd_4':'cpd_4_drhh','cpd_7':'cpd_7_drhh','cpd_15':'cpd_15_drhh','cpd_31':'cpd_31_drhh'})
cpd_all_拒量 = cpd_all_拒量.rename(columns = {'order_number': 'order_number_jl', 'cpd_1': 'cpd_1_jl','cpd_4':'cpd_4_jl','cpd_7':'cpd_7_jl','cpd_15':'cpd_15_jl','cpd_31':'cpd_31_jl'})
cpd_all_S量 = cpd_all_S量.rename(columns = {'order_number': 'order_number_sl', 'cpd_1': 'cpd_1_sl','cpd_4':'cpd_4_sl','cpd_7':'cpd_7_sl','cpd_15':'cpd_15_sl','cpd_31':'cpd_31_sl'})

cpd_all = cpd_all.merge(cpd_all_o, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_租物渠道, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_搜索渠道, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_单人会话, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_S量, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_抖音渠道, on = 'search_time', how = 'left')

cpd_all = cpd_all.merge(cpd_all_二手非抖音, on = 'search_time', how = 'left')
cpd_all = cpd_all.merge(cpd_all_拒量, on = 'search_time', how = 'left')


cpd_all.shape


with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    cpd_all.to_excel(writer, sheet_name='cpd', index=True)  # index=False 表示不导出行索引  

(36, 55)

## 11_3.hpd

In [ ]:


# 总体 总体非抖音非拒量 租物渠道 搜索渠道 抖音渠道 二手非抖音 单人聊天会话中的小程序消息卡片（分享） 拒量 S量
conditions_all_2 = ((dfzh3_tmp.归属渠道!='抖音渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_zm = ((dfzh3_tmp.归属渠道=='芝麻租物')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_ss = ((dfzh3_tmp.归属渠道=='搜索渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_drhh = ((dfzh3_tmp.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_sl = ((dfzh3_tmp.归属渠道=='S量')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_dy = (dfzh3_tmp.归属渠道=='抖音渠道')
conditions_esfdy = ((dfzh3_tmp["是否二手"]=="二手")&(dfzh3_tmp.归属渠道!='抖音渠道')&(dfzh3_tmp.是否拒量 != 1)&(dfzh3_tmp.是否号卡 != 1))
conditions_jl = (dfzh3_tmp.是否拒量 == 1)

hpd_all = dfzh3_tmp.groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_o = dfzh3_tmp[conditions_all_2].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_租物渠道 = dfzh3_tmp[conditions_zm].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_搜索渠道 = dfzh3_tmp[conditions_ss].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_单人会话 = dfzh3_tmp[conditions_drhh].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_S量 = dfzh3_tmp[conditions_sl].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_抖音渠道 = dfzh3_tmp[conditions_dy].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_二手非抖音 = dfzh3_tmp[conditions_esfdy].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()
hpd_all_拒量 = dfzh3_tmp[conditions_jl].groupby(['search_time']).agg({'order_number': 'count', 'hpd_1': 'sum', 'hpd_4': 'sum', 'hpd_7': 'sum', 'hpd_15': 'sum', 'hpd_31': 'sum'}).reset_index()

hpd_all_o = hpd_all_o.rename(columns = {'order_number': 'order_number_o', 'hpd_1': 'hpd_1_o','hpd_4':'hpd_4_o','hpd_7':'hpd_7_o','hpd_15':'hpd_15_o','hpd_31':'hpd_31_o'})
hpd_all_租物渠道 = hpd_all_租物渠道.rename(columns = {'order_number': 'order_number_zw', 'hpd_1': 'hpd_1_zw','hpd_4':'hpd_4_zw','hpd_7':'hpd_7_zw','hpd_15':'hpd_15_zw','hpd_31':'hpd_31_zw'})
hpd_all_搜索渠道 = hpd_all_搜索渠道.rename(columns = {'order_number': 'order_number_ss', 'hpd_1': 'hpd_1_ss','hpd_4':'hpd_4_ss','hpd_7':'hpd_7_ss','hpd_15':'hpd_15_ss','hpd_31':'hpd_31_ss'})
hpd_all_抖音渠道 = hpd_all_抖音渠道.rename(columns = {'order_number': 'order_number_dy', 'hpd_1': 'hpd_1_dy','hpd_4':'hpd_4_dy','hpd_7':'hpd_7_dy','hpd_15':'hpd_15_dy','hpd_31':'hpd_31_dy'})
hpd_all_二手非抖音 = hpd_all_二手非抖音.rename(columns = {'order_number': 'order_number_esfdy', 'hpd_1': 'hpd_1_esfdy','hpd_4':'hpd_4_esfdy','hpd_7':'hpd_7_esfdy','hpd_15':'hpd_15_esfdy','hpd_31':'hpd_31_esfdy'})
hpd_all_单人会话 = hpd_all_单人会话.rename(columns = {'order_number': 'order_number_drhh', 'hpd_1': 'hpd_1_drhh','hpd_4':'hpd_4_drhh','hpd_7':'hpd_7_drhh','hpd_15':'hpd_15_drhh','hpd_31':'hpd_31_drhh'})
hpd_all_拒量 = hpd_all_拒量.rename(columns = {'order_number': 'order_number_jl', 'hpd_1': 'hpd_1_jl','hpd_4':'hpd_4_jl','hpd_7':'hpd_7_jl','hpd_15':'hpd_15_jl','hpd_31':'hpd_31_jl'})
hpd_all_S量 = hpd_all_S量.rename(columns = {'order_number': 'order_number_sl', 'hpd_1': 'hpd_1_sl','hpd_4':'hpd_4_sl','hpd_7':'hpd_7_sl','hpd_15':'hpd_15_sl','hpd_31':'hpd_31_sl'})

hpd_all = hpd_all.merge(hpd_all_o, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_租物渠道, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_搜索渠道, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_单人会话, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_S量, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_抖音渠道, on = 'search_time', how = 'left')

hpd_all = hpd_all.merge(hpd_all_二手非抖音, on = 'search_time', how = 'left')
hpd_all = hpd_all.merge(hpd_all_拒量, on = 'search_time', how = 'left')


hpd_all.shape


with pd.ExcelWriter(f'E:/myfile/业务相关数据/fstq相关数据输出_m_{Today}.xlsx', engine='openpyxl', mode='a') as writer:  
#     # 将df写入新的sheet，名为'new_sheet'  
    hpd_all.to_excel(writer, sheet_name='hpd', index=True)  # index=False 表示不导出行索引  

(36, 55)

# 12.分日

# 以下是数据检查

In [ ]:
# （1）11月下单的首逾检查
df_tag4[:2]
df[(df.search_time == '2024-11')&(df['2_agr1'] == 1)].shape
df[(df.search_time == '2024-11')&(df['2_agr1'] == 1)&(df['归属渠道'] == '搜索渠道')].shape
df[(df.search_time == '2024-11')&(df['2_agr1'] == 1)&(df['归属渠道'] == '芝麻租物')].shape
df[(df.search_time == '2024-11')&(df['2_agr1'] == 1)&(df['归属渠道'] == '八派信息')].shape
df[(df.search_time == '2024-11')&(df.fpd1 == 1)][['order_id','order_number','fpd1','fpd4','2_agr1','2_agr4','归属渠道','search_time']]
df[(df.search_time == '2024-11')&(df['2_agr1'] == 1)&(df['归属渠道'] == '芝麻租物')][['order_id','order_number','fpd1','fpd4','2_agr1','2_agr4','归属渠道','search_time']]

df_xz_concat[(df_xz_concat.search_time == '2024-11') & (df_xz_concat.逾期天数>0)][['order_id','order_number','归属渠道','search_time','状态',
   '应付日期','实付日期','实付日期new','逾期天数','max_overdue_days']]

,order_id,fpd1,fpd4,fpd7,fpd15,fpd30,spd1,spd4,spd7,spd15,...,tpd1_rate,tpd4_rate,tpd7_rate,tpd15_rate,tpd30_rate,qpd1_rate,qpd4_rate,qpd7_rate,qpd15_rate,qpd30_rate
week_start_date,,,,,,,,,,,,,,,,,,,,,
2023-09-11,23,1,1,1,1,1,2,1,1,1,...,21.74%,8.70%,4.35%,4.35%,4.35%,26.09%,8.70%,4.35%,4.35%,4.35%
2023-09-18,49,0,0,0,0,0,1,1,0,0,...,6.12%,4.08%,4.08%,4.08%,4.08%,20.41%,10.20%,10.20%,8.16%,6.12%


(411, 286)

(160, 286)

(197, 286)

(6, 286)

,order_id,order_number,fpd1,fpd4,2_agr1,2_agr4,归属渠道,search_time
24150,1670688,A2024110113220670,1,0,1.0,1.0,八派信息,2024-11
24151,1670767,A20241101134208149,1,0,1.0,1.0,搜索渠道,2024-11
24167,1671376,A20241101152533170,1,1,1.0,1.0,芝麻租物,2024-11
24171,1671696,A20241101162057121,1,0,1.0,1.0,芝麻租物,2024-11
24189,1672503,A20241101195135116,1,1,1.0,1.0,八派信息,2024-11
24197,1672926,A2024110123471583,1,0,1.0,1.0,芝麻租物,2024-11
24202,1673827,A2024110210160276,1,1,1.0,1.0,搜索渠道,2024-11
24208,1674017,A20241102105310266,1,0,1.0,1.0,搜索渠道,2024-11
24211,1674125,A2024110211121073,1,1,1.0,1.0,芝麻租物,2024-11
24257,1677114,A2024110309265468,1,0,1.0,1.0,芝麻租物,2024-11


,order_id,order_number,fpd1,fpd4,2_agr1,2_agr4,归属渠道,search_time
24129,1669581,A2024110109350797,0,0,1.0,1.0,芝麻租物,2024-11
24130,1669626,A20241101095443142,0,0,1.0,1.0,芝麻租物,2024-11
24131,1669690,A2024110110110243,0,0,1.0,1.0,芝麻租物,2024-11
24132,1669693,A2024110110114046,0,0,1.0,1.0,芝麻租物,2024-11
24133,1669818,A20241101103701171,0,0,1.0,1.0,芝麻租物,2024-11
...,...,...,...,...,...,...,...,...
24536,1691481,A2024110619202051,0,0,1.0,0.0,芝麻租物,2024-11
24538,1674886,A202411021401144,0,0,1.0,1.0,芝麻租物,2024-11
24539,1690828,A20241106163223162,1,0,1.0,0.0,芝麻租物,2024-11
24543,1691704,A2024110621091720,0,0,1.0,0.0,芝麻租物,2024-11


,order_id,order_number,归属渠道,search_time,状态,应付日期,实付日期,实付日期new,逾期天数,max_overdue_days
323313,1670688,A2024110113220670,八派信息,2024-11,已逾期支付,2024-12-04,2024-12-06,2024-12-06,2,2
323325,1670767,A20241101134208149,搜索渠道,2024-11,已逾期支付,2024-12-04,2024-12-06,2024-12-06,2,2
323517,1671376,A20241101152533170,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,6,6
323565,1671696,A20241101162057121,芝麻租物,2024-11,已逾期支付,2024-12-04,2024-12-05,2024-12-05,1,1
323781,1672503,A20241101195135116,八派信息,2024-11,已逾期,2024-12-04,NaT,NaT,6,6
323877,1672926,A2024110123471583,芝麻租物,2024-11,已逾期支付,2024-12-04,2024-12-05,2024-12-05,1,1
323937,1673827,A2024110210160276,搜索渠道,2024-11,已逾期,2024-12-05,NaT,NaT,5,5
324009,1674017,A20241102105310266,搜索渠道,2024-11,已逾期支付,2024-12-05,2024-12-06,2024-12-06,1,1
324045,1674125,A2024110211121073,芝麻租物,2024-11,已逾期,2024-12-05,NaT,NaT,5,5
324597,1677114,A2024110309265468,芝麻租物,2024-11,已逾期支付,2024-12-06,2024-12-08,2024-12-08,2,2


In [ ]:
df_xz_concat[df_xz_concat.order_number == 'A20240902174329243'][['归属渠道']]

,归属渠道
283242,搜索渠道
283243,搜索渠道
283244,搜索渠道
283245,搜索渠道
283246,搜索渠道
283247,搜索渠道
283248,搜索渠道
283249,搜索渠道
283250,搜索渠道
283251,搜索渠道


In [ ]:
# （2）vintage数据核对
df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

df_xz_concat[df_xz_concat.order_number == 'A2023073012170638'][['逾期天数','实付日期new','应付日期','实付金额']]

(330987, 160)

is_relet
0.0    330987
1.0     33459
2.0        62
Name: count, dtype: int64

,逾期天数,实付日期new,应付日期,实付金额
55502,0,2023-07-30,2023-08-02,180.38
55503,0,2023-07-30,2023-09-02,180.38
55504,0,2023-07-30,2023-10-02,180.38
55505,0,2023-09-02,2023-11-02,180.38
55506,0,2023-10-06,2023-12-02,180.38
55507,0,2023-11-10,2024-01-02,180.38
55508,0,2023-11-10,2024-02-02,180.38
55509,0,2024-02-27,2024-03-02,180.38
55510,0,2024-03-23,2024-04-02,180.38
55511,0,2024-04-05,2024-05-02,180.38


In [ ]:
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day == 31, df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        # df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        # df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        # df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])

# tmp = df_xz_concat[df_xz_concat.下单日期 == '2024-02'][['下单日期']][:2]
# tmp
# tmp = df_xz_concat[df_xz_concat.下单日期 == '2023-01-31'].shape

In [ ]:
# from pandas.tseries.offsets import MonthEnd

tmp = df_xz_concat[(df_xz_concat.下单日期 == '2023-06-30')][['order_number','下单日期','实付日期new','应付日期','实付金额','date_str']][:3]
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 31号的跳级了，所以不要加1
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n)
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

tmp = mob_date(tmp)

var = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_5','mob_date_6','mob_date_7','mob_date_8','mob_date_9','mob_date_10','mob_date_11','mob_date_12','date_str']
# tmp = df_fq_concat[(df_fq_concat.下单日期 == '2023-04-30')][['order_number','下单日期','实付日期new','应付日期','实付金额'] + var][:3]
tmp[['order_number','下单日期','实付日期new','应付日期','实付金额'] + var]
# tmp['应付日期'] = np.where((tmp['实付日期new'].notna())&((tmp['应付日期'] - tmp['实付日期new']).dt.days >= 0),tmp['实付日期new'],tmp['应付日期'])
# [((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
#                           & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
#                           & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
#                           | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]
# tmp.sort_values(by = '实付日期new')

# i = 1
# tmp[((tmp[f'mob_date_{i}'] - tmp['应付日期']).dt.days >= 0) 
#                           & ((tmp['date_str'] - tmp['应付日期']).dt.days > 0) 
#                           & ((tmp['date_str'] - tmp[f'mob_date_{i}']).dt.days > 0)
                        #   | (tmp['下单日期'] == tmp['实付日期new'])] 

,order_number,下单日期,实付日期new,应付日期,实付金额,mob_date_0,mob_date_1,mob_date_2,mob_date_3,mob_date_4,mob_date_5,mob_date_6,mob_date_7,mob_date_8,mob_date_9,mob_date_10,mob_date_11,mob_date_12,date_str
41456,A2023063003392242,2023-06-30,2023-06-30,2023-07-03,1.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-12-10
41457,A2023063003392242,2023-06-30,2023-07-30,2023-08-03,533.8,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-12-10
41458,A2023063003392242,2023-06-30,2023-08-31,2023-09-03,533.8,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-12-10


In [ ]:
# df_fq_concat[df_fq_concat.order_id == 8924]
# df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount == 0)][['order_id','order_number','应付日期','当前期数','purchase_amount']][:1]
# df_fq_concat[(df_fq_concat.order_number=='A202207131230485')][['order_id','order_number','应付日期','当前期数','purchase_amount']]
df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount == 0)].shape

(730, 160)

In [ ]:
df_mob_total[df_mob_total.下单月份 == '2023-07'].shape
df_mob_total[:1]
df_mob_total[df_mob_total.下单月份 == '2023-01'].to_excel('tmp.xlsx', index = False)

(11466, 16)

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,MOB,overdue_days_1,overdue_days_4,overdue_days_7,overdue_days_15,overdue_days_31,下单月份,purchase_amount,归属渠道,是否二手,unpaid_money_mob
0,A202207050851373,2022-07-05,357.95,0,357.95,mob_0,0,0,0,0,0,2022-07,5150.0,未知渠道,二手,4792.05


In [ ]:
# df_mon2 = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道!='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon2 
df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.search_time == '2023-07')].shape
df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.search_time == '2023-07')].purchase_amount.sum()
# df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道!='抖音渠道')&(df_fq_concat.search_time == '2023-07')].to

(882, 160)

6142708.0

In [ ]:
# df_fq_concat[:2]
# df_mob_total[df_mob_total.order_number == 'A2023072814030812']
df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['实付金额','当前期数','状态','应付日期','实付日期','paid_money_mob1','mob_date_1','paid_money_mob2','mob_date_2']]

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

,实付金额,当前期数,状态,应付日期,实付日期,paid_money_mob1,mob_date_1,paid_money_mob2,mob_date_2
56018,1.00,1.0,正常还款,2023-08-03,2023-07-31,1.00,2023-08-31,1.00,2023-09-30
56019,479.73,2.0,正常还款,2023-09-03,2023-08-30,479.73,2023-08-31,479.73,2023-09-30
56020,479.73,3.0,正常还款,2023-10-03,2023-09-27,NaN,2023-08-31,479.73,2023-09-30
56021,479.73,4.0,正常还款,2023-11-03,2023-10-30,NaN,2023-08-31,NaN,2023-09-30
56022,479.73,5.0,正常还款,2023-12-03,2023-11-27,NaN,2023-08-31,NaN,2023-09-30
56023,479.73,6.0,已逾期支付,2024-01-03,2024-01-06,NaN,2023-08-31,NaN,2023-09-30
56024,479.73,7.0,已逾期支付,2024-02-03,2024-02-04,NaN,2023-08-31,NaN,2023-09-30
56025,479.73,8.0,正常还款,2024-03-03,2024-03-01,NaN,2023-08-31,NaN,2023-09-30
56026,479.73,9.0,已逾期支付,2024-04-03,2024-04-05,NaN,2023-08-31,NaN,2023-09-30
56027,479.73,10.0,已逾期支付,2024-05-03,2024-05-07,NaN,2023-08-31,NaN,2023-09-30


In [ ]:
# xuzu = pd.read_excel('xuzu.xlsx')
# xuzu.shape

# df_xz_concat.is_relet.value_counts()

# tmp = df_xz_concat.drop_duplicates(subset=['order_number','is_relet'])[['order_number','is_relet']]
# tmp.shape

# tmp1 = xuzu.merge(tmp, on = 'order_number', how = 'left')
# tmp1.is_relet.value_counts()
# tmp1
# A20230731183935113
# A2023073119383493

In [ ]:
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number,tprm.search_time,tprm.unrecovered_cost,tprm.overdue_type,tprm.purchase_amount
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where tprm.order_number in ('A20230731183935113','A2023073119383493')
# ;
# '''
# query(sql)

In [ ]:
# df_mob_total[df_mob_total.order_number=='A2024092919033313']

In [ ]:
# df_xz_concat[df_xz_concat.order_number == 'A20241009103203160'][['应付日期','实付日期','实付日期new','当前期数','逾期天数','状态']]
# df_xz_concat[df_xz_concat.order_number == 'A20241104144812228'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape


# 正常，用了优惠劵还款，没有逾期
# A20241009103203160

# 10.7-10.13 下单的下面这几单FPD7+分子
# A20241009183740113
# A2024101011183869
# A2024101015064941
# A202410111101329
# A2024101321532294

# df_xz_concat[df_xz_concat.逾期天数 > 30][['order_number']].tail()
#逾期客户 逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313

# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228

In [ ]:
# vintage_money_d4 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# # vintage_money_d4['采购金额'] = df_mon['采购金额']
# # vintage_money_d4
# vintage_divide_money_d4 = vintage_money_d4.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d4['采购金额'] = df_mon2["purchase_amount"]
# vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [ ]:
# # df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道!='抖音渠道') & (df_mob_total.下单月份 == '2023-12')].['order_number'].nunique()
# df_mob_total[(df_mob_total.归属渠道!='抖音渠道') & (df_mob_total.下单月份=='2023-07')].to_excel('tmp.xlsx', index = False)

In [ ]:
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number,tprm.search_time,tprm.unrecovered_cost,tprm.overdue_type,tprm.purchase_amount
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where tprm.search_time = '2023-07'
# ;
# '''

# query(sql).to_excel('tmp1.xlsx', index = False)
# # (27787, 62)

In [ ]:
# df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']
# df_mob_total[df_mob_total.order_number == 'A202307011100232'][['purchase_amount','paid_money_mob']]

# df_mob_total[df_mob_total.order_number == 'A202307011100232']

In [ ]:
# from pandas.tseries.offsets import MonthEnd
# def mob_date(df):
#     df['下单日期'] = pd.to_datetime(df['下单日期'])
#     df['实付日期new'] = pd.to_datetime(df['实付日期new'])
#     # note n 续租的没有统计
#     for n in range(0,13):  
#         df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
#         df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
#         df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
#         df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
#         df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
#     return df

# df_xz_concat = mob_date(df_xz_concat)

In [ ]:
# df_xz_concat[df_xz_concat.order_number == 'A202307011100232']

In [ ]:
df_xz_concat[(df_xz_concat.search_time == '2024-11') & (df_xz_concat.逾期天数>0)][['order_id','order_number','归属渠道','search_time','状态',
   '应付日期','实付日期','实付日期new','逾期天数','max_overdue_days']]

,order_id,order_number,归属渠道,search_time,状态,应付日期,实付日期,实付日期new,逾期天数,max_overdue_days
322592,1670688,A2024110113220670,八派信息,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322604,1670767,A20241101134208149,搜索渠道,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322796,1671376,A20241101152533170,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322844,1671696,A20241101162057121,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323060,1672503,A20241101195135116,八派信息,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323156,1672926,A2024110123471583,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323971,1677693,A20241103112643124,搜索渠道,2024-11,已逾期支付,2024-11-06,NaT,2024-11-08,2,0
324247,1678631,A20241103164306128,八派信息,2024-11,已逾期支付,2024-11-06,NaT,2024-11-10,4,0
324307,1679028,A2024110319324271,芝麻租物,2024-11,已逾期支付,2024-11-06,NaT,2024-11-11,5,0
324391,1679420,A2024110323515469,芝麻租物,2024-11,已逾期支付,2024-11-06,NaT,2024-11-10,4,0


In [ ]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'paid_money_mob9', 'mob_date_10', 'paid_date_mob10', 'paid_money_mob10',
       'mob_date_11', 'paid_date_mob11', 'paid_money_mob11', 'mob_date_12',
       'paid_date_mob12', 'paid_money_mob12'],
      dtype='object', length=158)

In [ ]:
# dfzh3[(dfzh3["归属渠道"]=="抖音渠道") & (dfzh3.search_time == '2024-11')]
# 7629.0	- 6947.0
# [['order_number','下单日期']]
# 押金
sql = '''-- 租后应收监控  
SELECT  order_id,residue_money
from db_digua_business.t_order_pledge
;
'''
tmp3 = query(sql)
tmp3.shape

tmp = dfzh3[(dfzh3["归属渠道"]=="抖音渠道") & (dfzh3.search_time.isin(['2024-06','2024-07','2024-08','2024-09','2024-11']))][['order_number','order_id','下单日期', '还款状态','逾期类型','未回收成本','采购金额']]
# dfzh3[(dfzh3["归属渠道"]=="抖音渠道") & (dfzh3.search_time.isin(['2024-06','2024-07','2024-08','2024-09']))][['order_number','下单日期', '还款状态','逾期类型','未回收成本','采购金额','all_deposit']][:2]

tmp1 = df_xz_concat[df_xz_concat.当前期数 == 1][['order_number','实付金额']]
tmp2 = tmp.merge(tmp1, on = 'order_number', how = 'left')
tmp2 = tmp2.merge(tmp3, on = 'order_id', how = 'left')
tmp2.to_excel('tmp2.xlsx', index = False)
# dfzh3[(dfzh3["归属渠道"]=="抖音渠道") & (dfzh3.search_time.isin(['2024-06','2024-07','2024-08','2024-09']))][['order_number','下单日期', '还款状态','逾期类型','未回收成本','采购金额']].to_excel('tmp.xlsx', index = False)

(3003, 2)

In [ ]:
df_xz_concat[df_xz_concat.当前期数 == 1][['order_number','实付金额']].shape
# df_xz_concat[df_xz_concat.order_number == 'A2024110418235587'].to_excel('tmp1.xlsx', index = False)

(31746, 2)

In [ ]:
df[(df.归属渠道 == '广州易合顺')&(df.下单月份 == '2025-01')]['当前逾期1+'].value_counts()
df[(df.归属渠道 == '广州易合顺')&(df.下单月份 == '2025-01')&(df['当前逾期1+'] == '逾期')][['order_number','now_overdue_days']]

# df = pd.merge(dfzh3,df_yq_total,on="order_id",how="left")
df_xz_concat[df_xz_concat.order_number == 'A20250109123651303'][['order_number','逾期天数']]
dfzh3[dfzh3.order_number == 'A20250109123651303'][['order_number','now_overdue_days']][:2]
dfzh2[dfzh2.order_number == 'A20250109123651303'][['order_number','now_overdue_days']][:2]
dfzh1[dfzh1.order_number == 'A20250109123651303'][['order_number','now_overdue_days']][:2]
df_zhys[df_zhys.order_number == 'A20250109123651303'][['order_number','now_overdue_days']][:2]

当前逾期1+
正常         84
逾期         15
未到首期还款日    10
Name: count, dtype: int64

,order_number,now_overdue_days
30236,A20250106121548126,6
30342,A20250107113304219,6
30345,A20250107114228299,6
30394,A20250107175228339,6
30402,A20250107193914197,6
30403,A20250107194634235,6
30537,A20250109123651303,6
30538,A20250109125127410,6
30669,A2025011016044328,6
30670,A2025011016094873,6


,order_number,逾期天数
407441,A20250109123651303,0
407442,A20250109123651303,0
407443,A20250109123651303,0
407444,A20250109123651303,0
407445,A20250109123651303,0
407446,A20250109123651303,0
407447,A20250109123651303,0
407448,A20250109123651303,0
407449,A20250109123651303,0
407450,A20250109123651303,0


,order_number,now_overdue_days
30537,A20250109123651303,6


,order_number,now_overdue_days
30537,A20250109123651303,6


,order_number,now_overdue_days
30537,A20250109123651303,6


,order_number,now_overdue_days
32517,A20250109123651303,6


In [ ]:
# note 勿删
vintage：分母固定，分子为对应mob月末的逾期情况
FPDn+：分母分子长度一样，分子为出现过逾期情况
滚动率计算用 overdue_days_mob
历史最大逾期天数用 逾期天数
# note 20241211 SPD TPD QPD 判断逻辑有问题，还没修正
# note 联通卡号的订单，还款日固定为每月15号，逾期天数未更新

FSTQ 对订单状态无判断，如已经取消的订单，也在分母
计算逾期，没有考虑归还逾期（vintage没有体现）

overdue_days_mob：月末最大逾期天数，减数为月末，被减数为开始逾期日期
逾期天数：本期逾期天数，减数为实还日期或系统当前日期，被减数为本期应还日期



# 查看12月的状态  A20241103112643124 A20241103164306128
# df_xz_concat[(df_xz_concat.search_time == '2024-11') & (df_xz_concat.逾期天数>0)][['order_id','order_number','归属渠道','search_time','状态',
#    '应付日期','实付日期','实付日期new','逾期天数','max_overdue_days']]

,order_id,order_number,归属渠道,search_time,状态,应付日期,实付日期,实付日期new,逾期天数,max_overdue_days
322592,1670688,A2024110113220670,八派信息,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322604,1670767,A20241101134208149,搜索渠道,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322796,1671376,A20241101152533170,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
322844,1671696,A20241101162057121,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323060,1672503,A20241101195135116,八派信息,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323156,1672926,A2024110123471583,芝麻租物,2024-11,已逾期,2024-12-04,NaT,NaT,1,1
323971,1677693,A20241103112643124,搜索渠道,2024-11,已逾期支付,2024-11-06,NaT,2024-11-08,2,0
324247,1678631,A20241103164306128,八派信息,2024-11,已逾期支付,2024-11-06,NaT,2024-11-10,4,0
324307,1679028,A2024110319324271,芝麻租物,2024-11,已逾期支付,2024-11-06,NaT,2024-11-11,5,0
324391,1679420,A2024110323515469,芝麻租物,2024-11,已逾期支付,2024-11-06,NaT,2024-11-10,4,0
